In [2]:
from peft import PeftConfig, PeftModel, PeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "4,7"

adapter_model_name = "/vepfs/wcf/G/zecheng/ckpt/simpo_llama_10k_random_4/checkpoint-800" 
# base_model_name = "/vepfs/wcf/G/zecheng/ckpt/llama-2-7b-80k-sft-low_rank-random-sft-2/checkpoint-200"
base_model_name = "/vepfs/wcf/G/zecheng/hf_models/Llama-2-7b-hf"
config = AutoConfig.from_pretrained("/vepfs/wcf/G/zecheng/hf_models/Llama-2-7b-hf")
config.rope_scaling = {"type": "linear", "factor": 16}
config.rope_theta = 10000
config.max_position_embeddings = 4096

model = AutoModelForCausalLM.from_pretrained(base_model_name, config=config, use_flash_attention_2="flash_attention_2", torch_dtype="auto", device_map="auto")
model = PeftModelForCausalLM.from_pretrained(model, adapter_model_name, torch_dtype="auto")
# base_model = AutoModelForCausalLM.from_pretrained(base_model_name, config=config)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.98s/it]
/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  

In [3]:
Long_Context_50K = """<s> 
The best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.
<s><s>  

Want to start a startup?  Get funded by
Y Combinator.




July 2004(This essay is derived from a talk at Oscon 2004.)
A few months ago I finished a new 
book, 
and in reviews I keep
noticing words like "provocative'' and "controversial.'' To say
nothing of "idiotic.''I didn't mean to make the book controversial.  I was trying to make
it efficient.  I didn't want to waste people's time telling them
things they already knew.  It's more efficient just to give them
the diffs.  But I suppose that's bound to yield an alarming book.EdisonsThere's no controversy about which idea is most controversial:
the suggestion that variation in wealth might not be as big a
problem as we think.I didn't say in the book that variation in wealth was in itself a
good thing.  I said in some situations it might be a sign of good
things.  A throbbing headache is not a good thing, but it can be
a sign of a good thing-- for example, that you're recovering
consciousness after being hit on the head.Variation in wealth can be a sign of variation in productivity.
(In a society of one, they're identical.) And that
is almost certainly a good thing: if your society has no variation
in productivity, it's probably not because everyone is Thomas
Edison.  It's probably because you have no Thomas Edisons.In a low-tech society you don't see much variation in productivity.
If you have a tribe of nomads collecting sticks for a fire, how
much more productive is the best stick gatherer going to be than
the worst?  A factor of two?  Whereas when you hand people a complex tool
like a computer, the variation in what they can do with
it is enormous.That's not a new idea.  Fred Brooks wrote about it in 1974, and
the study he quoted was published in 1968.  But I think he
underestimated the variation between programmers.  He wrote about productivity in lines
of code:  the best programmers can solve a given problem in a tenth
the time.  But what if the problem isn't given? In programming, as
in many fields, the hard part isn't solving problems, but deciding
what problems to solve.  Imagination is hard to measure, but
in practice it dominates the kind of productivity that's measured
in lines of code.Productivity varies in any field, but there are few in which it
varies so much.  The variation between programmers
is so great that it becomes a difference in kind.  I don't
think this is something intrinsic to programming, though.  In every field,
technology magnifies differences in productivity.  I think what's
happening in programming is just that we have a lot of technological
leverage.  But in every field the lever is getting longer, so the
variation we see is something that more and more fields will see
as time goes on.  And the success of companies, and countries, will
depend increasingly on how they deal with it.If variation in productivity increases with technology, then the
contribution of the most productive individuals will not only be
disproportionately large, but will actually grow with time.  When
you reach the point where 90% of a group's output is created by 1%
of its members, you lose big if something (whether Viking raids,
or central planning) drags their productivity down to the average.If we want to get the most out of them, we need to understand these
especially productive people.  What motivates them?  What do they
need to do their jobs?  How do you recognize them? How do you
get them to come and work for you?  And then of course there's the
question, how do you become one?More than MoneyI know a handful of super-hackers, so I sat down and thought about
what they have in common.  Their defining quality is probably that
they really love to program.  Ordinary programmers write code to pay
the bills.  Great hackers think of it as something they do for fun,
and which they're delighted to find people will pay them for.Great programmers are sometimes said to be indifferent to money.
This isn't quite true.  It is true that all they really care about
is doing interesting work.  But if you make enough money, you get
to work on whatever you want, and for that reason hackers are
attracted by the idea of making really large amounts of money.
But as long as they still have to show up for work every day, they
care more about what they do there than how much they get paid for
it.Economically, this is a fact of the greatest importance, because
it means you don't have to pay great hackers anything like what
they're worth.  A great programmer might be ten or a hundred times
as productive as an ordinary one, but he'll consider himself lucky
to get paid three times as much.  As I'll explain later, this is
partly because great hackers don't know how good they are.  But
it's also because money is not the main thing they want.What do hackers want?  Like all craftsmen, hackers like good tools.
In fact, that's an understatement.  Good hackers find it unbearable
to use bad tools.  They'll simply refuse to work on projects with
the wrong infrastructure.At a startup I once worked for, one of the things pinned up on our
bulletin board was an ad from IBM.  It was a picture of an AS400,
and the headline read, I think, "hackers despise
it.'' [1]When you decide what infrastructure to use for a project, you're
not just making a technical decision.  You're also making a social
decision, and this may be the more important of the two.  For
example, if your company wants to write some software, it might
seem a prudent choice to write it in Java.  But when you choose a
language, you're also choosing a community.  The programmers you'll
be able to hire to work on a Java project won't be as
smart as the
ones you could get to work on a project written in Python.
And the quality of your hackers probably matters more than the
language you choose.  Though, frankly, the fact that good hackers
prefer Python to Java should tell you something about the relative
merits of those languages.Business types prefer the most popular languages because they view
languages as standards. They don't want to bet the company on
Betamax.  The thing about languages, though, is that they're not
just standards.  If you have to move bits over a network, by all
means use TCP/IP.  But a programming language isn't just a format.
A programming language is a medium of expression.I've read that Java has just overtaken Cobol as the most popular
language.  As a standard, you couldn't wish for more.  But as a
medium of expression, you could do a lot better.  Of all the great
programmers I can think of, I know of only one who would voluntarily
program in Java.  And of all the great programmers I can think of
who don't work for Sun, on Java, I know of zero.Great hackers also generally insist on using open source software.
Not just because it's better, but because it gives them more control.
Good hackers insist on control.  This is part of what makes them
good hackers:  when something's broken, they need to fix it.  You
want them to feel this way about the software they're writing for
you.  You shouldn't be surprised when they feel the same way about
the operating system.A couple years ago a venture capitalist friend told me about a new
startup he was involved with.  It sounded promising.  But the next
time I talked to him, he said they'd decided to build their software
on Windows NT, and had just hired a very experienced NT developer
to be their chief technical officer.  When I heard this, I thought,
these guys are doomed.  One, the CTO couldn't be a first rate
hacker, because to become an eminent NT developer he would have
had to use NT voluntarily, multiple times, and I couldn't imagine
a great hacker doing that; and two, even if he was good, he'd have
a hard time hiring anyone good to work for him if the project had
to be built on NT. [2]The Final FrontierAfter software, the most important tool to a hacker is probably
his office.  Big companies think the function of office space is to express
rank.  But hackers use their offices for more than that: they
use their office as a place to think in.  And if you're a technology
company, their thoughts are your product.  So making hackers work
in a noisy, distracting environment is like having a paint factory
where the air is full of soot.The cartoon strip Dilbert has a lot to say about cubicles, and with
good reason.  All the hackers I know despise them.  The mere prospect
of being interrupted is enough to prevent hackers from working on
hard problems.  If you want to get real work done in an office with
cubicles, you have two options: work at home, or come in early or
late or on a weekend, when no one else is there.  Don't companies
realize this is a sign that something is broken?  An office
environment is supposed to be something that helps
you work, not something you work despite.Companies like Cisco are proud that everyone there has a cubicle,
even the CEO.  But they're not so advanced as they think; obviously
they still view office space as a badge of rank.  Note too that
Cisco is famous for doing very little product development in house.
They get new technology by buying the startups that created it-- where
presumably the hackers did have somewhere quiet to work.One big company that understands what hackers need is Microsoft.
I once saw a recruiting ad for Microsoft with a big picture of a
door.  Work for us, the premise was, and we'll give you a place to
work where you can actually get work done.   And you know, Microsoft
is remarkable among big companies in that they are able to develop
software in house.  Not well, perhaps, but well enough.If companies want hackers to be productive, they should look at
what they do at home.  At home, hackers can arrange things themselves
so they can get the most done.  And when they work at home, hackers
don't work in noisy, open spaces; they work in rooms with doors.  They
work in cosy, neighborhoody places with people around and somewhere
to walk when they need to mull something over, instead of in glass
boxes set in acres of parking lots.  They have a sofa they can take
a nap on when they feel tired, instead of sitting in a coma at
their desk, pretending to work.  There's no crew of people with
vacuum cleaners that roars through every evening during the prime
hacking hours.  There are no meetings or, God forbid, corporate
retreats or team-building exercises.  And when you look at what
they're doing on that computer, you'll find it reinforces what I
said earlier about tools.  They may have to use Java and Windows
at work, but at home, where they can choose for themselves, you're
more likely to find them using Perl and Linux.Indeed, these statistics about Cobol or Java being the most popular
language can be misleading.  What we ought to look at, if we want
to know what tools are best, is what hackers choose when they can
choose freely-- that is, in projects of their own.  When you ask
that question, you find that open source operating systems already
have a dominant market share, and the number one language is probably
Perl.InterestingAlong with good tools, hackers want interesting projects.  What
makes a project interesting?  Well, obviously overtly sexy
applications like stealth planes or special effects software would
be interesting to work on.  But any application can be interesting
if it poses novel technical challenges.  So it's hard to predict
which problems hackers will like, because some become
interesting only when the people working on them discover a new
kind of solution.  Before ITA
(who wrote the software inside Orbitz),
the people working on airline fare searches probably thought it
was one of the most boring applications imaginable.  But ITA made
it interesting by 
redefining the problem in a more ambitious way.I think the same thing happened at Google.  When Google was founded,
the conventional wisdom among the so-called portals was that search
was boring and unimportant.  But the guys at Google didn't think
search was boring, and that's why they do it so well.This is an area where managers can make a difference.  Like a parent
saying to a child, I bet you can't clean up your whole room in
ten minutes, a good manager can sometimes redefine a problem as a
more interesting one.  Steve Jobs seems to be particularly good at
this, in part simply by having high standards.  There were a lot
of small, inexpensive computers before the Mac.  He redefined the
problem as: make one that's beautiful.  And that probably drove
the developers harder than any carrot or stick could.They certainly delivered.  When the Mac first appeared, you didn't
even have to turn it on to know it would be good; you could tell
from the case.  A few weeks ago I was walking along the street in
Cambridge, and in someone's trash I saw what appeared to be a Mac
carrying case.  I looked inside, and there was a Mac SE.  I carried
it home and plugged it in, and it booted.  The happy Macintosh
face, and then the finder.  My God, it was so simple.  It was just
like ... Google.Hackers like to work for people with high standards.  But it's not
enough just to be exacting.  You have to insist on the right things.
Which usually means that you have to be a hacker yourself.  I've
seen occasional articles about how to manage programmers.  Really
there should be two articles: one about what to do if
you are yourself a programmer, and one about what to do if you're not.  And the 
second could probably be condensed into two words:  give up.The problem is not so much the day to day management.  Really good
hackers are practically self-managing.  The problem is, if you're
not a hacker, you can't tell who the good hackers are.  A similar
problem explains why American cars are so ugly.  I call it the
design paradox.  You might think that you could make your products
beautiful just by hiring a great designer to design them.  But if
you yourself don't have good taste, 
how are you going to recognize
a good designer?  By definition you can't tell from his portfolio.
And you can't go by the awards he's won or the jobs he's had,
because in design, as in most fields, those tend to be driven by
fashion and schmoozing, with actual ability a distant third.
There's no way around it:  you can't manage a process intended to
produce beautiful things without knowing what beautiful is.  American
cars are ugly because American car companies are run by people with
bad taste.Many people in this country think of taste as something elusive,
or even frivolous.  It is neither.  To drive design, a manager must
be the most demanding user of a company's products.  And if you
have really good taste, you can, as Steve Jobs does, make satisfying
you the kind of problem that good people like to work on.Nasty Little ProblemsIt's pretty easy to say what kinds of problems are not interesting:
those where instead of solving a few big, clear, problems, you have
to solve a lot of nasty little ones.  One of the worst kinds of
projects is writing an interface to a piece of software that's
full of bugs.  Another is when you have to customize
something for an individual client's complex and ill-defined needs.
To hackers these kinds of projects are the death of a thousand
cuts.The distinguishing feature of nasty little problems is that you
don't learn anything from them.   Writing a compiler is interesting
because it teaches you what a compiler is.  But writing an interface
to a buggy piece of software doesn't teach you anything, because the
bugs are random.  [3] So it's not just fastidiousness that makes good
hackers avoid nasty little problems.  It's more a question of
self-preservation.  Working on nasty little problems makes you
stupid.  Good hackers avoid it for the same reason models avoid
cheeseburgers.Of course some problems inherently have this character.  And because
of supply and demand, they pay especially well.  So a company that
found a way to get great hackers to work on tedious problems would
be very successful.  How would you do it?One place this happens is in startups.  At our startup we had 
Robert Morris working as a system administrator.  That's like having the
Rolling Stones play at a bar mitzvah.  You can't hire that kind of
talent.  But people will do any amount of drudgery for companies
of which they're the founders.  [4]Bigger companies solve the problem by partitioning the company.
They get smart people to work for them by establishing a separate
R&D department where employees don't have to work directly on
customers' nasty little problems. [5] In this model, the research
department functions like a mine. They produce new ideas; maybe
the rest of the company will be able to use them.You may not have to go to this extreme.  
Bottom-up programming
suggests another way to partition the company: have the smart people
work as toolmakers.  If your company makes software to do x, have
one group that builds tools for writing software of that type, and
another that uses these tools to write the applications.  This way
you might be able to get smart people to write 99% of your code,
but still keep them almost as insulated from users as they would
be in a traditional research department.  The toolmakers would have
users, but they'd only be the company's own developers.  [6]If Microsoft used this approach, their software wouldn't be so full
of security holes, because the less smart people writing the actual
applications wouldn't be doing low-level stuff like allocating
memory.  Instead of writing Word directly in C, they'd be plugging
together big Lego blocks of Word-language.  (Duplo, I believe, is
the technical term.)ClumpingAlong with interesting problems, what good hackers like is other
good hackers.  Great hackers tend to clump together-- sometimes
spectacularly so, as at Xerox Parc.   So you won't attract good
hackers in linear proportion to how good an environment you create
for them.  The tendency to clump means it's more like the square
of the environment.  So it's winner take all.  At any given time,
there are only about ten or twenty places where hackers most want to
work, and if you aren't one of them, you won't just have fewer
great hackers, you'll have zero.Having great hackers is not, by itself, enough to make a company
successful.  It works well for Google and ITA, which are two of
the hot spots right now, but it didn't help Thinking Machines or
Xerox.  Sun had a good run for a while, but their business model
is a down elevator.  In that situation, even the best hackers can't
save you.I think, though, that all other things being equal, a company that
can attract great hackers will have a huge advantage.  There are
people who would disagree with this.  When we were making the rounds
of venture capital firms in the 1990s, several told us that software
companies didn't win by writing great software, but through brand,
and dominating channels, and doing the right deals.They really seemed to believe this, and I think I know why.  I
think what a lot of VCs are looking for, at least unconsciously,
is the next Microsoft.  And of course if Microsoft is your model,
you shouldn't be looking for companies that hope to win by writing
great software.  But VCs are mistaken to look for the next Microsoft,
because no startup can be the next Microsoft unless some other
company is prepared to bend over at just the right moment and be
the next IBM.It's a mistake to use Microsoft as a model, because their whole
culture derives from that one lucky break.  Microsoft is a bad data
point.  If you throw them out, you find that good products do tend
to win in the market.  What VCs should be looking for is the next
Apple, or the next Google.I think Bill Gates knows this.  What worries him about Google is
not the power of their brand, but the fact that they have
better hackers. [7]
RecognitionSo who are the great hackers?  How do you know when you meet one?
That turns out to be very hard.  Even hackers can't tell.  I'm
pretty sure now that my friend Trevor Blackwell is a great hacker.
You may have read on Slashdot how he made his 
own Segway.  The
remarkable thing about this project was that he wrote all the
software in one day (in Python, incidentally).For Trevor, that's
par for the course.  But when I first met him, I thought he was a
complete idiot.  He was standing in Robert Morris's office babbling
at him about something or other, and I remember standing behind
him making frantic gestures at Robert to shoo this nut out of his
office so we could go to lunch.  Robert says he misjudged Trevor
at first too.  Apparently when Robert first met him, Trevor had
just begun a new scheme that involved writing down everything about
every aspect of his life on a stack of index cards, which he carried
with him everywhere.  He'd also just arrived from Canada, and had
a strong Canadian accent and a mullet.The problem is compounded by the fact that hackers, despite their
reputation for social obliviousness, sometimes put a good deal of
effort into seeming smart.  When I was in grad school I used to
hang around the MIT AI Lab occasionally. It was kind of intimidating
at first.  Everyone there spoke so fast.  But after a while I
learned the trick of speaking fast.  You don't have to think any
faster; just use twice as many words to say everything.  With this amount of noise in the signal, it's hard to tell good
hackers when you meet them.  I can't tell, even now.  You also
can't tell from their resumes.  It seems like the only way to judge
a hacker is to work with him on something.And this is the reason that high-tech areas 
only happen around universities.  The active ingredient
here is not so much the professors as the students.  Startups grow up
around universities because universities bring together promising young
people and make them work on the same projects.  The
smart ones learn who the other smart ones are, and together
they cook up new projects of their own.Because you can't tell a great hacker except by working with him,
hackers themselves can't tell how good they are.  This is true to
a degree in most fields.  I've found that people who
are great at something are not so much convinced of their own
greatness as mystified at why everyone else seems so incompetent.
But it's particularly hard for hackers to know how good they are,
because it's hard to compare their work.  This is easier in most
other fields.  In the hundred meters, you know in 10 seconds who's
fastest.  Even in math there seems to be a general consensus about
which problems are hard to solve, and what constitutes a good
solution.  But hacking is like writing.  Who can say which of two
novels is better?  Certainly not the authors.With hackers, at least, other hackers can tell.  That's because,
unlike novelists, hackers collaborate on projects.  When you get
to hit a few difficult problems over the net at someone, you learn
pretty quickly how hard they hit them back.  But hackers can't
watch themselves at work.  So if you ask a great hacker how good
he is, he's almost certain to reply, I don't know.  He's not just
being modest.  He really doesn't know.And none of us know, except about people we've actually worked
with.  Which puts us in a weird situation: we don't know who our
heroes should be.  The hackers who become famous tend to become
famous by random accidents of PR.  Occasionally I need to give an
example of a great hacker, and I never know who to use.  The first
names that come to mind always tend to be people I know personally,
but it seems lame to use them.  So, I think, maybe I should say
Richard Stallman, or Linus Torvalds, or Alan Kay, or someone famous
like that.  But I have no idea if these guys are great hackers.
I've never worked with them on anything.If there is a Michael Jordan of hacking, no one knows, including
him.CultivationFinally, the question the hackers have all been wondering about:
how do you become a great hacker?  I don't know if it's possible
to make yourself into one.  But it's certainly possible to do things
that make you stupid, and if you can make yourself stupid, you
can probably make yourself smart too.The key to being a good hacker may be to work on what you like.
When I think about the great hackers I know, one thing they have
in common is the extreme 
difficulty of making them work 
on anything they
don't want to.  I don't know if this is cause or effect; it may be
both.To do something well you have to love it.  
So to the extent you
can preserve hacking as something you love, you're likely to do it
well.  Try to keep the sense of wonder you had about programming at
age 14.  If you're worried that your current job is rotting your
brain, it probably is.The best hackers tend to be smart, of course, but that's true in
a lot of fields.  Is there some quality that's unique to hackers?
I asked some friends, and the number one thing they mentioned was
curiosity.  
I'd always supposed that all smart people were curious--
that curiosity was simply the first derivative of knowledge.  But
apparently hackers are particularly curious, especially about how
things work.  That makes sense, because programs are in effect
giant descriptions of how things work.Several friends mentioned hackers' ability to concentrate-- their
ability, as one put it, to "tune out everything outside their own
heads.''  I've certainly noticed this.  And I've heard several 
hackers say that after drinking even half a beer they can't program at
all.   So maybe hacking does require some special ability to focus.
Perhaps great hackers can load a large amount of context into their
head, so that when they look at a line of code, they see not just
that line but the whole program around it.  John McPhee
wrote that Bill Bradley's success as a basketball player was due
partly to his extraordinary peripheral vision.  "Perfect'' eyesight
means about 47 degrees of vertical peripheral vision.  Bill Bradley
had 70; he could see the basket when he was looking at the floor.
Maybe great hackers have some similar inborn ability.  (I cheat by
using a very dense language, 
which shrinks the court.)This could explain the disconnect over cubicles.  Maybe the people
in charge of facilities, not having any concentration to shatter,
have no idea that working in a cubicle feels to a hacker like having
one's brain in a blender.  (Whereas Bill, if the rumors of autism
are true, knows all too well.)One difference I've noticed between great hackers and smart people
in general is that hackers are more 
politically incorrect.  To the
extent there is a secret handshake among good hackers, it's when they
know one another well enough to express opinions that would get
them stoned to death by the general public.  And I can see why
political incorrectness would be a useful quality in programming.
Programs are very complex and, at least in the hands of good
programmers, very fluid.  In such situations it's helpful to have
a habit of questioning assumptions.Can you cultivate these qualities?  I don't know.  But you can at
least not repress them.  So here is my best shot at a recipe.  If
it is possible to make yourself into a great hacker, the way to do
it may be to make the following deal with yourself: you never have
to work on boring projects (unless your family will starve otherwise),
and in return, you'll never allow yourself to do a half-assed job.
All the great hackers I know seem to have made that deal, though
perhaps none of them had any choice in the matter.Notes
[1] In fairness, I have to say that IBM makes decent hardware.  I
wrote this on an IBM laptop.[2] They did turn out to be doomed.  They shut down a few months
later.[3] I think this is what people mean when they talk
about the "meaning of life."  On the face of it, this seems an 
odd idea.  Life isn't an expression; how could it have meaning?
But it can have a quality that feels a lot like meaning.  In a project
like a compiler, you have to solve a lot of problems, but the problems
all fall into a pattern, as in a signal.  Whereas when the problems
you have to solve are random, they seem like noise.
[4] Einstein at one point worked designing refrigerators. (He had equity.)[5] It's hard to say exactly what constitutes research in the
computer world, but as a first approximation, it's software that
doesn't have users.I don't think it's publication that makes the best hackers want to work
in research departments.  I think it's mainly not having to have a
three hour meeting with a product manager about problems integrating
the Korean version of Word 13.27 with the talking paperclip.[6] Something similar has been happening for a long time in the
construction industry. When you had a house built a couple hundred
years ago, the local builders built everything in it.  But increasingly
what builders do is assemble components designed and manufactured
by someone else.  This has, like the arrival of desktop publishing,
given people the freedom to experiment in disastrous ways, but it
is certainly more efficient.[7] Google is much more dangerous to Microsoft than Netscape was.
Probably more dangerous than any other company has ever been.  Not
least because they're determined to fight.  On their job listing
page, they say that one of their "core values'' is "Don't be evil.''
From a company selling soybean oil or mining equipment, such a
statement would merely be eccentric.  But I think all of us in the
computer world recognize who that is a declaration of war on.Thanks to Jessica Livingston, Robert Morris, and Sarah Harlin
for reading earlier versions of this talk.

Want to start a startup?  Get funded by
Y Combinator.




January 2006To do something well you have to like it.   That idea is not exactly
novel.  We've got it down to four words: "Do what you love."  But
it's not enough just to tell people that.  Doing what you love is
complicated.The very idea is foreign to what most of us learn as kids.  When I
was a kid, it seemed as if work and fun were opposites by definition.
Life had two states: some of the time adults were making you do
things, and that was called work; the rest of the time you could
do what you wanted, and that was called playing.  Occasionally the
things adults made you do were fun, just as, occasionally, playing
wasn't—for example, if you fell and hurt yourself.  But except
for these few anomalous cases, work was pretty much defined as
not-fun.And it did not seem to be an accident. School, it was implied, was
tedious because it was preparation for grownup work.The world then was divided into two groups, grownups and kids.
Grownups, like some kind of cursed race, had to work.  Kids didn't,
but they did have to go to school, which was a dilute version of
work meant to prepare us for the real thing.  Much as we disliked
school, the grownups all agreed that grownup work was worse, and
that we had it easy.Teachers in particular all seemed to believe implicitly that work
was not fun.  Which is not surprising: work wasn't fun for most of
them.  Why did we have to memorize state capitals instead of playing
dodgeball?  For the same reason they had to watch over a bunch of
kids instead of lying on a beach.  You couldn't just do what you
wanted.I'm not saying we should let little kids do whatever they want.
They may have to be made to work on certain things.  But if we make
kids work on dull stuff, it might be wise to tell them that tediousness
is not the defining quality of work, and indeed that the reason
they have to work on dull stuff now is so they can work on more
interesting stuff later.
[1]Once, when I was about 9 or 10, my father told me I could be whatever
I wanted when I grew up, so long as I enjoyed it.  I remember that
precisely because it seemed so anomalous.  It was like being told
to use dry water.  Whatever I thought he meant, I didn't think he
meant work could literally be fun—fun like playing.  It
took me years to grasp that.JobsBy high school, the prospect of an actual job was on the horizon.
Adults would sometimes come to speak to us about their work, or we
would go to see them at work.  It was always understood that they
enjoyed what they did.  In retrospect I think one may have: the
private jet pilot.  But I don't think the bank manager really did.The main reason they all acted as if they enjoyed their work was
presumably the upper-middle class convention that you're supposed
to.  It would not merely be bad for your career to say that you
despised your job, but a social faux-pas.Why is it conventional to pretend to like what you do?  The first
sentence of this essay explains that.  If you have to like something
to do it well, then the most successful people will all like what
they do.  That's where the upper-middle class tradition comes from.
Just as houses all over America are full of 
chairs
that are, without
the owners even knowing it, nth-degree imitations of chairs designed
250 years ago for French kings, conventional attitudes about work
are, without the owners even knowing it, nth-degree imitations of
the attitudes of people who've done great things.What a recipe for alienation.  By the time they reach an age to
think about what they'd like to do, most kids have been thoroughly
misled about the idea of loving one's work.  School has trained
them to regard work as an unpleasant duty.  Having a job is said
to be even more onerous than schoolwork.  And yet all the adults
claim to like what they do.  You can't blame kids for thinking "I
am not like these people; I am not suited to this world."Actually they've been told three lies: the stuff they've been taught
to regard as work in school is not real work; grownup work is not
(necessarily) worse than schoolwork; and many of the adults around
them are lying when they say they like what they do.The most dangerous liars can be the kids' own parents.  If you take
a boring job to give your family a high standard of living, as so
many people do, you risk infecting your kids with the idea that
work is boring. 
[2]
Maybe it would be better for kids in this one
case if parents were not so unselfish.  A parent who set an example
of loving their work might help their kids more than an expensive
house.
[3]It was not till I was in college that the idea of work finally broke
free from the idea of making a living.  Then the important question
became not how to make money, but what to work on.  Ideally these
coincided, but some spectacular boundary cases (like Einstein in
the patent office) proved they weren't identical.The definition of work was now to make some original contribution
to the world, and in the process not to starve.  But after the habit
of so many years my idea of work still included a large component
of pain.  Work still seemed to require discipline, because only
hard problems yielded grand results, and hard problems couldn't
literally be fun.   Surely one had to force oneself to work on them.If you think something's supposed to hurt, you're less likely to
notice if you're doing it wrong.  That about sums up my experience
of graduate school.BoundsHow much are you supposed to like what you do?  Unless you
know that, you don't know when to stop searching. And if, like most
people, you underestimate it, you'll tend to stop searching too
early.  You'll end up doing something chosen for you by your parents,
or the desire to make money, or prestige—or sheer inertia.Here's an upper bound: Do what you love doesn't mean, do what you
would like to do most this second.  Even Einstein probably
had moments when he wanted to have a cup of coffee, but told himself
he ought to finish what he was working on first.It used to perplex me when I read about people who liked what they
did so much that there was nothing they'd rather do.  There didn't
seem to be any sort of work I liked that much.  If I had a
choice of (a) spending the next hour working on something or (b)
be teleported to Rome and spend the next hour wandering about, was
there any sort of work I'd prefer?  Honestly, no.But the fact is, almost anyone would rather, at any given moment,
float about in the Carribbean, or have sex, or eat some delicious
food, than work on hard problems.  The rule about doing what you
love assumes a certain length of time.  It doesn't mean, do what
will make you happiest this second, but what will make you happiest
over some longer period, like a week or a month.Unproductive pleasures pall eventually.  After a while you get tired
of lying on the beach.  If you want to stay happy, you have to do
something.As a lower bound, you have to like your work more than any unproductive
pleasure.  You have to like what you do enough that the concept of
"spare time" seems mistaken.  Which is not to say you have to spend
all your time working.  You can only work so much before you get
tired and start to screw up.  Then you want to do something else—even something mindless.  But you don't regard this time as the
prize and the time you spend working as the pain you endure to earn
it.I put the lower bound there for practical reasons.  If your work
is not your favorite thing to do, you'll have terrible problems
with procrastination.  You'll have to force yourself to work,  and
when you resort to that the results are distinctly inferior.To be happy I think you have to be doing something you not only
enjoy, but admire.  You have to be able to say, at the end, wow,
that's pretty cool.  This doesn't mean you have to make something.
If you learn how to hang glide, or to speak a foreign language
fluently, that will be enough to make you say, for a while at least,
wow, that's pretty cool.  What there has to be is a test.So one thing that falls just short of the standard, I think, is
reading books.  Except for some books in math and the hard sciences,
there's no test of how well you've read a book, and that's why
merely reading books doesn't quite feel like work.  You have to do
something with what you've read to feel productive.I think the best test is one Gino Lee taught me: to try to do things
that would make your friends say wow.  But it probably wouldn't
start to work properly till about age 22, because most people haven't
had a big enough sample to pick friends from before then.SirensWhat you should not do, I think, is worry about the opinion of
anyone beyond your friends.  You shouldn't worry about prestige.
Prestige is the opinion of the rest of the world.  When you can ask
the opinions of people whose judgement you respect, what does it
add to consider the opinions of people you don't even know? 
[4]This is easy advice to give.  It's hard to follow, especially when
you're young.  
[5]
Prestige is like a powerful magnet that warps
even your beliefs about what you enjoy.  It causes you to work not
on what you like, but what you'd like to like.That's what leads people to try to write novels, for example.  They
like reading novels.  They notice that people who write them win
Nobel prizes.  What could be more wonderful, they think, than to
be a novelist?  But liking the idea of being a novelist is not
enough; you have to like the actual work of novel-writing if you're
going to be good at it; you have to like making up elaborate lies.Prestige is just fossilized inspiration.  If you do anything well
enough, you'll make it prestigious.  Plenty of things we now
consider prestigious were anything but at first.  Jazz comes to
mind—though almost any established art form would do.   So just
do what you like, and let prestige take care of itself.Prestige is especially dangerous to the ambitious.  If you want to
make ambitious people waste their time on errands, the way to do
it is to bait the hook with prestige.  That's the recipe for getting
people to give talks, write forewords, serve on committees, be
department heads, and so on.  It might be a good rule simply to
avoid any prestigious task. If it didn't suck, they wouldn't have
had to make it prestigious.Similarly, if you admire two kinds of work equally, but one is more
prestigious, you should probably choose the other.  Your opinions
about what's admirable are always going to be slightly influenced
by prestige, so if the two seem equal to you, you probably have
more genuine admiration for the less prestigious one.The other big force leading people astray is money.  Money by itself
is not that dangerous.  When something pays well but is regarded
with contempt, like telemarketing, or prostitution, or personal
injury litigation, ambitious people aren't tempted by it.  That
kind of work ends up being done by people who are "just trying to
make a living."  (Tip: avoid any field whose practitioners say
this.)  The danger is when money is combined with prestige, as in,
say, corporate law, or medicine.  A comparatively safe and prosperous
career with some automatic baseline prestige is dangerously tempting
to someone young, who hasn't thought much about what they really
like.The test of whether people love what they do is whether they'd do
it even if they weren't paid for it—even if they had to work at
another job to make a living.  How many corporate lawyers would do
their current work if they had to do it for free, in their spare
time, and take day jobs as waiters to support themselves?This test is especially helpful in deciding between different kinds
of academic work, because fields vary greatly in this respect.  Most
good mathematicians would work on math even if there were no jobs
as math professors, whereas in the departments at the other end of
the spectrum, the availability of teaching jobs is the driver:
people would rather be English professors than work in ad agencies,
and publishing papers is the way you compete for such jobs.  Math
would happen without math departments, but it is the existence of
English majors, and therefore jobs teaching them, that calls into
being all those thousands of dreary papers about gender and identity
in the novels of Conrad.  No one does 
that 
kind of thing for fun.The advice of parents will tend to err on the side of money.  It
seems safe to say there are more undergrads who want to be novelists
and whose parents want them to be doctors than who want to be doctors
and whose parents want them to be novelists.  The kids think their
parents are "materialistic." Not necessarily.  All parents tend to
be more conservative for their kids than they would for themselves,
simply because, as parents, they share risks more than rewards.  If
your eight year old son decides to climb a tall tree, or your teenage
daughter decides to date the local bad boy, you won't get a share
in the excitement, but if your son falls, or your daughter gets
pregnant, you'll have to deal with the consequences.DisciplineWith such powerful forces leading us astray, it's not surprising
we find it so hard to discover what we like to work on.  Most people
are doomed in childhood by accepting the axiom that work = pain.
Those who escape this are nearly all lured onto the rocks by prestige
or money.  How many even discover something they love to work on?
A few hundred thousand, perhaps, out of billions.It's hard to find work you love; it must be, if so few do.  So don't
underestimate this task.  And don't feel bad if you haven't succeeded
yet.  In fact, if you admit to yourself that you're discontented,
you're a step ahead of most people, who are still in denial.  If
you're surrounded by colleagues who claim to enjoy work that you
find contemptible, odds are they're lying to themselves.  Not
necessarily, but probably.Although doing great work takes less discipline than people think—because the way to do great work is to find something you like so
much that you don't have to force yourself to do it—finding
work you love does usually require discipline.   Some people are
lucky enough to know what they want to do when they're 12, and just
glide along as if they were on railroad tracks.  But this seems the
exception.  More often people who do great things have careers with
the trajectory of a ping-pong ball.  They go to school to study A,
drop out and get a job doing B, and then become famous for C after
taking it up on the side.Sometimes jumping from one sort of work to another is a sign of
energy, and sometimes it's a sign of laziness.  Are you dropping
out, or boldly carving a new path?  You often can't tell yourself.
Plenty of people who will later do great things seem to be disappointments
early on, when they're trying to find their niche.Is there some test you can use to keep yourself honest?  One is to
try to do a good job at whatever you're doing, even if you don't
like it.  Then at least you'll know you're not using dissatisfaction
as an excuse for being lazy.  Perhaps more importantly, you'll get
into the habit of doing things well.Another test you can use is: always produce.  For example, if you
have a day job you don't take seriously because you plan to be a
novelist, are you producing?  Are you writing pages of fiction,
however bad?  As long as you're producing, you'll know you're not
merely using the hazy vision of the grand novel you plan to write
one day as an opiate.  The view of it will be obstructed by the all
too palpably flawed one you're actually writing."Always produce" is also a heuristic for finding the work you love.
If you subject yourself to that constraint, it will automatically
push you away from things you think you're supposed to work on,
toward things you actually like.  "Always produce" will discover
your life's work the way water, with the aid of gravity, finds the
hole in your roof.Of course, figuring out what you like to work on doesn't mean you
get to work on it.  That's a separate question.  And if you're
ambitious you have to keep them separate: you have to make a conscious
effort to keep your ideas about what you want from being contaminated
by what seems possible. 
[6]It's painful to keep them apart, because it's painful to observe
the gap between them. So most people pre-emptively lower their
expectations.  For example, if you asked random people on the street
if they'd like to be able to draw like Leonardo, you'd find most
would say something like "Oh, I can't draw."  This is more a statement
of intention than fact; it means, I'm not going to try.  Because
the fact is, if you took a random person off the street and somehow
got them to work as hard as they possibly could at drawing for the
next twenty years, they'd get surprisingly far.  But it would require
a great moral effort; it would mean staring failure in the eye every
day for years.  And so to protect themselves people say "I can't."Another related line you often hear is that not everyone can do
work they love—that someone has to do the unpleasant jobs.  Really?
How do you make them?  In the US the only mechanism for forcing
people to do unpleasant jobs is the draft, and that hasn't been
invoked for over 30 years.  All we can do is encourage people to
do unpleasant work, with money and prestige.If there's something people still won't do, it seems as if society
just has to make do without.  That's what happened with domestic
servants.  For millennia that was the canonical example of a job
"someone had to do."  And yet in the mid twentieth century servants
practically disappeared in rich countries, and the rich have just
had to do without.So while there may be some things someone has to do, there's a good
chance anyone saying that about any particular job is mistaken.
Most unpleasant jobs would either get automated or go undone if no
one were willing to do them.Two RoutesThere's another sense of "not everyone can do work they love"
that's all too true, however.  One has to make a living, and it's
hard to get paid for doing work you love.  There are two routes to
that destination:

  The organic route: as you become more eminent, gradually to
  increase the parts of your job that you like at the expense of
  those you don't.The two-job route: to work at things you don't like to get money
  to work on things you do.

The organic route is more common.  It happens naturally to anyone
who does good work.  A young architect has to take whatever work
he can get, but if he does well he'll gradually be in a position
to pick and choose among projects.  The disadvantage of this route
is that it's slow and uncertain.  Even tenure is not real freedom.The two-job route has several variants depending on how long you
work for money at a time.  At one extreme is the "day job," where
you work regular hours at one job to make money, and work on what
you love in your spare time.  At the other extreme you work at
something till you make enough not to 
have to work for money again.The two-job route is less common than the organic route, because
it requires a deliberate choice.  It's also more dangerous.  Life
tends to get more expensive as you get older, so it's easy to get
sucked into working longer than you expected at the money job.
Worse still, anything you work on changes you.  If you work too
long on tedious stuff, it will rot your brain.  And the best paying
jobs are most dangerous, because they require your full attention.The advantage of the two-job route is that it lets you jump over
obstacles.  The landscape of possible jobs isn't flat; there are
walls of varying heights between different kinds of work. 
[7]
The trick of maximizing the parts of your job that you like can get you
from architecture to product design, but not, probably, to music.
If you make money doing one thing and then work on another, you
have more freedom of choice.Which route should you take?  That depends on how sure you are of
what you want to do, how good you are at taking orders, how much
risk you can stand, and the odds that anyone will pay (in your
lifetime) for what you want to do.  If you're sure of the general
area you want to work in and it's something people are likely to
pay you for, then you should probably take the organic route.  But
if you don't know what you want to work on, or don't like to take
orders, you may want to take the two-job route, if you can stand
the risk.Don't decide too soon.  Kids who know early what they want to do
seem impressive, as if they got the answer to some math question
before the other kids.  They have an answer, certainly, but odds
are it's wrong.A friend of mine who is a quite successful doctor complains constantly
about her job.  When people applying to medical school ask her for
advice, she wants to shake them and yell "Don't do it!"  (But she
never does.) How did she get into this fix?  In high school she
already wanted to be a doctor.  And she is so ambitious and determined
that she overcame every obstacle along the way—including,
unfortunately, not liking it.Now she has a life chosen for her by a high-school kid.When you're young, you're given the impression that you'll get
enough information to make each choice before you need to make it.
But this is certainly not so with work.  When you're deciding what
to do, you have to operate on ridiculously incomplete information.
Even in college you get little idea what various types of work are
like.  At best you may have a couple internships, but not all jobs
offer internships, and those that do don't teach you much more about
the work than being a batboy teaches you about playing baseball.In the design of lives, as in the design of most other things, you
get better results if you use flexible media.  So unless you're
fairly sure what you want to do, your best bet may be to choose a
type of work that could turn into either an organic or two-job
career.  That was probably part of the reason I chose computers.
You can be a professor, or make a lot of money, or morph it into
any number of other kinds of work.It's also wise, early on, to seek jobs that let you do many different
things, so you can learn faster what various kinds of work are like.
Conversely, the extreme version of the two-job route is dangerous
because it teaches you so little about what you like.  If you work
hard at being a bond trader for ten years, thinking that you'll
quit and write novels when you have enough money, what happens when
you quit and then discover that you don't actually like writing
novels?Most people would say, I'd take that problem.  Give me a million
dollars and I'll figure out what to do.  But it's harder than it
looks.  Constraints give your life shape.  Remove them and most
people have no idea what to do: look at what happens to those who
win lotteries or inherit money.  Much as everyone thinks they want
financial security, the happiest people are not those who have it,
but those who like what they do.  So a plan that promises freedom
at the expense of knowing what to do with it may not be as good as
it seems.Whichever route you take, expect a struggle.  Finding work you love
is very difficult.  Most people fail.  Even if you succeed, it's
rare to be free to work on what you want till your thirties or
forties.  But if you have the destination in sight you'll be more
likely to arrive at it.  If you know you can love work, you're in
the home stretch, and if you know what work you love, you're
practically there.Notes[1]
Currently we do the opposite: when we make kids do boring work,
like arithmetic drills, instead of admitting frankly that it's
boring, we try to disguise it with superficial decorations.[2]
One father told me about a related phenomenon: he found himself
concealing from his family how much he liked his work.  When he
wanted to go to work on a saturday, he found it easier to say that
it was because he "had to" for some reason, rather than admitting
he preferred to work than stay home with them.[3]
Something similar happens with suburbs.  Parents move to suburbs
to raise their kids in a safe environment, but suburbs are so dull
and artificial that by the time they're fifteen the kids are convinced
the whole world is boring.[4]
I'm not saying friends should be the only audience for your
work.  The more people you can help, the better.  But friends should
be your compass.[5]
Donald Hall said young would-be poets were mistaken to be so
obsessed with being published.  But you can imagine what it would
do for a 24 year old to get a poem published in The New Yorker.
Now to people he meets at parties he's a real poet.  Actually he's
no better or worse than he was before, but to a clueless audience
like that, the approval of an official authority makes all the
difference.   So it's a harder problem than Hall realizes.  The
reason the young care so much about prestige is that the people
they want to impress are not very discerning.[6]
This is isomorphic to the principle that you should prevent
your beliefs about how things are from being contaminated by how
you wish they were.  Most people let them mix pretty promiscuously.
The continuing popularity of religion is the most visible index of
that.[7]
A more accurate metaphor would be to say that the graph of jobs
is not very well connected.Thanks to Trevor Blackwell, Dan Friedman, Sarah Harlin,
Jessica Livingston, Jackie McDonough, Robert Morris, Peter Norvig, 
David Sloo, and Aaron Swartz
for reading drafts of this.May 2006(This essay is derived from a keynote at Xtech.)Could you reproduce Silicon Valley elsewhere, or is there something
unique about it?It wouldn't be surprising if it were hard to reproduce in other
countries, because you couldn't reproduce it in most of the US
either.  What does it take to make a silicon valley even here?What it takes is the right people.  If you could get the right ten
thousand people to move from Silicon Valley to Buffalo, Buffalo
would become Silicon Valley.  
[1]That's a striking departure from the past.  Up till a couple decades
ago, geography was destiny for cities.  All great cities were located
on waterways, because cities made money by trade, and water was the
only economical way to ship.Now you could make a great city anywhere, if you could get the right
people to move there.  So the question of how to make a silicon
valley becomes: who are the right people, and how do you get them
to move?Two TypesI think you only need two kinds of people to create a technology
hub: rich people and nerds.  They're the limiting reagents in the
reaction that produces startups, because they're the only ones
present when startups get started.  Everyone else will move.Observation bears this out: within the US, towns have become startup
hubs if and only if they have both rich people and nerds.  Few
startups happen in Miami, for example, because although it's full
of rich people, it has few nerds.  It's not the kind of place nerds
like.Whereas Pittsburgh has the opposite problem: plenty of nerds, but
no rich people.  The top US Computer Science departments are said
to be MIT, Stanford, Berkeley, and Carnegie-Mellon.  MIT yielded
Route 128.  Stanford and Berkeley yielded Silicon Valley.  But
Carnegie-Mellon?  The record skips at that point.  Lower down the
list, the University of Washington yielded a high-tech community
in Seattle, and the University of Texas at Austin yielded one in
Austin.  But what happened in Pittsburgh?  And in Ithaca, home of
Cornell, which is also high on the list?I grew up in Pittsburgh and went to college at Cornell, so I can
answer for both.  The weather is terrible,  particularly in winter,
and there's no interesting old city to make up for it, as there is
in Boston.  Rich people don't want to live in Pittsburgh or Ithaca.
So while there are plenty of hackers who could start startups,
there's no one to invest in them.Not BureaucratsDo you really need the rich people?  Wouldn't it work to have the
government invest in the nerds?  No, it would not.  Startup investors
are a distinct type of rich people.  They tend to have a lot of
experience themselves in the technology business.  This (a) helps
them pick the right startups, and (b) means they can supply advice
and connections as well as money.  And the fact that they have a
personal stake in the outcome makes them really pay attention.Bureaucrats by their nature are the exact opposite sort of people
from startup investors. The idea of them making startup investments
is comic.  It would be like mathematicians running Vogue-- or
perhaps more accurately, Vogue editors running a math journal.
[2]Though indeed, most things bureaucrats do, they do badly.   We just
don't notice usually, because they only have to compete against
other bureaucrats.  But as startup investors they'd have to compete
against pros with a great deal more experience and motivation.Even corporations that have in-house VC groups generally forbid
them to make their own investment decisions.  Most are only allowed
to invest in deals where some reputable private VC firm is willing
to act as lead investor.Not BuildingsIf you go to see Silicon Valley, what you'll see are buildings.
But it's the people that make it Silicon Valley, not the buildings.
I read occasionally about attempts to set up "technology
parks" in other places, as if the active ingredient of Silicon
Valley were the office space.  An article about Sophia Antipolis
bragged that companies there included Cisco, Compaq, IBM, NCR, and
Nortel.  Don't the French realize these aren't startups?Building office buildings for technology companies won't get you a
silicon valley, because the key stage in the life of a startup
happens before they want that kind of space.  The key stage is when
they're three guys operating out of an apartment.  Wherever the
startup is when it gets funded, it will stay.  The defining quality
of Silicon Valley is not that Intel or Apple or Google have offices
there, but that they were started there.So if you want to reproduce Silicon Valley, what you need to reproduce
is those two or three founders sitting around a kitchen table
deciding to start a company.  And to reproduce that you need those
people.UniversitiesThe exciting thing is, all you need are the people.  If you could
attract a critical mass of nerds and investors to live somewhere,
you could reproduce Silicon Valley.  And both groups are highly
mobile.  They'll go where life is good.  So what makes a place good
to them?What nerds like is other nerds.  Smart people will go wherever other
smart people are.  And in particular, to great universities.  In
theory there could be other ways to attract them, but so far
universities seem to be indispensable.  Within the US, there are
no technology hubs without first-rate universities-- or at least,
first-rate computer science departments.So if you want to make a silicon valley, you not only need a
university, but one of the top handful in the world.  It has to be
good enough to act as a magnet, drawing the best people from thousands
of miles away.  And that means it has to stand up to existing magnets
like MIT and Stanford.This sounds hard.  Actually it might be easy.  My professor friends,
when they're deciding where they'd like to work, consider one thing
above all: the quality of the other faculty.  What attracts professors
is good colleagues.  So if you managed to recruit, en masse, a
significant number of the best young researchers, you could create
a first-rate university from nothing overnight.  And you could do
that for surprisingly little.  If you paid 200 people hiring bonuses
of $3 million apiece, you could put together a faculty that would
bear comparison with any in the world.  And from that point the
chain reaction would be self-sustaining.  So whatever it costs to
establish a mediocre university, for an additional half billion or
so you could have a great one.  
[3]PersonalityHowever, merely creating a new university would not be enough to
start a silicon valley. The university is just the seed.  It has
to be planted in the right soil, or it won't germinate.  Plant it
in the wrong place, and you just create Carnegie-Mellon.To spawn startups, your university has to be in a town that has
attractions other than the university.  It has to be a place where
investors want to live, and students want to stay after they graduate.The two like much the same things, because most startup investors
are nerds themselves.  So what do nerds look for in a town?  Their
tastes aren't completely different from other people's, because a
lot of the towns they like most in the US are also big tourist
destinations: San Francisco, Boston, Seattle.   But their tastes
can't be quite mainstream either, because they dislike other big
tourist destinations, like New York, Los Angeles, and Las Vegas.There has been a lot written lately about the "creative class." The
thesis seems to be that as wealth derives increasingly from ideas,
cities will prosper only if they attract those who have them.  That
is certainly true; in fact it was the basis of Amsterdam's prosperity
400 years ago.A lot of nerd tastes they share with the creative class in general.
For example, they like well-preserved old neighborhoods instead of
cookie-cutter suburbs, and locally-owned shops and restaurants
instead of national chains.  Like the rest of the creative class,
they want to live somewhere with personality.What exactly is personality?  I think it's the feeling that each
building is the work of a distinct group of people.  A town with
personality is one that doesn't feel mass-produced.  So if you want
to make a startup hub-- or any town to attract the "creative class"--
you probably have to ban large development projects.
When a large tract has been developed by a single organization, you
can always tell. 
[4]Most towns with personality are old, but they don't have to be.
Old towns have two advantages: they're denser, because they were
laid out before cars, and they're more varied, because they were
built one building at a time.  You could have both now.  Just have
building codes that ensure density, and ban large scale developments.A corollary is that you have to keep out the biggest developer of
all: the government.  A government that asks "How can we build a
silicon valley?" has probably ensured failure by the way they framed
the question.  You don't build a silicon valley; you let one grow.NerdsIf you want to attract nerds, you need more than a town with
personality.  You need a town with the right personality.  Nerds
are a distinct subset of the creative class, with different tastes
from the rest.  You can see this most clearly in New York, which
attracts a lot of creative people, but few nerds. 
[5]What nerds like is the kind of town where people walk around smiling.
This excludes LA, where no one walks at all, and also New York,
where people walk, but not smiling. When I was in grad school in
Boston, a friend came to visit from New York.  On the subway back
from the airport she asked "Why is everyone smiling?"  I looked and
they weren't smiling.  They just looked like they were compared to
the facial expressions she was used to.If you've lived in New York, you know where these facial expressions
come from.  It's the kind of place where your mind may be excited,
but your body knows it's having a bad time.  People don't so much
enjoy living there as endure it for the sake of the excitement.
And if you like certain kinds of excitement, New York is incomparable.
It's a hub of glamour, a magnet for all the shorter half-life
isotopes of style and fame.Nerds don't care about glamour, so to them the appeal of New York
is a mystery.  People who like New York will pay a fortune for a
small, dark, noisy apartment in order to live in a town where the
cool people are really cool.  A nerd looks at that deal and sees
only: pay a fortune for a small, dark, noisy apartment.Nerds will pay a premium to live in a town where the smart people
are really smart, but you don't have to pay as much for that.  It's
supply and demand: glamour is popular, so you have to pay a lot for
it.Most nerds like quieter pleasures.  They like cafes instead of
clubs; used bookshops instead of fashionable clothing shops; hiking
instead of dancing; sunlight instead of tall buildings.  A nerd's
idea of paradise is Berkeley or Boulder.YouthIt's the young nerds who start startups, so it's those specifically
the city has to appeal to.  The startup hubs in the US are all
young-feeling towns.  This doesn't mean they have to be new.
Cambridge has the oldest town plan in America, but it feels young
because it's full of students.What you can't have, if you want to create a silicon valley, is a
large, existing population of stodgy people.  It would be a waste
of time to try to reverse the fortunes of a declining industrial town
like Detroit or Philadelphia by trying to encourage startups.  Those
places have too much momentum in the wrong direction.  You're better
off starting with a blank slate in the form of a small town.  Or
better still, if there's a town young people already flock to, that
one.The Bay Area was a magnet for the young and optimistic for decades
before it was associated with technology.  It was a place people
went in search of something new.  And so it became synonymous with
California nuttiness.  There's still a lot of that there.  If you
wanted to start a new fad-- a new way to focus one's "energy," for
example, or a new category of things not to eat-- the Bay Area would
be the place to do it.  But a place that tolerates oddness in the
search for the new is exactly what you want in a startup hub, because
economically that's what startups are.  Most good startup ideas
seem a little crazy; if they were obviously good ideas, someone
would have done them already.(How many people are going to want computers in their houses?
What, another search engine?)That's the connection between technology and liberalism.  Without
exception the high-tech cities in the US are also the most liberal.
But it's not because liberals are smarter that this is so.  It's
because liberal cities tolerate odd ideas, and smart people by
definition have odd ideas.Conversely, a town that gets praised for being "solid" or representing
"traditional values" may be a fine place to live, but it's never
going to succeed as a startup hub.  The 2004 presidential election,
though a disaster in other respects, conveniently supplied us with
a county-by-county 
map of such places.  
[6]To attract the young, a town must have an intact center.  In most
American cities the center has been abandoned, and the growth, if
any, is in the suburbs.  Most American cities have been turned
inside out. But none of the startup hubs has: not San Francisco,
or Boston, or Seattle.  They all have intact centers.
[7]
My guess is that no city with a dead center could be turned into a
startup hub.  Young people don't want to live in the suburbs.Within the US, the two cities I think could most easily be turned
into new silicon valleys are Boulder and Portland.  Both have the
kind of effervescent feel that attracts the young.  They're each
only a great university short of becoming a silicon valley, if they
wanted to.TimeA great university near an attractive town.  Is that all it takes?
That was all it took to make the original Silicon Valley.  Silicon
Valley traces its origins to William Shockley, one of the inventors
of the transistor.  He did the research that won him the Nobel Prize
at Bell Labs, but when he started his own company in 1956 he moved
to Palo Alto to do it.   At the time that was an odd thing to do.
Why did he?  Because he had grown up there and remembered how nice
it was.  Now Palo Alto is suburbia, but then it was a charming
college town-- a charming college town with perfect weather and San
Francisco only an hour away.The companies that rule Silicon Valley now are all descended in
various ways from Shockley Semiconductor.  Shockley was a difficult
man, and in 1957 his top people-- "the traitorous eight"-- left to
start a new company, Fairchild Semiconductor.  Among them were
Gordon Moore and Robert Noyce, who went on to found Intel, and
Eugene Kleiner, who founded the VC firm Kleiner Perkins.  Forty-two
years later, Kleiner Perkins funded Google, and the partner responsible
for the deal was John Doerr, who came to Silicon Valley in 1974 to
work for Intel.So although a lot of the newest companies in Silicon Valley don't
make anything out of silicon, there always seem to be multiple links
back to Shockley.  There's a lesson here: startups beget startups.
People who work for startups start their own.  People who get rich
from startups fund new ones.  I suspect this kind of organic growth
is the only way to produce a startup hub, because it's the only way
to grow the expertise you need.That has two important implications.  The first is that you need
time to grow a silicon valley.  The university you could create in
a couple years, but the startup community around it has to grow
organically.   The cycle time is limited by the time it takes a
company to succeed, which probably averages about five years.The other implication of the organic growth hypothesis is that you
can't be somewhat of a startup hub.  You either have a self-sustaining
chain reaction, or not.  Observation confirms this too: cities
either have a startup scene, or they don't.  There is no middle
ground.  Chicago has the third largest metropolitan area in America.
As source of startups it's negligible compared to Seattle, number 15.The good news is that the initial seed can be quite small.  Shockley
Semiconductor, though itself not very successful, was big enough.
It brought a critical mass of experts in an important new technology
together in a place they liked enough to stay.CompetingOf course, a would-be silicon valley faces an obstacle the original
one didn't: it has to compete with Silicon Valley.  Can that be
done?  Probably.One of Silicon Valley's biggest advantages is its venture capital
firms.  This was not a factor in Shockley's day, because VC funds
didn't exist.  In fact, Shockley Semiconductor and Fairchild
Semiconductor were not startups at all in our sense.  They were
subsidiaries-- of Beckman Instruments and Fairchild Camera and
Instrument respectively.  Those companies were apparently willing
to establish subsidiaries wherever the experts wanted to live.Venture investors, however, prefer to fund startups within an hour's
drive.  For one, they're more likely to notice startups nearby.
But when they do notice startups in other towns they prefer them
to move.  They don't want to have to travel to attend board meetings,
and in any case the odds of succeeding are higher in a startup hub.The centralizing effect of venture firms is a double one: they cause
startups to form around them, and those draw in more startups through
acquisitions.  And although the first may be weakening because it's
now so cheap to start some startups, the second seems as strong as ever.
Three of the most admired
"Web 2.0" companies were started outside the usual startup hubs,
but two of them have already been reeled in through acquisitions.Such centralizing forces make it harder for new silicon valleys to
get started.  But by no means impossible.  Ultimately power rests
with the founders.  A startup with the best people will beat one
with funding from famous VCs, and a startup that was sufficiently
successful would never have to move.  So a town that
could exert enough pull over the right people could resist and
perhaps even surpass Silicon Valley.For all its power, Silicon Valley has a great weakness: the paradise
Shockley found in 1956 is now one giant parking lot.  San Francisco
and Berkeley are great, but they're forty miles away.  Silicon
Valley proper is soul-crushing suburban sprawl.  It
has fabulous weather, which makes it significantly better than the
soul-crushing sprawl of most other American cities.  But a competitor
that managed to avoid sprawl would have real leverage.  All a city
needs is to be the kind of place the next traitorous eight look at
and say "I want to stay here," and that would be enough to get the
chain reaction started.Notes[1]
It's interesting to consider how low this number could be
made.  I suspect five hundred would be enough, even if they could
bring no assets with them.  Probably just thirty, if I could pick them, 
would be enough to turn Buffalo into a significant startup hub.[2]
Bureaucrats manage to allocate research funding moderately
well, but only because (like an in-house VC fund) they outsource
most of the work of selection.  A professor at a famous university
who is highly regarded by his peers will get funding, pretty much
regardless of the proposal.  That wouldn't work for startups, whose
founders aren't sponsored by organizations, and are often unknowns.[3]
You'd have to do it all at once, or at least a whole department
at a time, because people would be more likely to come if they
knew their friends were.  And you should probably start from scratch,
rather than trying to upgrade an existing university, or much energy
would be lost in friction.[4]
Hypothesis: Any plan in which multiple independent buildings
are gutted or demolished to be "redeveloped" as a single project
is a net loss of personality for the city, with the exception of
the conversion of buildings not previously public, like warehouses.[5]
A few startups get started in New York, but less
than a tenth as many per capita as in Boston, and mostly
in less nerdy fields like finance and media.[6]
Some blue counties are false positives (reflecting the
remaining power of Democractic party machines), but there are no
false negatives.  You can safely write off all the red counties.[7]
Some "urban renewal" experts took a shot at destroying Boston's
in the 1960s, leaving the area around city hall a bleak wasteland,
but most neighborhoods successfully resisted them.Thanks to Chris Anderson, Trevor Blackwell, Marc Hedlund,
Jessica Livingston, Robert Morris, Greg Mcadoo, Fred Wilson,
and Stephen Wolfram for
reading drafts of this, and to Ed Dumbill for inviting me to speak.(The second part of this talk became Why Startups
Condense in America.)January 2015Corporate Development, aka corp dev, is the group within companies
that buys other companies. If you're talking to someone from corp
dev, that's why, whether you realize it yet or not.It's usually a mistake to talk to corp dev unless (a) you want to
sell your company right now and (b) you're sufficiently likely to
get an offer at an acceptable price.  In practice that means startups
should only talk to corp dev when they're either doing really well
or really badly.  If you're doing really badly, meaning the company
is about to die, you may as well talk to them, because you have
nothing to lose. And if you're doing really well, you can safely
talk to them, because you both know the price will have to be high,
and if they show the slightest sign of wasting your time, you'll
be confident enough to tell them to get lost.The danger is to companies in the middle.  Particularly to young
companies that are growing fast, but haven't been doing it for long
enough to have grown big yet.  It's usually a mistake for a promising
company less than a year old even to talk to corp dev.But it's a mistake founders constantly make.  When someone from
corp dev wants to meet, the founders tell themselves they should
at least find out what they want.  Besides, they don't want to
offend Big Company by refusing to meet.Well, I'll tell you what they want.  They want to talk about buying
you.  That's what the title "corp dev" means.   So before agreeing
to meet with someone from corp dev, ask yourselves, "Do we want to
sell the company right now?"  And if the answer is no, tell them
"Sorry, but we're focusing on growing the company."  They won't be
offended.  And certainly the founders of Big Company won't be
offended. If anything they'll think more highly of you.  You'll
remind them of themselves.  They didn't sell either; that's why
they're in a position now to buy other companies.
[1]Most founders who get contacted by corp dev already know what it
means.  And yet even when they know what corp dev does and know
they don't want to sell, they take the meeting.  Why do they do it?
The same mix of denial and wishful thinking that underlies most
mistakes founders make. It's flattering to talk to someone who wants
to buy you.  And who knows, maybe their offer will be surprisingly
high.  You should at least see what it is, right?No.  If they were going to send you an offer immediately by email,
sure, you might as well open it.  But that is not how conversations
with corp dev work.  If you get an offer at all, it will be at the
end of a long and unbelievably distracting process.  And if the
offer is surprising, it will be surprisingly low.Distractions are the thing you can least afford in a startup.  And
conversations with corp dev are the worst sort of distraction,
because as well as consuming your attention they undermine your
morale.  One of the tricks to surviving a grueling process is not
to stop and think how tired you are.  Instead you get into a sort
of flow. 
[2]
Imagine what it would do to you if at mile 20 of a
marathon, someone ran up beside you and said "You must feel really
tired.  Would you like to stop and take a rest?"  Conversations
with corp dev are like that but worse, because the suggestion of
stopping gets combined in your mind with the imaginary high price
you think they'll offer.And then you're really in trouble.  If they can, corp dev people
like to turn the tables on you. They like to get you to the point
where you're trying to convince them to buy instead of them trying
to convince you to sell.  And surprisingly often they succeed.This is a very slippery slope, greased with some of the most powerful
forces that can work on founders' minds, and attended by an experienced
professional whose full time job is to push you down it.Their tactics in pushing you down that slope are usually fairly
brutal. Corp dev people's whole job is to buy companies, and they
don't even get to choose which.  The only way their performance is
measured is by how cheaply they can buy you, and the more ambitious
ones will stop at nothing to achieve that. For example, they'll
almost always start with a lowball offer, just to see if you'll
take it. Even if you don't, a low initial offer will demoralize you
and make you easier to manipulate.And that is the most innocent of their tactics. Just wait till
you've agreed on a price and think you have a done deal, and then
they come back and say their boss has vetoed the deal and won't do
it for more than half the agreed upon price. Happens all the time.
If you think investors can behave badly, it's nothing compared to
what corp dev people can do.  Even corp dev people at companies
that are otherwise benevolent.I remember once complaining to a
friend at Google about some nasty trick their corp dev people had
pulled on a YC startup."What happened to Don't be Evil?" I asked."I don't think corp dev got the memo," he replied.The tactics you encounter in M&A conversations can be like nothing
you've experienced in the otherwise comparatively 
upstanding world
of Silicon Valley.  It's as if a chunk of genetic material from the
old-fashioned robber baron business world got incorporated into the
startup world.
[3]The simplest way to protect yourself is to use the trick that John
D. Rockefeller, whose grandfather was an alcoholic, used to protect
himself from becoming one.  He once told a Sunday school class

  Boys, do you know why I never became a drunkard?  Because I never
  took the first drink.

Do you want to sell your company right now?  Not eventually, right
now.  If not, just don't take the first meeting.  They won't be
offended.  And you in turn will be guaranteed to be spared one of
the worst experiences that can happen to a startup.If you do want to sell, there's another set of 
techniques
 for doing
that.  But the biggest mistake founders make in dealing with corp
dev is not doing a bad job of talking to them when they're ready
to, but talking to them before they are.  So if you remember only
the title of this essay, you already know most of what you need to
know about M&A in the first year.Notes[1]
I'm not saying you should never sell.  I'm saying you should
be clear in your own mind about whether you want to sell or not,
and not be led by manipulation or wishful thinking into trying to
sell earlier than you otherwise would have.[2]
In a startup, as in most competitive sports, the task at hand
almost does this for you; you're too busy to feel tired.  But when
you lose that protection, e.g. at the final whistle, the fatigue
hits you like a wave.  To talk to corp dev is to let yourself feel
it mid-game.[3]
To be fair, the apparent misdeeds of corp dev people are magnified
by the fact that they function as the face of a large organization
that often doesn't know its own mind.  Acquirers can be surprisingly
indecisive about acquisitions, and their flakiness is indistinguishable
from dishonesty by the time it filters down to you.Thanks to Marc Andreessen, Jessica Livingston, Geoff
Ralston, and Qasar Younis for reading drafts of this.September 2007In high school I decided I was going to study philosophy in college.
I had several motives, some more honorable than others.  One of the
less honorable was to shock people.  College was regarded as job
training where I grew up, so studying philosophy seemed an impressively
impractical thing to do.  Sort of like slashing holes in your clothes
or putting a safety pin through your ear, which were other forms
of impressive impracticality then just coming into fashion.But I had some more honest motives as well.  I thought studying
philosophy would be a shortcut straight to wisdom.  All the people
majoring in other things would just end up with a bunch of domain
knowledge.  I would be learning what was really what.I'd tried to read a few philosophy books.  Not recent ones; you
wouldn't find those in our high school library.  But I tried to
read Plato and Aristotle.  I doubt I believed I understood them,
but they sounded like they were talking about something important.
I assumed I'd learn what in college.The summer before senior year I took some college classes.  I learned
a lot in the calculus class, but I didn't learn much in Philosophy
101.  And yet my plan to study philosophy remained intact.  It was
my fault I hadn't learned anything.  I hadn't read the books we
were assigned carefully enough.  I'd give Berkeley's Principles
of Human Knowledge another shot in college.  Anything so admired
and so difficult to read must have something in it, if one could
only figure out what.Twenty-six years later, I still don't understand Berkeley.  I have
a nice edition of his collected works.  Will I ever read it?  Seems
unlikely.The difference between then and now is that now I understand why
Berkeley is probably not worth trying to understand.  I think I see
now what went wrong with philosophy, and how we might fix it.WordsI did end up being a philosophy major for most of college.  It
didn't work out as I'd hoped.  I didn't learn any magical truths
compared to which everything else was mere domain knowledge.  But
I do at least know now why I didn't.  Philosophy doesn't really
have a subject matter in the way math or history or most other
university subjects do.  There is no core of knowledge one must
master.  The closest you come to that is a knowledge of what various
individual philosophers have said about different topics over the
years.  Few were sufficiently correct that people have forgotten
who discovered what they discovered.Formal logic has some subject matter. I took several classes in
logic.  I don't know if I learned anything from them.
[1]
It does seem to me very important to be able to flip ideas around in
one's head: to see when two ideas don't fully cover the space of
possibilities, or when one idea is the same as another but with a
couple things changed.  But did studying logic teach me the importance
of thinking this way, or make me any better at it?  I don't know.There are things I know I learned from studying philosophy.  The
most dramatic I learned immediately, in the first semester of
freshman year, in a class taught by Sydney Shoemaker.  I learned
that I don't exist.  I am (and you are) a collection of cells that
lurches around driven by various forces, and calls itself I.  But
there's no central, indivisible thing that your identity goes with.
You could conceivably lose half your brain and live.  Which means
your brain could conceivably be split into two halves and each
transplanted into different bodies.  Imagine waking up after such
an operation.  You have to imagine being two people.The real lesson here is that the concepts we use in everyday life
are fuzzy, and break down if pushed too hard.  Even a concept as
dear to us as I.  It took me a while to grasp this, but when I
did it was fairly sudden, like someone in the nineteenth century
grasping evolution and realizing the story of creation they'd been
told as a child was all wrong. 
[2]
Outside of math there's a limit
to how far you can push words; in fact, it would not be a bad
definition of math to call it the study of terms that have precise
meanings.  Everyday words are inherently imprecise.  They work well
enough in everyday life that you don't notice.  Words seem to work,
just as Newtonian physics seems to.  But you can always make them
break if you push them far enough.I would say that this has been, unfortunately for philosophy, the
central fact of philosophy.  Most philosophical debates are not
merely afflicted by but driven by confusions over words.  Do we
have free will?  Depends what you mean by "free." Do abstract ideas
exist?  Depends what you mean by "exist."Wittgenstein is popularly credited with the idea that most philosophical
controversies are due to confusions over language.  I'm not sure
how much credit to give him.  I suspect a lot of people realized
this, but reacted simply by not studying philosophy, rather than
becoming philosophy professors.How did things get this way?  Can something people have spent
thousands of years studying really be a waste of time?  Those are
interesting questions.  In fact, some of the most interesting
questions you can ask about philosophy.  The most valuable way to
approach the current philosophical tradition may be neither to get
lost in pointless speculations like Berkeley, nor to shut them down
like Wittgenstein, but to study it as an example of reason gone
wrong.HistoryWestern philosophy really begins with Socrates, Plato, and Aristotle.
What we know of their predecessors comes from fragments and references
in later works; their doctrines could be described as speculative
cosmology that occasionally strays into analysis.  Presumably they
were driven by whatever makes people in every other society invent
cosmologies.
[3]With Socrates, Plato, and particularly Aristotle, this tradition
turned a corner.  There started to be a lot more analysis.  I suspect
Plato and Aristotle were encouraged in this by progress in math.
Mathematicians had by then shown that you could figure things out
in a much more conclusive way than by making up fine sounding stories
about them.  
[4]People talk so much about abstractions now that we don't realize
what a leap it must have been when they first started to.  It was
presumably many thousands of years between when people first started
describing things as hot or cold and when someone asked "what is
heat?"  No doubt it was a very gradual process.  We don't know if
Plato or Aristotle were the first to ask any of the questions they
did.  But their works are the oldest we have that do this on a large
scale, and there is a freshness (not to say naivete) about them
that suggests some of the questions they asked were new to them,
at least.Aristotle in particular reminds me of the phenomenon that happens
when people discover something new, and are so excited by it that
they race through a huge percentage of the newly discovered territory
in one lifetime.  If so, that's evidence of how new this kind of
thinking was. 
[5]This is all to explain how Plato and Aristotle can be very impressive
and yet naive and mistaken.  It was impressive even to ask the
questions they did.  That doesn't mean they always came up with
good answers.  It's not considered insulting to say that ancient
Greek mathematicians were naive in some respects, or at least lacked
some concepts that would have made their lives easier.  So I hope
people will not be too offended if I propose that ancient philosophers
were similarly naive.  In particular, they don't seem to have fully
grasped what I earlier called the central fact of philosophy: that
words break if you push them too far."Much to the surprise of the builders of the first digital computers,"
Rod Brooks wrote, "programs written for them usually did not work."
[6]
Something similar happened when people first started trying
to talk about abstractions.  Much to their surprise, they didn't
arrive at answers they agreed upon.  In fact, they rarely seemed
to arrive at answers at all.They were in effect arguing about artifacts induced by sampling at
too low a resolution.The proof of how useless some of their answers turned out to be is
how little effect they have.  No one after reading Aristotle's
Metaphysics does anything differently as a result.
[7]Surely I'm not claiming that ideas have to have practical applications
to be interesting?  No, they may not have to.  Hardy's boast that
number theory had no use whatsoever wouldn't disqualify it.  But
he turned out to be mistaken.  In fact, it's suspiciously hard to
find a field of math that truly has no practical use.  And Aristotle's
explanation of the ultimate goal of philosophy in Book A of the
Metaphysics implies that philosophy should be useful too.Theoretical KnowledgeAristotle's goal was to find the most general of general principles.
The examples he gives are convincing: an ordinary worker builds
things a certain way out of habit; a master craftsman can do more
because he grasps the underlying principles.  The trend is clear:
the more general the knowledge, the more admirable it is.  But then
he makes a mistake—possibly the most important mistake in the
history of philosophy.  He has noticed that theoretical knowledge
is often acquired for its own sake, out of curiosity, rather than
for any practical need.  So he proposes there are two kinds of
theoretical knowledge: some that's useful in practical matters and
some that isn't.  Since people interested in the latter are interested
in it for its own sake, it must be more noble.  So he sets as his
goal in the Metaphysics the exploration of knowledge that has no
practical use.  Which means no alarms go off when he takes on grand
but vaguely understood questions and ends up getting lost in a sea
of words.His mistake was to confuse motive and result.  Certainly, people
who want a deep understanding of something are often driven by
curiosity rather than any practical need.  But that doesn't mean
what they end up learning is useless.  It's very valuable in practice
to have a deep understanding of what you're doing; even if you're
never called on to solve advanced problems, you can see shortcuts
in the solution of simple ones, and your knowledge won't break down
in edge cases, as it would if you were relying on formulas you
didn't understand.  Knowledge is power.  That's what makes theoretical
knowledge prestigious.  It's also what causes smart people to be
curious about certain things and not others; our DNA is not so
disinterested as we might think.So while ideas don't have to have immediate practical applications
to be interesting, the kinds of things we find interesting will
surprisingly often turn out to have practical applications.The reason Aristotle didn't get anywhere in the Metaphysics was
partly that he set off with contradictory aims: to explore the most
abstract ideas, guided by the assumption that they were useless.
He was like an explorer looking for a territory to the north of
him, starting with the assumption that it was located to the south.And since his work became the map used by generations of future
explorers, he sent them off in the wrong direction as well. 
[8]
Perhaps worst of all, he protected them from both the criticism of
outsiders and the promptings of their own inner compass by establishing
the principle that the most noble sort of theoretical knowledge had
to be useless.The Metaphysics is mostly a failed experiment.  A few ideas from
it turned out to be worth keeping; the bulk of it has had no effect
at all.  The Metaphysics is among the least read of all famous
books.  It's not hard to understand the way Newton's Principia
is, but the way a garbled message is.Arguably it's an interesting failed experiment.  But unfortunately
that was not the conclusion Aristotle's successors derived from
works like the Metaphysics. 
[9]
Soon after, the western world
fell on intellectual hard times.  Instead of version 1s to be
superseded, the works of Plato and Aristotle became revered texts
to be mastered and discussed.  And so things remained for a shockingly
long time.  It was not till around 1600 (in Europe, where the center
of gravity had shifted by then) that one found people confident
enough to treat Aristotle's work as a catalog of mistakes.  And
even then they rarely said so outright.If it seems surprising that the gap was so long, consider how little
progress there was in math between Hellenistic times and the
Renaissance.In the intervening years an unfortunate idea took hold:  that it
was not only acceptable to produce works like the Metaphysics,
but that it was a particularly prestigious line of work, done by a
class of people called philosophers.  No one thought to go back and
debug Aristotle's motivating argument.  And so instead of correcting
the problem Aristotle discovered by falling into it—that you can
easily get lost if you talk too loosely about very abstract ideas—they 
continued to fall into it.The SingularityCuriously, however, the works they produced continued to attract
new readers.  Traditional philosophy occupies a kind of singularity
in this respect.  If you write in an unclear way about big ideas,
you produce something that seems tantalizingly attractive to
inexperienced but intellectually ambitious students.  Till one knows
better, it's hard to distinguish something that's hard to understand
because the writer was unclear in his own mind from something like
a mathematical proof that's hard to understand because the ideas
it represents are hard to understand.  To someone who hasn't learned
the difference, traditional philosophy seems extremely attractive:
as hard (and therefore impressive) as math, yet broader in scope.
That was what lured me in as a high school student.This singularity is even more singular in having its own defense
built in.  When things are hard to understand, people who suspect
they're nonsense generally keep quiet.  There's no way to prove a
text is meaningless.  The closest you can get is to show that the
official judges of some class of texts can't distinguish them from
placebos. 
[10]And so instead of denouncing philosophy, most people who suspected
it was a waste of time just studied other things.  That alone is
fairly damning evidence, considering philosophy's claims.  It's
supposed to be about the ultimate truths. Surely all smart people
would be interested in it, if it delivered on that promise.Because philosophy's flaws turned away the sort of people who might
have corrected them, they tended to be self-perpetuating.  Bertrand
Russell wrote in a letter in 1912:

  Hitherto the people attracted to philosophy have been mostly those
  who loved the big generalizations, which were all wrong, so that
  few people with exact minds have taken up the subject.
[11]

His response was to launch Wittgenstein at it, with dramatic results.I think Wittgenstein deserves to be famous not for the discovery
that most previous philosophy was a waste of time, which judging
from the circumstantial evidence must have been made by every smart
person who studied a little philosophy and declined to pursue it
further, but for how he acted in response.
[12]
Instead of quietly
switching to another field, he made a fuss, from inside.  He was
Gorbachev.The field of philosophy is still shaken from the fright Wittgenstein
gave it. 
[13]
Later in life he spent a lot of time talking about
how words worked.  Since that seems to be allowed, that's what a
lot of philosophers do now.  Meanwhile, sensing a vacuum in the
metaphysical speculation department, the people who used to do
literary criticism have been edging Kantward, under new names like
"literary theory," "critical theory," and when they're feeling
ambitious, plain "theory."  The writing is the familiar word salad:

  Gender is not like some of the other grammatical modes which
  express precisely a mode of conception without any reality that
  corresponds to the conceptual mode, and consequently do not express
  precisely something in reality by which the intellect could be
  moved to conceive a thing the way it does, even where that motive
  is not something in the thing as such.
  [14]

The singularity I've described is not going away.  There's a market
for writing that sounds impressive and can't be disproven. There
will always be both supply and demand.  So if one group abandons
this territory, there will always be others ready to occupy it.A ProposalWe may be able to do better.  Here's an intriguing possibility.
Perhaps we should do what Aristotle meant to do, instead of what
he did.  The goal he announces in the Metaphysics seems one worth
pursuing: to discover the most general truths.  That sounds good.
But instead of trying to discover them because they're useless,
let's try to discover them because they're useful.I propose we try again, but that we use that heretofore despised
criterion, applicability, as a guide to keep us from wondering
off into a swamp of abstractions.  Instead of trying to answer the
question:

  What are the most general truths?

let's try to answer the question

  Of all the useful things we can say, which are the most general?

The test of utility I propose is whether we cause people who read
what we've written to do anything differently afterward.  Knowing
we have to give definite (if implicit) advice will keep us from
straying beyond the resolution of the words we're using.The goal is the same as Aristotle's; we just approach it from a
different direction.As an example of a useful, general idea, consider that of the
controlled experiment.  There's an idea that has turned out to be
widely applicable.  Some might say it's part of science, but it's
not part of any specific science; it's literally meta-physics (in
our sense of "meta").   The idea of evolution is another. It turns
out to have quite broad applications—for example, in genetic
algorithms and even product design.  Frankfurt's distinction between
lying and bullshitting seems a promising recent example.
[15]These seem to me what philosophy should look like: quite general
observations that would cause someone who understood them to do
something differently.Such observations will necessarily be about things that are imprecisely
defined.  Once you start using words with precise meanings, you're
doing math.  So starting from utility won't entirely solve the
problem I described above—it won't flush out the metaphysical
singularity.  But it should help.  It gives people with good
intentions a new roadmap into abstraction.  And they may thereby
produce things that make the writing of the people with bad intentions
look bad by comparison.One drawback of this approach is that it won't produce the sort of
writing that gets you tenure.  And not just because it's not currently
the fashion.  In order to get tenure in any field you must not
arrive at conclusions that members of tenure committees can disagree
with.  In practice there are two kinds of solutions to this problem.
In math and the sciences, you can prove what you're saying, or at
any rate adjust your conclusions so you're not claiming anything
false ("6 of 8 subjects had lower blood pressure after the treatment").
In the humanities you can either avoid drawing any definite conclusions
(e.g. conclude that an issue is a complex one), or draw conclusions
so narrow that no one cares enough to disagree with you.The kind of philosophy I'm advocating won't be able to take either
of these routes.  At best you'll be able to achieve the essayist's
standard of proof, not the mathematician's or the experimentalist's.
And yet you won't be able to meet the usefulness test without
implying definite and fairly broadly applicable conclusions.  Worse
still, the usefulness test will tend to produce results that annoy
people: there's no use in telling people things they already believe,
and people are often upset to be told things they don't.Here's the exciting thing, though.  Anyone can do this.  Getting
to general plus useful by starting with useful and cranking up the
generality may be unsuitable for junior professors trying to get
tenure, but it's better for everyone else, including professors who
already have it.  This side of the mountain is a nice gradual slope.
You can start by writing things that are useful but very specific,
and then gradually make them more general.  Joe's has good burritos.
What makes a good burrito?  What makes good food?  What makes
anything good?  You can take as long as you want.  You don't have
to get all the way to the top of the mountain.  You don't have to
tell anyone you're doing philosophy.If it seems like a daunting task to do philosophy, here's an
encouraging thought.  The field is a lot younger than it seems.
Though the first philosophers in the western tradition lived about
2500 years ago, it would be misleading to say the field is 2500
years old, because for most of that time the leading practitioners
weren't doing much more than writing commentaries on Plato or
Aristotle while watching over their shoulders for the next invading
army.  In the times when they weren't, philosophy was hopelessly
intermingled with religion.  It didn't shake itself free till a
couple hundred years ago, and even then was afflicted by the
structural problems I've described above.  If I say this, some will
say it's a ridiculously overbroad and uncharitable generalization,
and others will say it's old news, but here goes: judging from their
works, most philosophers up to the present have been wasting their
time.  So in a sense the field is still at the first step. 
[16]That sounds a preposterous claim to make.  It won't seem so
preposterous in 10,000 years.  Civilization always seems old, because
it's always the oldest it's ever been.  The only way to say whether
something is really old or not is by looking at structural evidence,
and structurally philosophy is young; it's still reeling from the
unexpected breakdown of words.Philosophy is as young now as math was in 1500.  There is a lot
more to discover.Notes
[1]
In practice formal logic is not much use, because despite
some progress in the last 150 years we're still only able to formalize
a small percentage of statements.  We may never do that much better,
for the same reason 1980s-style "knowledge representation" could
never have worked; many statements may have no representation more
concise than a huge, analog brain state.[2]
It was harder for Darwin's contemporaries to grasp this than
we can easily imagine.  The story of creation in the Bible is not
just a Judeo-Christian concept; it's roughly what everyone must
have believed since before people were people.  The hard part of
grasping evolution was to realize that species weren't, as they
seem to be, unchanging, but had instead evolved from different,
simpler organisms over unimaginably long periods of time.Now we don't have to make that leap.  No one in an industrialized
country encounters the idea of evolution for the first time as an
adult.  Everyone's taught about it as a child, either as truth or
heresy.[3]
Greek philosophers before Plato wrote in verse.  This must
have affected what they said.  If you try to write about the nature
of the world in verse, it inevitably turns into incantation.  Prose
lets you be more precise, and more tentative.[4]
Philosophy is like math's
ne'er-do-well brother.  It was born when Plato and Aristotle looked
at the works of their predecessors and said in effect "why can't
you be more like your brother?"  Russell was still saying the same
thing 2300 years later.Math is the precise half of the most abstract ideas, and philosophy
the imprecise half.  It's probably inevitable that philosophy will
suffer by comparison, because there's no lower bound to its precision.
Bad math is merely boring, whereas bad philosophy is nonsense.  And
yet there are some good ideas in the imprecise half.[5]
Aristotle's best work was in logic and zoology, both of which
he can  be said to have invented.  But the most dramatic departure
from his predecessors was a new, much more analytical style of
thinking.  He was arguably the first scientist.[6]
Brooks, Rodney, Programming in Common Lisp, Wiley, 1985, p.
94.[7]
Some would say we depend on Aristotle more than we realize,
because his ideas were one of the ingredients in our common culture.
Certainly a lot of the words we use have a connection with Aristotle,
but it seems a bit much to suggest that we wouldn't have the concept
of the essence of something or the distinction between matter and
form if Aristotle hadn't written about them.One way to see how much we really depend on Aristotle would be to
diff European culture with Chinese: what ideas did European culture
have in 1800 that Chinese culture didn't, in virtue of Aristotle's
contribution?[8]
The meaning of the word "philosophy" has changed over time.
In ancient times it covered a broad range of topics, comparable in
scope to our "scholarship" (though without the methodological
implications).  Even as late as Newton's time it included what we
now call "science."  But core of the subject today is still what
seemed to Aristotle the core: the attempt to discover the most
general truths.Aristotle didn't call this "metaphysics."  That name got assigned
to it because the books we now call the Metaphysics came after
(meta = after) the Physics in the standard edition of Aristotle's
works compiled by Andronicus of Rhodes three centuries later.  What
we call "metaphysics" Aristotle called "first philosophy."[9]
Some of Aristotle's immediate successors may have realized
this, but it's hard to say because most of their works are lost.[10]
Sokal, Alan, "Transgressing the Boundaries: Toward a Transformative
Hermeneutics of Quantum Gravity," Social Text 46/47, pp. 217-252.Abstract-sounding nonsense seems to be most attractive when it's
aligned with some axe the audience already has to grind.  If this
is so we should find it's most popular with groups that are (or
feel) weak.  The powerful don't need its reassurance.[11]
Letter to Ottoline Morrell, December 1912.  Quoted in:Monk, Ray, Ludwig Wittgenstein: The Duty of Genius, Penguin, 1991,
p. 75.[12]
A preliminary result, that all metaphysics between Aristotle
and 1783 had been a waste of time, is due to I. Kant.[13]
Wittgenstein asserted a sort of mastery to which the inhabitants
of early 20th century Cambridge seem to have been peculiarly
vulnerable—perhaps partly because so many had been raised religious
and then stopped believing, so had a vacant space in their heads
for someone to tell them what to do (others chose Marx or Cardinal
Newman), and partly because a quiet, earnest place like Cambridge
in that era had no natural immunity to messianic figures, just as
European politics then had no natural immunity to dictators.[14]
This is actually from the Ordinatio of Duns Scotus (ca.
1300), with "number" replaced by "gender."  Plus ca change.Wolter, Allan (trans), Duns Scotus: Philosophical Writings, Nelson,
1963, p. 92.[15]
Frankfurt, Harry, On Bullshit,  Princeton University Press,
2005.[16]
Some introductions to philosophy now take the line that
philosophy is worth studying as a process rather than for any
particular truths you'll learn.  The philosophers whose works they
cover would be rolling in their graves at that.  They hoped they
were doing more than serving as examples of how to argue: they hoped
they were getting results.  Most were wrong, but it doesn't seem
an impossible hope.This argument seems to me like someone in 1500 looking at the lack
of results achieved by alchemy and saying its value was as a process.
No, they were going about it wrong.  It turns out it is possible
to transmute lead into gold (though not economically at current
energy prices), but the route to that knowledge was to
backtrack and try another approach.Thanks to Trevor Blackwell, Paul Buchheit, Jessica Livingston, 
Robert Morris, Mark Nitzberg, and Peter Norvig for reading drafts of this.September 2017The most valuable insights are both general and surprising. 
F = ma for example. But general and surprising is a hard
combination to achieve. That territory tends to be picked
clean, precisely because those insights are so valuable.Ordinarily, the best that people can do is one without the
other: either surprising without being general (e.g.
gossip), or general without being surprising (e.g.
platitudes).Where things get interesting is the moderately valuable
insights.  You get those from small additions of whichever
quality was missing.  The more common case is a small
addition of generality: a piece of gossip that's more than
just gossip, because it teaches something interesting about
the world. But another less common approach is to focus on
the most general ideas and see if you can find something new
to say about them. Because these start out so general, you
only need a small delta of novelty to produce a useful
insight.A small delta of novelty is all you'll be able to get most
of the time. Which means if you take this route, your ideas
will seem a lot like ones that already exist. Sometimes
you'll find you've merely rediscovered an idea that did
already exist.  But don't be discouraged.  Remember the huge
multiplier that kicks in when you do manage to think of
something even a little new.Corollary: the more general the ideas you're talking about,
the less you should worry about repeating yourself.  If you
write enough, it's inevitable you will.  Your brain is much
the same from year to year and so are the stimuli that hit
it. I feel slightly bad when I find I've said something
close to what I've said before, as if I were plagiarizing
myself. But rationally one shouldn't.  You won't say
something exactly the same way the second time, and that
variation increases the chance you'll get that tiny but
critical delta of novelty.And of course, ideas beget ideas.  (That sounds 
familiar.)
An idea with a small amount of novelty could lead to one
with more. But only if you keep going. So it's doubly
important not to let yourself be discouraged by people who
say there's not much new about something you've discovered.
"Not much new" is a real achievement when you're talking
about the most general ideas. It's not true that there's nothing new under the sun.  There
are some domains where there's almost nothing new.  But
there's a big difference between nothing and almost nothing,
when it's multiplied by the area under the sun.
Thanks to Sam Altman, Patrick Collison, and Jessica
Livingston for reading drafts of this.January 2003(This article is derived from a keynote talk at the fall 2002 meeting
of NEPLS.)Visitors to this country are often surprised to find that
Americans like to begin a conversation by asking "what do you do?"
I've never liked this question.  I've rarely had a
neat answer to it.  But I think I have finally solved the problem.
Now, when someone asks me what I do, I look them straight
in the eye and say "I'm designing a 
new dialect of Lisp."   
I recommend this answer to anyone who doesn't like being asked what
they do.  The conversation will turn immediately to other topics.I don't consider myself to be doing research on programming languages.
I'm just designing one, in the same way that someone might design
a building or a chair or a new typeface.
I'm not trying to discover anything new.  I just want
to make a language that will be good to program in.  In some ways,
this assumption makes life a lot easier.The difference between design and research seems to be a question
of new versus good.  Design doesn't have to be new, but it has to  
be good.  Research doesn't have to be good, but it has to be new.
I think these two paths converge at the top: the best design
surpasses its predecessors by using new ideas, and the best research
solves problems that are not only new, but actually worth solving.
So ultimately we're aiming for the same destination, just approaching
it from different directions.What I'm going to talk about today is what your target looks like
from the back.  What do you do differently when you treat
programming languages as a design problem instead of a research topic?The biggest difference is that you focus more on the user.
Design begins by asking, who is this
for and what do they need from it?  A good architect,
for example, does not begin by creating a design that he then
imposes on the users, but by studying the intended users and figuring
out what they need.Notice I said "what they need," not "what they want."  I don't mean
to give the impression that working as a designer means working as 
a sort of short-order cook, making whatever the client tells you
to.  This varies from field to field in the arts, but
I don't think there is any field in which the best work is done by
the people who just make exactly what the customers tell them to.The customer is always right in
the sense that the measure of good design is how well it works
for the user.  If you make a novel that bores everyone, or a chair
that's horribly uncomfortable to sit in, then you've done a bad
job, period.  It's no defense to say that the novel or the chair  
is designed according to the most advanced theoretical principles.And yet, making what works for the user doesn't mean simply making
what the user tells you to.  Users don't know what all the choices
are, and are often mistaken about what they really want.The answer to the paradox, I think, is that you have to design
for the user, but you have to design what the user needs, not simply  
what he says he wants.
It's much like being a doctor.  You can't just treat a patient's
symptoms.  When a patient tells you his symptoms, you have to figure
out what's actually wrong with him, and treat that.This focus on the user is a kind of axiom from which most of the
practice of good design can be derived, and around which most design
issues center.If good design must do what the user needs, who is the user?  When
I say that design must be for users, I don't mean to imply that good 
design aims at some kind of  
lowest common denominator.  You can pick any group of users you
want.  If you're designing a tool, for example, you can design it
for anyone from beginners to experts, and what's good design
for one group might be bad for another.  The point
is, you have to pick some group of users.  I don't think you can
even talk about good or bad design except with
reference to some intended user.You're most likely to get good design if the intended users include
the designer himself.  When you design something
for a group that doesn't include you, it tends to be for people
you consider to be less sophisticated than you, not more sophisticated.That's a problem, because looking down on the user, however benevolently,
seems inevitably to corrupt the designer.
I suspect that very few housing
projects in the US were designed by architects who expected to live
in them.   You can see the same thing
in programming languages.  C, Lisp, and Smalltalk were created for
their own designers to use.  Cobol, Ada, and Java, were created   
for other people to use.If you think you're designing something for idiots, the odds are
that you're not designing something good, even for idiots.
Even if you're designing something for the most sophisticated
users, though, you're still designing for humans.  It's different 
in research.  In math you
don't choose abstractions because they're
easy for humans to understand; you choose whichever make the
proof shorter.  I think this is true for the sciences generally.
Scientific ideas are not meant to be ergonomic.Over in the arts, things are very different.  Design is
all about people.  The human body is a strange
thing, but when you're designing a chair,
that's what you're designing for, and there's no way around it.
All the arts have to pander to the interests and limitations
of humans.   In painting, for example, all other things being
equal a painting with people in it will be more interesting than
one without.  It is not merely an accident of history that
the great paintings of the Renaissance are all full of people.
If they hadn't been, painting as a medium wouldn't have the prestige
that it does.Like it or not, programming languages are also for people,
and I suspect the human brain is just as lumpy and idiosyncratic
as the human body.  Some ideas are easy for people to grasp
and some aren't.  For example, we seem to have a very limited
capacity for dealing with detail.  It's this fact that makes
programing languages a good idea in the first place; if we
could handle the detail, we could just program in machine
language.Remember, too, that languages are not
primarily a form for finished programs, but something that
programs have to be developed in.  Anyone in the arts could
tell you that you might want different mediums for the
two situations.  Marble, for example, is a nice, durable
medium for finished ideas, but a hopelessly inflexible one
for developing new ideas.A program, like a proof,
is a pruned version of a tree that in the past has had
false starts branching off all over it.  So the test of
a language is not simply how clean the finished program looks
in it, but how clean the path to the finished program was.
A design choice that gives you elegant finished programs
may not give you an elegant design process.  For example, 
I've written a few macro-defining macros full of nested
backquotes that look now like little gems, but writing them
took hours of the ugliest trial and error, and frankly, I'm still
not entirely sure they're correct.We often act as if the test of a language were how good
finished programs look in it.
It seems so convincing when you see the same program
written in two languages, and one version is much shorter.
When you approach the problem from the direction of the
arts, you're less likely to depend on this sort of
test.  You don't want to end up with a programming
language like marble.For example, it is a huge win in developing software to
have an interactive toplevel, what in Lisp is called a
read-eval-print loop.  And when you have one this has
real effects on the design of the language.  It would not
work well for a language where you have to declare
variables before using them, for example.  When you're
just typing expressions into the toplevel, you want to be 
able to set x to some value and then start doing things
to x.  You don't want to have to declare the type of x
first.  You may dispute either of the premises, but if
a language has to have a toplevel to be convenient, and
mandatory type declarations are incompatible with a
toplevel, then no language that makes type declarations  
mandatory could be convenient to program in.In practice, to get good design you have to get close, and stay
close, to your users.  You have to calibrate your ideas on actual
users constantly, especially in the beginning.  One of the reasons
Jane Austen's novels are so good is that she read them out loud to
her family.  That's why she never sinks into self-indulgently arty
descriptions of landscapes,
or pretentious philosophizing.  (The philosophy's there, but it's
woven into the story instead of being pasted onto it like a label.)
If you open an average "literary" novel and imagine reading it out loud
to your friends as something you'd written, you'll feel all too
keenly what an imposition that kind of thing is upon the reader.In the software world, this idea is known as Worse is Better.
Actually, there are several ideas mixed together in the concept of
Worse is Better, which is why people are still arguing about
whether worse
is actually better or not.  But one of the main ideas in that
mix is that if you're building something new, you should get a
prototype in front of users as soon as possible.The alternative approach might be called the Hail Mary strategy.
Instead of getting a prototype out quickly and gradually refining
it, you try to create the complete, finished, product in one long
touchdown pass.  As far as I know, this is a
recipe for disaster.  Countless startups destroyed themselves this
way during the Internet bubble.  I've never heard of a case
where it worked.What people outside the software world may not realize is that
Worse is Better is found throughout the arts.
In drawing, for example, the idea was discovered during the
Renaissance.  Now almost every drawing teacher will tell you that
the right way to get an accurate drawing is not to
work your way slowly around the contour of an object, because errors will
accumulate and you'll find at the end that the lines don't meet.
Instead you should draw a few quick lines in roughly the right place,
and then gradually refine this initial sketch.In most fields, prototypes
have traditionally been made out of different materials.
Typefaces to be cut in metal were initially designed  
with a brush on paper.  Statues to be cast in bronze   
were modelled in wax.  Patterns to be embroidered on tapestries
were drawn on paper with ink wash.  Buildings to be
constructed from stone were tested on a smaller scale in wood.What made oil paint so exciting, when it
first became popular in the fifteenth century, was that you
could actually make the finished work from the prototype.
You could make a preliminary drawing if you wanted to, but you
weren't held to it; you could work out all the details, and
even make major changes, as you finished the painting.You can do this in software too.  A prototype doesn't have to
be just a model; you can refine it into the finished product.
I think you should always do this when you can.  It lets you
take advantage of new insights you have along the way.  But
perhaps even more important, it's good for morale.Morale is key in design.  I'm surprised people
don't talk more about it.  One of my first
drawing teachers told me: if you're bored when you're
drawing something, the drawing will look boring.
For example, suppose you have to draw a building, and you
decide to draw each brick individually.  You can do this
if you want, but if you get bored halfway through and start
making the bricks mechanically instead of observing each one,   
the drawing will look worse than if you had merely suggested
the bricks.Building something by gradually refining a prototype is good
for morale because it keeps you engaged.  In software, my  
rule is: always have working code.  If you're writing
something that you'll be able to test in an hour, then you
have the prospect of an immediate reward to motivate you.
The same is true in the arts, and particularly in oil painting.
Most painters start with a blurry sketch and gradually
refine it.
If you work this way, then in principle
you never have to end the day with something that actually
looks unfinished.  Indeed, there is even a saying among
painters: "A painting is never finished, you just stop
working on it."  This idea will be familiar to anyone who
has worked on software.Morale is another reason that it's hard to design something
for an unsophisticated user.   It's hard to stay interested in
something you don't like yourself.  To make something  
good, you have to be thinking, "wow, this is really great,"
not "what a piece of shit; those fools will love it."Design means making things for humans.  But it's not just the
user who's human.  The designer is human too.Notice all this time I've been talking about "the designer."
Design usually has to be under the control of a single person to
be any good.   And yet it seems to be possible for several people
to collaborate on a research project.  This seems to
me one of the most interesting differences between research and
design.There have been famous instances of collaboration in the arts,
but most of them seem to have been cases of molecular bonding rather
than nuclear fusion.  In an opera it's common for one person to
write the libretto and another to write the music.   And during the Renaissance, 
journeymen from northern
Europe were often employed to do the landscapes in the
backgrounds of Italian paintings.  But these aren't true collaborations.
They're more like examples of Robert Frost's
"good fences make good neighbors."  You can stick instances
of good design together, but within each individual project,
one person has to be in control.I'm not saying that good design requires that one person think
of everything.  There's nothing more valuable than the advice
of someone whose judgement you trust.  But after the talking is
done, the decision about what to do has to rest with one person.Why is it that research can be done by collaborators and  
design can't?  This is an interesting question.  I don't 
know the answer.  Perhaps,
if design and research converge, the best research is also
good design, and in fact can't be done by collaborators.
A lot of the most famous scientists seem to have worked alone.
But I don't know enough to say whether there
is a pattern here.  It could be simply that many famous scientists
worked when collaboration was less common.Whatever the story is in the sciences, true collaboration
seems to be vanishingly rare in the arts.  Design by committee is a
synonym for bad design.  Why is that so?  Is there some way to
beat this limitation?I'm inclined to think there isn't-- that good design requires
a dictator.  One reason is that good design has to   
be all of a piece.  Design is not just for humans, but
for individual humans.  If a design represents an idea that  
fits in one person's head, then the idea will fit in the user's
head too.Related:November 2021(This essay is derived from a talk at the Cambridge Union.)When I was a kid, I'd have said there wasn't. My father told me so.
Some people like some things, and other people like other things,
and who's to say who's right?It seemed so obvious that there was no such thing as good taste
that it was only through indirect evidence that I realized my father
was wrong. And that's what I'm going to give you here: a proof by
reductio ad absurdum. If we start from the premise that there's no
such thing as good taste, we end up with conclusions that are
obviously false, and therefore the premise must be wrong.We'd better start by saying what good taste is. There's a narrow
sense in which it refers to aesthetic judgements and a broader one
in which it refers to preferences of any kind. The strongest proof
would be to show that taste exists in the narrowest sense, so I'm
going to talk about taste in art. You have better taste than me if
the art you like is better than the art I like.If there's no such thing as good taste, then there's no such thing
as good art. Because if there is such a
thing as good art, it's
easy to tell which of two people has better taste. Show them a lot
of works by artists they've never seen before and ask them to
choose the best, and whoever chooses the better art has better
taste.So if you want to discard the concept of good taste, you also have
to discard the concept of good art. And that means you have to
discard the possibility of people being good at making it. Which
means there's no way for artists to be good at their jobs. And not
just visual artists, but anyone who is in any sense an artist. You
can't have good actors, or novelists, or composers, or dancers
either. You can have popular novelists, but not good ones.We don't realize how far we'd have to go if we discarded the concept
of good taste, because we don't even debate the most obvious cases.
But it doesn't just mean we can't say which of two famous painters
is better. It means we can't say that any painter is better than a
randomly chosen eight year old.That was how I realized my father was wrong. I started studying
painting. And it was just like other kinds of work I'd done: you
could do it well, or badly, and if you tried hard, you could get
better at it. And it was obvious that Leonardo and Bellini were
much better at it than me. That gap between us was not imaginary.
They were so good. And if they could be good, then art could be
good, and there was such a thing as good taste after all.Now that I've explained how to show there is such a thing as good
taste, I should also explain why people think there isn't. There
are two reasons. One is that there's always so much disagreement
about taste. Most people's response to art is a tangle of unexamined
impulses. Is the artist famous? Is the subject attractive? Is this
the sort of art they're supposed to like? Is it hanging in a famous
museum, or reproduced in a big, expensive book? In practice most
people's response to art is dominated by such extraneous factors.And the people who do claim to have good taste are so often mistaken.
The paintings admired by the so-called experts in one generation
are often so different from those admired a few generations later.
It's easy to conclude there's nothing real there at all. It's only
when you isolate this force, for example by trying to paint and
comparing your work to Bellini's, that you can see that it does in
fact exist.The other reason people doubt that art can be good is that there
doesn't seem to be any room in the art for this goodness. The
argument goes like this. Imagine several people looking at a work
of art and judging how good it is. If being good art really is a
property of objects, it should be in the object somehow. But it
doesn't seem to be; it seems to be something happening in the heads
of each of the observers. And if they disagree, how do you choose
between them?The solution to this puzzle is to realize that the purpose of art
is to work on its human audience, and humans have a lot in common.
And to the extent the things an object acts upon respond in the
same way, that's arguably what it means for the object to have the
corresponding property. If everything a particle interacts with
behaves as if the particle had a mass of m, then it has a mass of
m. So the distinction between "objective" and "subjective" is not
binary, but a matter of degree, depending on how much the subjects
have in common. Particles interacting with one another are at one
pole, but people interacting with art are not all the way at the
other; their reactions aren't random.Because people's responses to art aren't random, art can be designed
to operate on people, and be good or bad depending on how effectively
it does so. Much as a vaccine can be. If someone were talking about
the ability of a vaccine to confer immunity, it would seem very
frivolous to object that conferring immunity wasn't really a property
of vaccines, because acquiring immunity is something that happens
in the immune system of each individual person. Sure, people's
immune systems vary, and a vaccine that worked on one might not
work on another, but that doesn't make it meaningless to talk about
the effectiveness of a vaccine.The situation with art is messier, of course. You can't measure
effectiveness by simply taking a vote, as you do with vaccines.
You have to imagine the responses of subjects with a deep knowledge
of art, and enough clarity of mind to be able to ignore extraneous
influences like the fame of the artist. And even then you'd still
see some disagreement. People do vary, and judging art is hard,
especially recent art. There is definitely not a total order either
of works or of people's ability to judge them. But there is equally
definitely a partial order of both. So while it's not possible to
have perfect taste, it is possible to have good taste.
Thanks to the Cambridge Union for inviting me, and to Trevor
Blackwell, Jessica Livingston, and Robert Morris for reading drafts
of this.
May 2001

(I wrote this article to help myself understand exactly
what McCarthy discovered.  You don't need to know this stuff
to program in Lisp, but it should be helpful to 
anyone who wants to
understand the essence of Lisp  both in the sense of its
origins and its semantic core.  The fact that it has such a core
is one of Lisp's distinguishing features, and the reason why,
unlike other languages, Lisp has dialects.)In 1960, John 
McCarthy published a remarkable paper in
which he did for programming something like what Euclid did for
geometry. He showed how, given a handful of simple
operators and a notation for functions, you can
build a whole programming language.
He called this language Lisp, for "List Processing,"
because one of his key ideas was to use a simple
data structure called a list for both
code and data.It's worth understanding what McCarthy discovered, not
just as a landmark in the history of computers, but as
a model for what programming is tending to become in
our own time.  It seems to me that there have been
two really clean, consistent models of programming so
far: the C model and the Lisp model.
These two seem points of high ground, with swampy lowlands
between them.  As computers have grown more powerful,
the new languages being developed have been moving
steadily toward the Lisp model.  A popular recipe
for new programming languages in the past 20 years 
has been to take the C model of computing and add to
it, piecemeal, parts taken from the Lisp model,
like runtime typing and garbage collection.In this article I'm going to try to explain in the
simplest possible terms what McCarthy discovered.
The point is not just to learn about an interesting
theoretical result someone figured out forty years ago,
but to show where languages are heading.
The unusual thing about Lisp  in fact, the defining
quality of Lisp  is that it can be written in
itself.  To understand what McCarthy meant by this,
we're going to retrace his steps, with his mathematical
notation translated into running Common Lisp code.February 2007A few days ago I finally figured out something I've wondered about
for 25 years: the relationship between wisdom and intelligence.
Anyone can see they're not the same by the number of people who are
smart, but not very wise.  And yet intelligence and wisdom do seem
related.  How?What is wisdom?  I'd say it's knowing what to do in a lot of
situations.  I'm not trying to make a deep point here about the
true nature of wisdom, just to figure out how we use the word.  A
wise person is someone who usually knows the right thing to do.And yet isn't being smart also knowing what to do in certain
situations?  For example, knowing what to do when the teacher tells
your elementary school class to add all the numbers from 1 to 100?
[1]Some say wisdom and intelligence apply to different types of
problems—wisdom to human problems and intelligence to abstract
ones.  But that isn't true.  Some wisdom has nothing to do with
people: for example, the wisdom of the engineer who knows certain
structures are less prone to failure than others.  And certainly
smart people can find clever solutions to human problems as well
as abstract ones. 
[2]Another popular explanation is that wisdom comes from experience
while intelligence is innate.  But people are not simply wise in
proportion to how much experience they have.  Other things must
contribute to wisdom besides experience, and some may be innate: a
reflective disposition, for example.Neither of the conventional explanations of the difference between
wisdom and intelligence stands up to scrutiny.  So what is the
difference?  If we look at how people use the words "wise" and
"smart," what they seem to mean is different shapes of performance.Curve"Wise" and "smart" are both ways of saying someone knows what to
do.  The difference is that "wise" means one has a high average
outcome across all situations, and "smart" means one does spectacularly
well in a few.  That is, if you had a graph in which the x axis
represented situations and the y axis the outcome, the graph of the
wise person would be high overall, and the graph of the smart person
would have high peaks.The distinction is similar to the rule that one should judge talent
at its best and character at its worst.  Except you judge intelligence
at its best, and wisdom by its average.  That's how the two are
related: they're the two different senses in which the same curve
can be high.So a wise person knows what to do in most situations, while a smart
person knows what to do in situations where few others could.  We
need to add one more qualification: we should ignore cases where
someone knows what to do because they have inside information. 
[3]
But aside from that, I don't think we can get much more specific
without starting to be mistaken.Nor do we need to.  Simple as it is, this explanation predicts, or
at least accords with, both of the conventional stories about the
distinction between wisdom and intelligence.  Human problems are
the most common type, so being good at solving those is key in
achieving a high average outcome.   And it seems natural that a
high average outcome depends mostly on experience, but that dramatic
peaks can only be achieved by people with certain rare, innate
qualities; nearly anyone can learn to be a good swimmer, but to be
an Olympic swimmer you need a certain body type.This explanation also suggests why wisdom is such an elusive concept:
there's no such thing.  "Wise" means something—that one is
on average good at making the right choice.  But giving the name
"wisdom" to the supposed quality that enables one to do that doesn't
mean such a thing exists.  To the extent "wisdom" means anything,
it refers to a grab-bag of qualities as various as self-discipline,
experience, and empathy.  
[4]Likewise, though "intelligent" means something, we're asking for
trouble if we insist on looking for a single thing called "intelligence."
And whatever its components, they're not all innate.  We use the
word "intelligent" as an indication of ability: a smart person can
grasp things few others could.  It does seem likely there's some
inborn predisposition to intelligence (and wisdom too), but this
predisposition is not itself intelligence.One reason we tend to think of intelligence as inborn is that people
trying to measure it have concentrated on the aspects of it that
are most measurable.  A quality that's inborn will obviously be
more convenient to work with than one that's influenced by experience,
and thus might vary in the course of a study.  The problem comes
when we drag the word "intelligence" over onto what they're measuring.
If they're measuring something inborn, they can't be measuring
intelligence.  Three year olds aren't smart.   When we describe one
as smart, it's shorthand for "smarter than other three year olds."SplitPerhaps it's a technicality to point out that a predisposition to
intelligence is not the same as intelligence.  But it's an important
technicality, because it reminds us that we can become smarter,
just as we can become wiser.The alarming thing is that we may have to choose between the two.If wisdom and intelligence are the average and peaks of the same
curve, then they converge as the number of points on the curve
decreases.  If there's just one point, they're identical: the average
and maximum are the same.  But as the number of points increases,
wisdom and intelligence diverge.  And historically the number of
points on the curve seems to have been increasing: our ability is
tested in an ever wider range of situations.In the time of Confucius and Socrates, people seem to have regarded
wisdom, learning, and intelligence as more closely related than we
do.  Distinguishing between "wise" and "smart" is a modern habit.
[5]
And the reason we do is that they've been diverging.  As knowledge
gets more specialized, there are more points on the curve, and the
distinction between the spikes and the average becomes sharper,
like a digital image rendered with more pixels.One consequence is that some old recipes may have become obsolete.
At the very least we have to go back and figure out if they were
really recipes for wisdom or intelligence.  But the really striking
change, as intelligence and wisdom drift apart, is that we may have
to decide which we prefer.  We may not be able to optimize for both
simultaneously.Society seems to have voted for intelligence.  We no longer admire
the sage—not the way people did two thousand years ago.  Now
we admire the genius.  Because in fact the distinction we began
with has a rather brutal converse: just as you can be smart without
being very wise, you can be wise without being very smart.  That
doesn't sound especially admirable.  That gets you James Bond, who
knows what to do in a lot of situations, but has to rely on Q for
the ones involving math.Intelligence and wisdom are obviously not mutually exclusive.  In
fact, a high average may help support high peaks.  But there are
reasons to believe that at some point you have to choose between
them.  One is the example of very smart people, who are so often
unwise that in popular culture this now seems to be regarded as the
rule rather than the exception.  Perhaps the absent-minded professor
is wise in his way, or wiser than he seems, but he's not wise in
the way Confucius or Socrates wanted people to be. 
[6]NewFor both Confucius and Socrates, wisdom, virtue, and happiness were
necessarily related.  The wise man was someone who knew what the
right choice was and always made it; to be the right choice, it had
to be morally right; he was therefore always happy, knowing he'd
done the best he could.  I can't think of many ancient philosophers
who would have disagreed with that, so far as it goes."The superior man is always happy; the small man sad," said Confucius.
[7]Whereas a few years ago I read an interview with a mathematician
who said that most nights he went to bed discontented, feeling he
hadn't made enough progress.  
[8]
The Chinese and Greek words we
translate as "happy" didn't mean exactly what we do by it, but
there's enough overlap that this remark contradicts them.Is the mathematician a small man because he's discontented?  No;
he's just doing a kind of work that wasn't very common in Confucius's
day.Human knowledge seems to grow fractally.  Time after time, something
that seemed a small and uninteresting area—experimental error,
even—turns out, when examined up close, to have as much in
it as all knowledge up to that point.  Several of the fractal buds
that have exploded since ancient times involve inventing and
discovering new things.  Math, for example, used to be something a
handful of people did part-time.  Now it's the career of thousands.
And in work that involves making new things, some old rules don't
apply.Recently I've spent some time advising people, and there I find the
ancient rule still works: try to understand the situation as well
as you can, give the best advice you can based on your experience,
and then don't worry about it, knowing you did all you could.  But
I don't have anything like this serenity when I'm writing an essay.
Then I'm worried.  What if I run out of ideas?  And when I'm writing,
four nights out of five I go to bed discontented, feeling I didn't
get enough done.Advising people and writing are fundamentally different types of
work.  When people come to you with a problem and you have to figure
out the right thing to do, you don't (usually) have to invent
anything.  You just weigh the alternatives and try to judge which
is the prudent choice.  But prudence can't tell me what sentence
to write next.  The search space is too big.Someone like a judge or a military officer can in much of his work
be guided by duty, but duty is no guide in making things.  Makers
depend on something more precarious: inspiration.  And like most
people who lead a precarious existence, they tend to be worried,
not contented.  In that respect they're more like the small man of
Confucius's day, always one bad harvest (or ruler) away from
starvation. Except instead of being at the mercy of weather and
officials, they're at the mercy of their own imagination.LimitsTo me it was a relief just to realize it might be ok to be discontented.
The idea that a successful person should be happy has thousands of
years of momentum behind it.  If I was any good, why didn't I have
the easy confidence winners are supposed to have?  But that, I now
believe, is like a runner asking "If I'm such a good athlete, why
do I feel so tired?" Good runners still get tired; they just get
tired at higher speeds.People whose work is to invent or discover things are in the same
position as the runner.  There's no way for them to do the best
they can, because there's no limit to what they could do.  The
closest you can come is to compare yourself to other people.  But
the better you do, the less this matters.  An undergrad who gets
something published feels like a star.  But for someone at the top
of the field, what's the test of doing well?  Runners can at least
compare themselves to others doing exactly the same thing; if you
win an Olympic gold medal, you can be fairly content, even if you
think you could have run a bit faster.  But what is a novelist to
do?Whereas if you're doing the kind of work in which problems are
presented to you and you have to choose between several alternatives,
there's an upper bound on your performance: choosing the best every
time.  In ancient societies, nearly all work seems to have been of
this type.  The peasant had to decide whether a garment was worth
mending, and the king whether or not to invade his neighbor, but
neither was expected to invent anything.  In principle they could
have; the king could have invented firearms, then invaded his
neighbor.  But in practice innovations were so rare that they weren't
expected of you, any more than goalkeepers are expected to score
goals. 
[9]
In practice, it seemed as if there was a correct decision
in every situation, and if you made it you'd done your job perfectly,
just as a goalkeeper who prevents the other team from scoring is
considered to have played a perfect game.In this world, wisdom seemed paramount.  
[10]
Even now, most people
do work in which problems are put before them and they have to
choose the best alternative.  But as knowledge has grown more
specialized, there are more and more types of work in which people
have to make up new things, and in which performance is therefore
unbounded.  Intelligence has become increasingly important relative
to wisdom because there is more room for spikes.RecipesAnother sign we may have to choose between intelligence and wisdom
is how different their recipes are.  Wisdom seems to come largely
from curing childish qualities, and intelligence largely from
cultivating them.Recipes for wisdom, particularly ancient ones, tend to have a
remedial character.  To achieve wisdom one must cut away all the
debris that fills one's head on emergence from childhood, leaving
only the important stuff.  Both self-control and experience have
this effect: to eliminate the random biases that come from your own
nature and from the circumstances of your upbringing respectively.
That's not all wisdom is, but it's a large part of it.  Much of
what's in the sage's head is also in the head of every twelve year
old.  The difference is that in the head of the twelve year old
it's mixed together with a lot of random junk.The path to intelligence seems to be through working on hard problems.
You develop intelligence as you might develop muscles, through
exercise.  But there can't be too much compulsion here.  No amount
of discipline can replace genuine curiosity.  So cultivating
intelligence seems to be a matter of identifying some bias in one's
character—some tendency to be interested in certain types of
things—and nurturing it.  Instead of obliterating your
idiosyncrasies in an effort to make yourself a neutral vessel for
the truth, you select one and try to grow it from a seedling into
a tree.The wise are all much alike in their wisdom, but very smart people
tend to be smart in distinctive ways.Most of our educational traditions aim at wisdom. So perhaps one
reason schools work badly is that they're trying to make intelligence
using recipes for wisdom.  Most recipes for wisdom have an element
of subjection.  At the very least, you're supposed to do what the
teacher says.  The more extreme recipes aim to break down your
individuality the way basic training does.  But that's not the route
to intelligence.  Whereas wisdom comes through humility, it may
actually help, in cultivating intelligence, to have a mistakenly
high opinion of your abilities, because that encourages you to keep
working.  Ideally till you realize how mistaken you were.(The reason it's hard to learn new skills late in life is not just
that one's brain is less malleable.  Another probably even worse
obstacle is that one has higher standards.)I realize we're on dangerous ground here.  I'm not proposing the
primary goal of education should be to increase students' "self-esteem."
That just breeds laziness.  And in any case, it doesn't really fool
the kids, not the smart ones.  They can tell at a young age that a
contest where everyone wins is a fraud.A teacher has to walk a narrow path: you want to encourage kids to
come up with things on their own, but you can't simply applaud
everything they produce.  You have to be a good audience: appreciative,
but not too easily impressed.  And that's a lot of work.  You have
to have a good enough grasp of kids' capacities at different ages
to know when to be surprised.That's the opposite of traditional recipes for education.  Traditionally
the student is the audience, not the teacher; the student's job is
not to invent, but to absorb some prescribed body of material.  (The
use of the term "recitation" for sections in some colleges is a
fossil of this.) The problem with these old traditions is that
they're too much influenced by recipes for wisdom.DifferentI deliberately gave this essay a provocative title; of course it's
worth being wise.  But I think it's important to understand the
relationship between intelligence and wisdom, and particularly what
seems to be the growing gap between them.  That way we can avoid
applying rules and standards to intelligence that are really meant
for wisdom.  These two senses of "knowing what to do" are more
different than most people realize.  The path to wisdom is through
discipline, and the path to intelligence through carefully selected
self-indulgence.  Wisdom is universal, and intelligence idiosyncratic.
And while wisdom yields calmness, intelligence much of the time
leads to discontentment.That's particularly worth remembering.  A physicist friend recently
told me half his department was on Prozac.  Perhaps if we acknowledge
that some amount of frustration is inevitable in certain kinds
of work, we can mitigate its effects.  Perhaps we can box it up and
put it away some of the time, instead of letting it flow together
with everyday sadness to produce what seems an alarmingly large
pool.  At the very least, we can avoid being discontented about
being discontented.If you feel exhausted, it's not necessarily because there's something
wrong with you.  Maybe you're just running fast.Notes[1]
Gauss was supposedly asked this when he was 10.  Instead of
laboriously adding together the numbers like the other students,
he saw that they consisted of 50 pairs that each summed to 101 (100
+ 1, 99 + 2, etc), and that he could just multiply 101 by 50 to get
the answer, 5050.[2]
A variant is that intelligence is the ability to solve problems,
and wisdom the judgement to know how to use those solutions.   But
while this is certainly an important relationship between wisdom
and intelligence, it's not the distinction between them.  Wisdom
is useful in solving problems too, and intelligence can help in
deciding what to do with the solutions.[3]
In judging both intelligence and wisdom we have to factor out
some knowledge. People who know the combination of a safe will be
better at opening it than people who don't, but no one would say
that was a test of intelligence or wisdom.But knowledge overlaps with wisdom and probably also intelligence.
A knowledge of human nature is certainly part of wisdom.  So where
do we draw the line?Perhaps the solution is to discount knowledge that at some point
has a sharp drop in utility.  For example, understanding French
will help you in a large number of situations, but its value drops
sharply as soon as no one else involved knows French.  Whereas the
value of understanding vanity would decline more gradually.The knowledge whose utility drops sharply is the kind that has
little relation to other knowledge.  This includes mere conventions,
like languages and safe combinations, and also what we'd call
"random" facts, like movie stars' birthdays, or how to distinguish
1956 from 1957 Studebakers.[4]
People seeking some single thing called "wisdom" have been
fooled by grammar.  Wisdom is just knowing the right thing to do,
and there are a hundred and one different qualities that help in
that.  Some, like selflessness, might come from meditating in an
empty room, and others, like a knowledge of human nature, might
come from going to drunken parties.Perhaps realizing this will help dispel the cloud of semi-sacred
mystery that surrounds wisdom in so many people's eyes.  The mystery
comes mostly from looking for something that doesn't exist.  And
the reason there have historically been so many different schools
of thought about how to achieve wisdom is that they've focused on
different components of it.When I use the word "wisdom" in this essay, I mean no more than
whatever collection of qualities helps people make the right choice
in a wide variety of situations.[5]
Even in English, our sense of the word "intelligence" is
surprisingly recent.  Predecessors like "understanding" seem to
have had a broader meaning.[6]
There is of course some uncertainty about how closely the remarks
attributed to Confucius and Socrates resemble their actual opinions.
I'm using these names as we use the name "Homer," to mean the
hypothetical people who said the things attributed to them.[7]
Analects VII:36, Fung trans.Some translators use "calm" instead of "happy."  One source of
difficulty here is that present-day English speakers have a different
idea of happiness from many older societies.  Every language probably
has a word meaning "how one feels when things are going well," but
different cultures react differently when things go well.  We react
like children, with smiles and laughter.  But in a more reserved
society, or in one where life was tougher, the reaction might be a
quiet contentment.[8]
It may have been Andrew Wiles, but I'm not sure.  If anyone
remembers such an interview, I'd appreciate hearing from you.[9]
Confucius claimed proudly that he had never invented
anything—that he had simply passed on an accurate account of
ancient traditions.  [Analects VII:1] It's hard for us now to
appreciate how important a duty it must have been in preliterate
societies to remember and pass on the group's accumulated knowledge.
Even in Confucius's time it still seems to have been the first duty
of the scholar.[10]
The bias toward wisdom in ancient philosophy may be exaggerated
by the fact that, in both Greece and China, many of the first
philosophers (including Confucius and Plato) saw themselves as
teachers of administrators, and so thought disproportionately about
such matters.  The few people who did invent things, like storytellers,
must have seemed an outlying data point that could be ignored.Thanks to Trevor Blackwell, Sarah Harlin, Jessica Livingston,
and Robert Morris for reading drafts of this.December 2014I've read Villehardouin's chronicle of the Fourth Crusade at least
two times, maybe three.  And yet if I had to write down everything
I remember from it, I doubt it would amount to much more than a
page.  Multiply this times several hundred, and I get an uneasy
feeling when I look at my bookshelves. What use is it to read all
these books if I remember so little from them?A few months ago, as I was reading Constance Reid's excellent
biography of Hilbert, I figured out if not the answer to this
question, at least something that made me feel better about it.
She writes:

  Hilbert had no patience with mathematical lectures which filled
  the students with facts but did not teach them how to frame a
  problem and solve it. He often used to tell them that "a perfect
  formulation of a problem is already half its solution."

That has always seemed to me an important point, and I was even
more convinced of it after hearing it confirmed by Hilbert.But how had I come to believe in this idea in the first place?  A
combination of my own experience and other things I'd read.  None
of which I could at that moment remember!  And eventually I'd forget
that Hilbert had confirmed it too.  But my increased belief in the
importance of this idea would remain something I'd learned from
this book, even after I'd forgotten I'd learned it.Reading and experience train your model of the world.  And even if
you forget the experience or what you read, its effect on your model
of the world persists.  Your mind is like a compiled program you've
lost the source of.  It works, but you don't know why.The place to look for what I learned from Villehardouin's chronicle
is not what I remember from it, but my mental models of the crusades,
Venice, medieval culture, siege warfare, and so on.  Which doesn't
mean I couldn't have read more attentively, but at least the harvest
of reading is not so miserably small as it might seem.This is one of those things that seem obvious in retrospect.  But
it was a surprise to me and presumably would be to anyone else who
felt uneasy about (apparently) forgetting so much they'd read.Realizing it does more than make you feel a little better about
forgetting, though.  There are specific implications.For example, reading and experience are usually "compiled" at the
time they happen, using the state of your brain at that time.  The
same book would get compiled differently at different points in
your life.  Which means it is very much worth reading important
books multiple times.  I always used to feel some misgivings about
rereading books.  I unconsciously lumped reading together with work
like carpentry, where having to do something again is a sign you
did it wrong the first time.  Whereas now the phrase "already read"
seems almost ill-formed.Intriguingly, this implication isn't limited to books.  Technology
will increasingly make it possible to relive our experiences.  When
people do that today it's usually to enjoy them again (e.g. when
looking at pictures of a trip) or to find the origin of some bug in
their compiled code (e.g. when Stephen Fry succeeded in remembering
the childhood trauma that prevented him from singing).  But as
technologies for recording and playing back your life improve, it
may become common for people to relive experiences without any goal
in mind, simply to learn from them again as one might when rereading
a book.Eventually we may be able not just to play back experiences but
also to index and even edit them. So although not knowing how you
know things may seem part of being human, it may not be.
Thanks to Sam Altman, Jessica Livingston, and Robert Morris for reading 
drafts of this.

Want to start a startup?  Get funded by
Y Combinator.




April 2001, rev. April 2003(This article is derived from a talk given at the 2001 Franz
Developer Symposium.)
In the summer of 1995, my friend Robert Morris and I
started a startup called 
Viaweb.  
Our plan was to write
software that would let end users build online stores.
What was novel about this software, at the time, was
that it ran on our server, using ordinary Web pages
as the interface.A lot of people could have been having this idea at the
same time, of course, but as far as I know, Viaweb was
the first Web-based application.  It seemed such
a novel idea to us that we named the company after it:
Viaweb, because our software worked via the Web,
instead of running on your desktop computer.Another unusual thing about this software was that it
was written primarily in a programming language called
Lisp. It was one of the first big end-user
applications to be written in Lisp, which up till then
had been used mostly in universities and research labs. [1]The Secret WeaponEric Raymond has written an essay called "How to Become a Hacker,"
and in it, among other things, he tells would-be hackers what
languages they should learn.  He suggests starting with Python and
Java, because they are easy to learn.  The serious hacker will also
want to learn C, in order to hack Unix, and Perl for system
administration and cgi scripts.  Finally, the truly serious hacker
should consider learning Lisp:

  Lisp is worth learning for the profound enlightenment experience
  you will have when you finally get it; that experience will make
  you a better programmer for the rest of your days, even if you
  never actually use Lisp itself a lot.

This is the same argument you tend to hear for learning Latin.  It
won't get you a job, except perhaps as a classics professor, but
it will improve your mind, and make you a better writer in languages
you do want to use, like English.But wait a minute.  This metaphor doesn't stretch that far.  The
reason Latin won't get you a job is that no one speaks it.  If you
write in Latin, no one can understand you.  But Lisp is a computer
language, and computers speak whatever language you, the programmer,
tell them to.So if Lisp makes you a better programmer, like he says, why wouldn't
you want to use it? If a painter were offered a brush that would
make him a better painter, it seems to me that he would want to
use it in all his paintings, wouldn't he? I'm not trying to make
fun of Eric Raymond here.  On the whole, his advice is good.  What
he says about Lisp is pretty much the conventional wisdom.  But
there is a contradiction in the conventional wisdom:  Lisp will
make you a better programmer, and yet you won't use it.Why not?  Programming languages are just tools, after all.  If Lisp
really does yield better programs, you should use it.  And if it
doesn't, then who needs it?This is not just a theoretical question.  Software is a very
competitive business, prone to natural monopolies.  A company that
gets software written faster and better will, all other things
being equal, put its competitors out of business.  And when you're
starting a startup, you feel this very keenly.  Startups tend to
be an all or nothing proposition.  You either get rich, or you get
nothing.  In a startup, if you bet on the wrong technology, your
competitors will crush you.Robert and I both knew Lisp well, and we couldn't see any reason
not to trust our instincts and go with Lisp.  We knew that everyone
else was writing their software in C++ or Perl.  But we also knew
that that didn't mean anything.  If you chose technology that way,
you'd be running Windows.  When you choose technology, you have to
ignore what other people are doing, and consider only what will
work the best.This is especially true in a startup.  In a big company, you can
do what all the other big companies are doing.  But a startup can't
do what all the other startups do.  I don't think a lot of people
realize this, even in startups.The average big company grows at about ten percent a year.  So if
you're running a big company and you do everything the way the
average big company does it, you can expect to do as well as the
average big company-- that is, to grow about ten percent a year.The same thing will happen if you're running a startup, of course.
If you do everything the way the average startup does it, you should
expect average performance.  The problem here is, average performance
means that you'll go out of business.  The survival rate for startups
is way less than fifty percent.  So if you're running a startup,
you had better be doing something odd.  If not, you're in trouble.Back in 1995, we knew something that I don't think our competitors
understood, and few understand even now:  when you're writing
software that only has to run on your own servers, you can use
any language you want.  When you're writing desktop software,
there's a strong bias toward writing applications in the same
language as the operating system.  Ten years ago, writing applications
meant writing applications in C.  But with Web-based software,
especially when you have the source code of both the language and
the operating system, you can use whatever language you want.This new freedom is a double-edged sword, however.  Now that you
can use any language, you have to think about which one to use.
Companies that try to pretend nothing has changed risk finding that
their competitors do not.If you can use any language, which do you use?  We chose Lisp.
For one thing, it was obvious that rapid development would be
important in this market.  We were all starting from scratch, so
a company that could get new features done before its competitors
would have a big advantage.  We knew Lisp was a really good language
for writing software quickly, and server-based applications magnify
the effect of rapid development, because you can release software
the minute it's done.If other companies didn't want to use Lisp, so much the better.
It might give us a technological edge, and we needed all the help
we could get.  When we started Viaweb, we had no experience in
business.  We didn't know anything about marketing, or hiring
people, or raising money, or getting customers.  Neither of us had
ever even had what you would call a real job.  The only thing we
were good at was writing software.  We hoped that would save us.
Any advantage we could get in the software department, we would
take.So you could say that using Lisp was an experiment.  Our hypothesis
was that if we wrote our software in Lisp, we'd be able to get
features done faster than our competitors, and also to do things
in our software that they couldn't do.  And because Lisp was so
high-level, we wouldn't need a big development team, so our costs
would be lower.  If this were so, we could offer a better product
for less money, and still make a profit.  We would end up getting
all the users, and our competitors would get none, and eventually
go out of business.  That was what we hoped would happen, anyway.What were the results of this experiment?  Somewhat surprisingly,
it worked.  We eventually had many competitors, on the order of
twenty to thirty of them, but none of their software could compete
with ours.  We had a wysiwyg online store builder that ran on the
server and yet felt like a desktop application.  Our competitors
had cgi scripts.  And we were always far ahead of them in features.
Sometimes, in desperation, competitors would try to introduce
features that we didn't have.  But with Lisp our development cycle
was so fast that we could sometimes duplicate a new feature within
a day or two of a competitor announcing it in a press release.  By
the time journalists covering the press release got round to calling
us, we would have the new feature too.It must have seemed to our competitors that we had some kind of
secret weapon-- that we were decoding their Enigma traffic or
something.  In fact we did have a secret weapon, but it was simpler
than they realized.  No one was leaking news of their features to
us.   We were just able to develop software faster than anyone
thought possible.When I was about nine I happened to get hold of a copy of The Day
of the Jackal, by Frederick Forsyth.  The main character is an
assassin who is hired to kill the president of France.  The assassin
has to get past the police to get up to an apartment that overlooks
the president's route.  He walks right by them, dressed up as an
old man on crutches, and they never suspect him.Our secret weapon was similar.  We wrote our software in a weird
AI language, with a bizarre syntax full of parentheses.  For years
it had annoyed me to hear Lisp described that way.  But now it
worked to our advantage.  In business, there is nothing more valuable
than a technical advantage your competitors don't understand.  In
business, as in war, surprise is worth as much as force.And so, I'm a little embarrassed to say, I never said anything
publicly about Lisp while we were working on Viaweb.  We never
mentioned it to the press, and if you searched for Lisp on our Web
site, all you'd find were the titles of two books in my bio.  This
was no accident.  A startup should give its competitors as little
information as possible.  If they didn't know what language our
software was written in, or didn't care, I wanted to keep it that
way.[2]The people who understood our technology best were the customers.
They didn't care what language Viaweb was written in either, but
they noticed that it worked really well.  It let them build great
looking online stores literally in minutes.  And so, by word of
mouth mostly, we got more and more users.  By the end of 1996 we
had about 70 stores online.  At the end of 1997 we had 500.  Six
months later, when Yahoo bought us, we had 1070 users.  Today, as
Yahoo Store, this software continues to dominate its market.  It's
one of the more profitable pieces of Yahoo, and the stores built
with it are the foundation of Yahoo Shopping.  I left Yahoo in
1999, so I don't know exactly how many users they have now, but
the last I heard there were about 20,000.
The Blub ParadoxWhat's so great about Lisp?  And if Lisp is so great, why doesn't
everyone use it?  These sound like rhetorical questions, but actually
they have straightforward answers.  Lisp is so great not because
of some magic quality visible only to devotees, but because it is
simply the most powerful language available.  And the reason everyone
doesn't use it is that programming languages are not merely
technologies, but habits of mind as well, and nothing changes
slower.  Of course, both these answers need explaining.I'll begin with a shockingly controversial statement:  programming
languages vary in power.Few would dispute, at least, that high level languages are more
powerful than machine language.  Most programmers today would agree
that you do not, ordinarily, want to program in machine language.
Instead, you should program in a high-level language, and have a
compiler translate it into machine language for you.  This idea is
even built into the hardware now: since the 1980s, instruction sets
have been designed for compilers rather than human programmers.Everyone knows it's a mistake to write your whole program by hand
in machine language.  What's less often understood is that there
is a more general principle here: that if you have a choice of
several languages, it is, all other things being equal, a mistake
to program in anything but the most powerful one. [3]There are many exceptions to this rule.  If you're writing a program
that has to work very closely with a program written in a certain
language, it might be a good idea to write the new program in the
same language.  If you're writing a program that only has to do
something very simple, like number crunching or bit manipulation,
you may as well use a less abstract language, especially since it
may be slightly faster.  And if you're writing a short, throwaway
program, you may be better off just using whatever language has
the best library functions for the task.  But in general, for
application software, you want to be using the most powerful
(reasonably efficient) language you can get, and using anything
else is a mistake, of exactly the same kind, though possibly in a
lesser degree, as programming in machine language.You can see that machine language is very low level.  But, at least
as a kind of social convention, high-level languages are often all
treated as equivalent.  They're not.  Technically the term "high-level
language" doesn't mean anything very definite.  There's no dividing
line with machine languages on one side and all the high-level
languages on the other.  Languages fall along a continuum [4] of
abstractness, from the most powerful all the way down to machine
languages, which themselves vary in power.Consider Cobol.  Cobol is a high-level language, in the sense that
it gets compiled into machine language.  Would anyone seriously
argue that Cobol is equivalent in power to, say, Python?  It's
probably closer to machine language than Python.Or how about Perl 4?  Between Perl 4 and Perl 5, lexical closures
got added to the language.  Most Perl hackers would agree that Perl
5 is more powerful than Perl 4.  But once you've admitted that,
you've admitted that one high level language can be more powerful
than another.  And it follows inexorably that, except in special
cases, you ought to use the most powerful you can get.This idea is rarely followed to its conclusion, though.  After a
certain age, programmers rarely switch languages voluntarily.
Whatever language people happen to be used to, they tend to consider
just good enough.Programmers get very attached to their favorite languages, and I
don't want to hurt anyone's feelings, so to explain this point I'm
going to use a hypothetical language called Blub.  Blub falls right
in the middle of the abstractness continuum.  It is not the most
powerful language, but it is more powerful than Cobol or machine
language.And in fact, our hypothetical Blub programmer wouldn't use either
of them.  Of course he wouldn't program in machine language.  That's
what compilers are for.  And as for Cobol, he doesn't know how
anyone can get anything done with it.  It doesn't even have x (Blub
feature of your choice).As long as our hypothetical Blub programmer is looking down the
power continuum, he knows he's looking down.  Languages less powerful
than Blub are obviously less powerful, because they're missing some
feature he's used to.  But when our hypothetical Blub programmer
looks in the other direction, up the power continuum, he doesn't
realize he's looking up.  What he sees are merely weird languages.
He probably considers them about equivalent in power to Blub, but
with all this other hairy stuff thrown in as well.  Blub is good
enough for him, because he thinks in Blub.When we switch to the point of view of a programmer using any of
the languages higher up the power continuum, however, we find that
he in turn looks down upon Blub.  How can you get anything done in
Blub? It doesn't even have y.By induction, the only programmers in a position to see all the
differences in power between the various languages are those who
understand the most powerful one.  (This is probably what Eric
Raymond meant about Lisp making you a better programmer.) You can't
trust the opinions of the others, because of the Blub paradox:
they're satisfied with whatever language they happen to use, because
it dictates the way they think about programs.I know this from my own experience, as a high school kid writing
programs in Basic.  That language didn't even support recursion.
It's hard to imagine writing programs without using recursion, but
I didn't miss it at the time.  I thought in Basic.  And I was a
whiz at it.  Master of all I surveyed.The five languages that Eric Raymond recommends to hackers fall at
various points on the power continuum.  Where they fall relative
to one another is a sensitive topic.  What I will say is that I
think Lisp is at the top.  And to support this claim I'll tell you
about one of the things I find missing when I look at the other
four languages.  How can you get anything done in them, I think,
without macros? [5]Many languages have something called a macro.  But Lisp macros are
unique.  And believe it or not, what they do is related to the
parentheses.  The designers of Lisp didn't put all those parentheses
in the language just to be different.  To the Blub programmer, Lisp
code looks weird.  But those parentheses are there for a reason.
They are the outward evidence of a fundamental difference between
Lisp and other languages.Lisp code is made out of Lisp data objects.  And not in the trivial
sense that the source files contain characters, and strings are
one of the data types supported by the language.  Lisp code, after
it's read by the parser, is made of data structures that you can
traverse.If you understand how compilers work, what's really going on is
not so much that Lisp has a strange syntax as that Lisp has no
syntax.  You write programs in the parse trees that get generated
within the compiler when other languages are parsed.  But these
parse trees are fully accessible to your programs.  You can write
programs that manipulate them.  In Lisp, these programs are called
macros.  They are programs that write programs.Programs that write programs?  When would you ever want to do that?
Not very often, if you think in Cobol.  All the time, if you think
in Lisp.  It would be convenient here if I could give an example
of a powerful macro, and say there! how about that?  But if I did,
it would just look like gibberish to someone who didn't know Lisp;
there isn't room here to explain everything you'd need to know to
understand what it meant.  In 
Ansi Common Lisp I tried to move
things along as fast as I could, and even so I didn't get to macros
until page 160.But I think I can give a kind of argument that might be convincing.
The source code of the Viaweb editor was probably about 20-25%
macros.  Macros are harder to write than ordinary Lisp functions,
and it's considered to be bad style to use them when they're not
necessary.  So every macro in that code is there because it has to
be.  What that means is that at least 20-25% of the code in this
program is doing things that you can't easily do in any other
language.  However skeptical the Blub programmer might be about my
claims for the mysterious powers of Lisp, this ought to make him
curious.  We weren't writing this code for our own amusement.  We
were a tiny startup, programming as hard as we could in order to
put technical barriers between us and our competitors.A suspicious person might begin to wonder if there was some
correlation here.  A big chunk of our code was doing things that
are very hard to do in other languages.  The resulting software
did things our competitors' software couldn't do.  Maybe there was
some kind of connection.  I encourage you to follow that thread.
There may be more to that old man hobbling along on his crutches
than meets the eye.Aikido for StartupsBut I don't expect to convince anyone 
(over 25) 
to go out and learn
Lisp.  The purpose of this article is not to change anyone's mind,
but to reassure people already interested in using Lisp-- people
who know that Lisp is a powerful language, but worry because it
isn't widely used.  In a competitive situation, that's an advantage.
Lisp's power is multiplied by the fact that your competitors don't
get it.If you think of using Lisp in a startup, you shouldn't worry that
it isn't widely understood.  You should hope that it stays that
way. And it's likely to.  It's the nature of programming languages
to make most people satisfied with whatever they currently use.
Computer hardware changes so much faster than personal habits that
programming practice is usually ten to twenty years behind the
processor.  At places like MIT they were writing programs in
high-level languages in the early 1960s, but many companies continued
to write code in machine language well into the 1980s.  I bet a
lot of people continued to write machine language until the processor,
like a bartender eager to close up and go home, finally kicked them
out by switching to a risc instruction set.Ordinarily technology changes fast.  But programming languages are
different: programming languages are not just technology, but what
programmers think in.  They're half technology and half religion.[6]
And so the median language, meaning whatever language the median
programmer uses, moves as slow as an iceberg.  Garbage collection,
introduced by Lisp in about 1960, is now widely considered to be
a good thing.  Runtime typing, ditto, is growing in popularity.
Lexical closures, introduced by Lisp in the early 1970s, are now,
just barely, on the radar screen.  Macros, introduced by Lisp in the
mid 1960s, are still terra incognita.Obviously, the median language has enormous momentum.  I'm not
proposing that you can fight this powerful force.  What I'm proposing
is exactly the opposite: that, like a practitioner of Aikido, you
can use it against your opponents.If you work for a big company, this may not be easy.  You will have
a hard time convincing the pointy-haired boss to let you build
things in Lisp, when he has just read in the paper that some other
language is poised, like Ada was twenty years ago, to take over
the world.  But if you work for a startup that doesn't have
pointy-haired bosses yet, you can, like we did, turn the Blub
paradox to your advantage:  you can use technology that your
competitors, glued immovably to the median language, will never be
able to match.If you ever do find yourself working for a startup, here's a handy
tip for evaluating competitors.  Read their job listings.  Everything
else on their site may be stock photos or the prose equivalent,
but the job listings have to be specific about what they want, or
they'll get the wrong candidates.During the years we worked on Viaweb I read a lot of job descriptions.
A new competitor seemed to emerge out of the woodwork every month
or so.  The first thing I would do, after checking to see if they
had a live online demo, was look at their job listings.  After a
couple years of this I could tell which companies to worry about
and which not to.  The more of an IT flavor the job descriptions
had, the less dangerous the company was.  The safest kind were the
ones that wanted Oracle experience.  You never had to worry about
those.  You were also safe if they said they wanted C++ or Java
developers.  If they wanted Perl or Python programmers, that would
be a bit frightening-- that's starting to sound like a company
where the technical side, at least, is run by real hackers.  If I
had ever seen a job posting looking for Lisp hackers, I would have
been really worried.
Notes[1] Viaweb at first had two parts: the editor, written in Lisp,
which people used to build their sites, and the ordering system,
written in C, which handled orders.  The first version was mostly
Lisp, because the ordering system was small.  Later we added two
more modules, an image generator written in C, and a back-office
manager written mostly in Perl.In January 2003, Yahoo released a new version of the editor 
written in C++ and Perl.  It's hard to say whether the program is no
longer written in Lisp, though, because to translate this program
into C++ they literally had to write a Lisp interpreter: the source
files of all the page-generating templates are still, as far as I
know,  Lisp code.  (See Greenspun's Tenth Rule.)[2] Robert Morris says that I didn't need to be secretive, because
even if our competitors had known we were using Lisp, they wouldn't
have understood why:  "If they were that smart they'd already be
programming in Lisp."[3] All languages are equally powerful in the sense of being Turing
equivalent, but that's not the sense of the word programmers care
about. (No one wants to program a Turing machine.)  The kind of
power programmers care about may not be formally definable, but
one way to explain it would be to say that it refers to features
you could only get in the less powerful language by writing an
interpreter for the more powerful language in it. If language A
has an operator for removing spaces from strings and language B
doesn't, that probably doesn't make A more powerful, because you
can probably write a subroutine to do it in B.  But if A supports,
say, recursion, and B doesn't, that's not likely to be something
you can fix by writing library functions.[4] Note to nerds: or possibly a lattice, narrowing toward the top;
it's not the shape that matters here but the idea that there is at
least a partial order.[5] It is a bit misleading to treat macros as a separate feature.
In practice their usefulness is greatly enhanced by other Lisp
features like lexical closures and rest parameters.[6] As a result, comparisons of programming languages either take
the form of religious wars or undergraduate textbooks so determinedly
neutral that they're really works of anthropology.  People who
value their peace, or want tenure, avoid the topic.  But the question
is only half a religious one; there is something there worth
studying, especially if you want to design new languages.July 2006I've discovered a handy test for figuring out what you're addicted
to.  Imagine you were going to spend the weekend at a friend's house
on a little island off the coast of Maine.  There are no shops on
the island and you won't be able to leave while you're there.  Also,
you've never been to this house before, so you can't assume it will
have more than any house might.What, besides clothes and toiletries, do you make a point of packing?
That's what you're addicted to.  For example, if you find yourself
packing a bottle of vodka (just in case), you may want to stop and
think about that.For me the list is four things: books, earplugs, a notebook, and a
pen.There are other things I might bring if I thought of it, like music,
or tea, but I can live without them.  I'm not so addicted to caffeine
that I wouldn't risk the house not having any tea, just for a
weekend.Quiet is another matter.  I realize it seems a bit eccentric to
take earplugs on a trip to an island off the coast of Maine.  If
anywhere should be quiet, that should.  But what if the person in
the next room snored?  What if there was a kid playing basketball?
(Thump, thump, thump... thump.)  Why risk it?  Earplugs are small.Sometimes I can think with noise.  If I already have momentum on
some project, I can work in noisy places.  I can edit an essay or
debug code in an airport.  But airports are not so bad: most of the
noise is whitish.  I couldn't work with the sound of a sitcom coming
through the wall, or a car in the street playing thump-thump music.And of course there's another kind of thinking, when you're starting
something new, that requires complete quiet.   You never
know when this will strike. It's just as well to carry plugs.The notebook and pen are professional equipment, as it were.  Though
actually there is something druglike about them, in the sense that
their main purpose is to make me feel better.  I hardly ever go
back and read stuff I write down in notebooks.  It's just that if
I can't write things down, worrying about remembering one idea gets
in the way of having the next.  Pen and paper wick ideas.The best notebooks I've found are made by a company called Miquelrius.
I use their smallest size, which is about 2.5 x 4 in.
The secret to writing on such
narrow pages is to break words only when you run out of space, like
a Latin inscription.  I use the cheapest plastic Bic ballpoints,
partly because their gluey ink doesn't seep through pages, and
partly so I don't worry about losing them.I only started carrying a notebook about three years ago.  Before
that I used whatever scraps of paper I could find.  But the problem
with scraps of paper is that they're not ordered.  In a notebook
you can guess what a scribble means by looking at the pages
around it.  In the scrap era I was constantly finding notes I'd
written years before that might say something I needed to remember,
if I could only figure out what.As for books, I know the house would probably have something to
read.  On the average trip I bring four books and only read one of
them, because I find new books to read en route.  Really bringing
books is insurance.I realize this dependence on books is not entirely good—that what
I need them for is distraction.  The books I bring on trips are
often quite virtuous, the sort of stuff that might be assigned
reading in a college class.  But I know my motives"""

In [4]:
Long_Context_1K = """<s> 
The best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.
<s><s>  

Want to start a startup?  Get funded by
Y Combinator.




July 2004(This essay is derived from a talk at Oscon 2004.)
A few months ago I finished a new 
book, 
and in reviews I keep
noticing words like "provocative'' and "controversial.'' To say
nothing of "idiotic.''I didn't mean to make the book controversial.  I was trying to make
it efficient.  I didn't want to waste people's time telling them
things they already knew.  It's more efficient just to give them
the diffs.  But I suppose that's bound to yield an alarming book.EdisonsThere's no controversy about which idea is most controversial:
the suggestion that variation in wealth might not be as big a
problem as we think.I didn't say in the book that variation in wealth was in itself a
good thing.  I said in some situations it might be a sign of good
things.  A throbbing headache is not a good thing, but it can be
a sign of a good thing-- for example, that you're recovering
consciousness after being hit on the head.Variation in wealth can be a sign of variation in productivity.
(In a society of one, they're identical.) And that
is almost certainly a good thing: if your society has no variation
in productivity, it's probably not because everyone is Thomas
Edison.  It's probably because you have no Thomas Edisons.In a low-tech society you don't see much variation in productivity.
If you have a tribe of nomads collecting sticks for a fire, how
much more productive is the best stick gatherer going to be than
the worst?  A factor of two?  Whereas when you hand people a complex tool
like a computer, the variation in what they can do with
it is enormous.That's not a new idea.  Fred Brooks wrote about it in 1974, and
the study he quoted was published in 1968.  But I think he
underestimated the variation between programmers.  He wrote about productivity in lines
of code:  the best programmers can solve a given problem in a tenth
the time.  But what if the problem isn't given? In programming, as
in many fields, the hard part isn't solving problems, but deciding
what problems to solve.  Imagination is hard to measure, but
in practice it dominates the kind of productivity that's measured
in lines of code.Productivity varies in any field, but there are few in which it
varies so much.  The variation between programmers
is so great that it becomes a difference in kind.  I don't
think this is something intrinsic to programming, though.  In every field,
technology magnifies differences in productivity.  I think what's
happening in programming is just that we have a lot of technological
leverage.  But in every field the lever is getting longer, so the
variation we see is something that more and more fields will see
as time goes on.  And the success of companies, and countries, will
depend increasingly on how they deal with it.If variation in productivity increases with technology"""

In [5]:
Long_Context_4K = """<s> 
The best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.
<s><s>  

Want to start a startup?  Get funded by
Y Combinator.




July 2004(This essay is derived from a talk at Oscon 2004.)
A few months ago I finished a new 
book, 
and in reviews I keep
noticing words like "provocative'' and "controversial.'' To say
nothing of "idiotic.''I didn't mean to make the book controversial.  I was trying to make
it efficient.  I didn't want to waste people's time telling them
things they already knew.  It's more efficient just to give them
the diffs.  But I suppose that's bound to yield an alarming book.EdisonsThere's no controversy about which idea is most controversial:
the suggestion that variation in wealth might not be as big a
problem as we think.I didn't say in the book that variation in wealth was in itself a
good thing.  I said in some situations it might be a sign of good
things.  A throbbing headache is not a good thing, but it can be
a sign of a good thing-- for example, that you're recovering
consciousness after being hit on the head.Variation in wealth can be a sign of variation in productivity.
(In a society of one, they're identical.) And that
is almost certainly a good thing: if your society has no variation
in productivity, it's probably not because everyone is Thomas
Edison.  It's probably because you have no Thomas Edisons.In a low-tech society you don't see much variation in productivity.
If you have a tribe of nomads collecting sticks for a fire, how
much more productive is the best stick gatherer going to be than
the worst?  A factor of two?  Whereas when you hand people a complex tool
like a computer, the variation in what they can do with
it is enormous.That's not a new idea.  Fred Brooks wrote about it in 1974, and
the study he quoted was published in 1968.  But I think he
underestimated the variation between programmers.  He wrote about productivity in lines
of code:  the best programmers can solve a given problem in a tenth
the time.  But what if the problem isn't given? In programming, as
in many fields, the hard part isn't solving problems, but deciding
what problems to solve.  Imagination is hard to measure, but
in practice it dominates the kind of productivity that's measured
in lines of code.Productivity varies in any field, but there are few in which it
varies so much.  The variation between programmers
is so great that it becomes a difference in kind.  I don't
think this is something intrinsic to programming, though.  In every field,
technology magnifies differences in productivity.  I think what's
happening in programming is just that we have a lot of technological
leverage.  But in every field the lever is getting longer, so the
variation we see is something that more and more fields will see
as time goes on.  And the success of companies, and countries, will
depend increasingly on how they deal with it.If variation in productivity increases with technology, then the
contribution of the most productive individuals will not only be
disproportionately large, but will actually grow with time.  When
you reach the point where 90% of a group's output is created by 1%
of its members, you lose big if something (whether Viking raids,
or central planning) drags their productivity down to the average.If we want to get the most out of them, we need to understand these
especially productive people.  What motivates them?  What do they
need to do their jobs?  How do you recognize them? How do you
get them to come and work for you?  And then of course there's the
question, how do you become one?More than MoneyI know a handful of super-hackers, so I sat down and thought about
what they have in common.  Their defining quality is probably that
they really love to program.  Ordinary programmers write code to pay
the bills.  Great hackers think of it as something they do for fun,
and which they're delighted to find people will pay them for.Great programmers are sometimes said to be indifferent to money.
This isn't quite true.  It is true that all they really care about
is doing interesting work.  But if you make enough money, you get
to work on whatever you want, and for that reason hackers are
attracted by the idea of making really large amounts of money.
But as long as they still have to show up for work every day, they
care more about what they do there than how much they get paid for
it.Economically, this is a fact of the greatest importance, because
it means you don't have to pay great hackers anything like what
they're worth.  A great programmer might be ten or a hundred times
as productive as an ordinary one, but he'll consider himself lucky
to get paid three times as much.  As I'll explain later, this is
partly because great hackers don't know how good they are.  But
it's also because money is not the main thing they want.What do hackers want?  Like all craftsmen, hackers like good tools.
In fact, that's an understatement.  Good hackers find it unbearable
to use bad tools.  They'll simply refuse to work on projects with
the wrong infrastructure.At a startup I once worked for, one of the things pinned up on our
bulletin board was an ad from IBM.  It was a picture of an AS400,
and the headline read, I think, "hackers despise
it.'' [1]When you decide what infrastructure to use for a project, you're
not just making a technical decision.  You're also making a social
decision, and this may be the more important of the two.  For
example, if your company wants to write some software, it might
seem a prudent choice to write it in Java.  But when you choose a
language, you're also choosing a community.  The programmers you'll
be able to hire to work on a Java project won't be as
smart as the
ones you could get to work on a project written in Python.
And the quality of your hackers probably matters more than the
language you choose.  Though, frankly, the fact that good hackers
prefer Python to Java should tell you something about the relative
merits of those languages.Business types prefer the most popular languages because they view
languages as standards. They don't want to bet the company on
Betamax.  The thing about languages, though, is that they're not
just standards.  If you have to move bits over a network, by all
means use TCP/IP.  But a programming language isn't just a format.
A programming language is a medium of expression.I've read that Java has just overtaken Cobol as the most popular
language.  As a standard, you couldn't wish for more.  But as a
medium of expression, you could do a lot better.  Of all the great
programmers I can think of, I know of only one who would voluntarily
program in Java.  And of all the great programmers I can think of
who don't work for Sun, on Java, I know of zero.Great hackers also generally insist on using open source software.
Not just because it's better, but because it gives them more control.
Good hackers insist on control.  This is part of what makes them
good hackers:  when something's broken, they need to fix it.  You
want them to feel this way about the software they're writing for
you.  You shouldn't be surprised when they feel the same way about
the operating system.A couple years ago a venture capitalist friend told me about a new
startup he was involved with.  It sounded promising.  But the next
time I talked to him, he said they'd decided to build their software
on Windows NT, and had just hired a very experienced NT developer
to be their chief technical officer.  When I heard this, I thought,
these guys are doomed.  One, the CTO couldn't be a first rate
hacker, because to become an eminent NT developer he would have
had to use NT voluntarily, multiple times, and I couldn't imagine
a great hacker doing that; and two, even if he was good, he'd have
a hard time hiring anyone good to work for him if the project had
to be built on NT. [2]The Final FrontierAfter software, the most important tool to a hacker is probably
his office.  Big companies think the function of office space is to express
rank.  But hackers use their offices for more than that: they
use their office as a place to think in.  And if you're a technology
company, their thoughts are your product.  So making hackers work
in a noisy, distracting environment is like having a paint factory
where the air is full of soot.The cartoon strip Dilbert has a lot to say about cubicles, and with
good reason.  All the hackers I know despise them.  The mere prospect
of being interrupted is enough to prevent hackers from working on
hard problems.  If you want to get real work done in an office with
cubicles, you have two options: work at home, or come in early or
late or on a weekend, when no one else is there.  Don't companies
realize this is a sign that something is broken?  An office
environment is supposed to be something that helps
you work, not something you work despite.Companies like Cisco are proud that everyone there has a cubicle,
even the CEO.  But they're not so advanced as they think; obviously
they still view office space as a badge of rank.  Note too that
Cisco is famous for doing very little product development in house.
They get new technology by buying the startups that created it-- where
presumably the hackers did have somewhere quiet to work.One big company that understands what hackers need is Microsoft.
I once saw a recruiting ad for Microsoft with a big picture of a
door.  Work for us, the premise was, and we'll give you a place to
work where you can actually get work done.   And you know, Microsoft
is remarkable among big companies in that they are able to develop
software in house.  Not well, perhaps, but well enough.If companies want hackers to be productive, they should look at
what they do at home.  At home, hackers can arrange things themselves
so they can get the most done.  And when they work at home, hackers
don't work in noisy, open spaces; they work in rooms with doors.  They
work in cosy, neighborhoody places with people around and somewhere
to walk when they need to mull something over, instead of in glass
boxes set in acres of parking lots.  They have a sofa they can take
a nap on when they feel tired, instead of sitting in a coma at
their desk, pretending to work.  There's no crew of people with
vacuum cleaners that roars through every evening during the prime
hacking hours.  There are no meetings or, God forbid, corporate
retreats or team-building exercises.  And when you look at what
they're doing on that computer, you'll find it reinforces what I
said earlier about tools.  They may have to use Java and Windows
at work, but at home, where they can choose for themselves, you're
more likely to find them using Perl and Linux.Indeed, these statistics about Cobol or Java being the most popular
language can be misleading.  What we ought to look at, if we want
to know what tools are best, is what hackers choose when they can
choose freely-- that is, in projects of their own.  When you ask
that question, you find that open source operating systems already
have a dominant market share, and the number one language is probably
Perl.InterestingAlong with good tools, hackers want interesting projects.  What
makes a project interesting?  Well, obviously overtly sexy
applications like stealth planes or special effects software would
be interesting to work on.  But any application can be interesting
if it poses novel technical challenges.  So it's hard to predict
which problems hackers will like, because some become
interesting only when the people working on them discover a new
kind of solution.  Before ITA
(who wrote the software inside Orbitz),
the people working on airline fare searches probably thought it
was one of the most boring applications imaginable.  But ITA made
it interesting by 
redefining the problem in a more ambitious way.I think the same thing happened at Google.  When Google was founded,
the conventional wisdom among the so-called portals was that search
was boring and unimportant.  But the guys at Google didn't think
search was boring, and that's why they do it so well.This is an area where managers can make a difference.  Like a parent
saying to a child, I bet you can't clean up your whole room in
ten minutes, a good manager can sometimes redefine a problem as a
more interesting one.  Steve Jobs seems to be particularly good at
this, in part simply by having high standards.  There were a lot
of small, inexpensive computers before the Mac.  He redefined the
problem as: make one that's beautiful.  And that probably drove
the developers harder than any carrot or stick could.They certainly delivered.  When the Mac first appeared, you didn't
even have to turn it on to know it would be good; you could tell
from the case.  A few weeks ago I was walking along the street in
Cambridge, and in someone's trash I saw what appeared to be a Mac
carrying case.  I looked inside, and there was a Mac SE.  I carried
it home and plugged it in, and it booted.  The happy Macintosh
face, and then the finder.  My God, it was so simple.  It was just
like ... Google.Hackers like to work for people with high standards.  But it's not
enough just to be exacting.  You have to insist on the right things.
Which usually means that you have to be a hacker yourself.  I've
seen occasional articles about how to manage programmers.  Really
there should be two articles: one about what to do if
you are yourself a programmer, and one about what to do if you're not.  And the 
second could probably be condensed into two words:  give up.The problem is not so much the day to day management.  Really good
hackers are practically self-managing.  The problem is, if you're
not a hacker, you can't tell who the good hackers are.  A similar
problem explains why American cars are so ugly.  I call it the
design paradox.  You might think that you could make your products
beautiful just by hiring a great designer to design them.  But if
you yourself don't have good taste, 
how are you going to recognize
a good designer?  By definition you can't tell from his portfolio.
And you can't go by the awards he's won or the jobs he's had,
because in design, as in most fields, those tend to be driven by
fashion and schmoozing, with actual ability a distant third.
There's no way around it:  you can't manage a process intended to
produce beautiful things without knowing what beautiful is.  American
cars are ugly because American car companies are run by people with
bad taste.Many people in this country think of taste as something elusive,
or even frivolous.  It is neither.  To drive design, a manager must
be the most demanding user of a company's products.  And if you
have really good taste, you can, as Steve Jobs does, make satisfying
you the kind of problem that good people like to work on.Nasty Little ProblemsIt's pretty easy to say what kinds of problems are not interesting:
those where instead of solving a few big, clear, problems, you have
to solve a lot of nasty little ones.  One of the worst kinds of
projects is writing an interface to a piece of software that's
full of bugs.  Another is when you have to customize
something for an individual client's complex and ill-defined needs.
To hackers these kinds of projects are the death of a thousand
cuts.The distinguishing feature of nasty little problems is that you
don't learn anything from them.   Writing a compiler is interesting
because it teaches you what a compiler is.  But writing an interface
to a buggy piece of software doesn't teach you anything, because the
bugs are random.  [3] So it's not just fastidiousness that makes good
"""

In [6]:
Long_Context_14K = """<s> 
The best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.
<s><s>  

Want to start a startup?  Get funded by
Y Combinator.




July 2004(This essay is derived from a talk at Oscon 2004.)
A few months ago I finished a new 
book, 
and in reviews I keep
noticing words like "provocative'' and "controversial.'' To say
nothing of "idiotic.''I didn't mean to make the book controversial.  I was trying to make
it efficient.  I didn't want to waste people's time telling them
things they already knew.  It's more efficient just to give them
the diffs.  But I suppose that's bound to yield an alarming book.EdisonsThere's no controversy about which idea is most controversial:
the suggestion that variation in wealth might not be as big a
problem as we think.I didn't say in the book that variation in wealth was in itself a
good thing.  I said in some situations it might be a sign of good
things.  A throbbing headache is not a good thing, but it can be
a sign of a good thing-- for example, that you're recovering
consciousness after being hit on the head.Variation in wealth can be a sign of variation in productivity.
(In a society of one, they're identical.) And that
is almost certainly a good thing: if your society has no variation
in productivity, it's probably not because everyone is Thomas
Edison.  It's probably because you have no Thomas Edisons.In a low-tech society you don't see much variation in productivity.
If you have a tribe of nomads collecting sticks for a fire, how
much more productive is the best stick gatherer going to be than
the worst?  A factor of two?  Whereas when you hand people a complex tool
like a computer, the variation in what they can do with
it is enormous.That's not a new idea.  Fred Brooks wrote about it in 1974, and
the study he quoted was published in 1968.  But I think he
underestimated the variation between programmers.  He wrote about productivity in lines
of code:  the best programmers can solve a given problem in a tenth
the time.  But what if the problem isn't given? In programming, as
in many fields, the hard part isn't solving problems, but deciding
what problems to solve.  Imagination is hard to measure, but
in practice it dominates the kind of productivity that's measured
in lines of code.Productivity varies in any field, but there are few in which it
varies so much.  The variation between programmers
is so great that it becomes a difference in kind.  I don't
think this is something intrinsic to programming, though.  In every field,
technology magnifies differences in productivity.  I think what's
happening in programming is just that we have a lot of technological
leverage.  But in every field the lever is getting longer, so the
variation we see is something that more and more fields will see
as time goes on.  And the success of companies, and countries, will
depend increasingly on how they deal with it.If variation in productivity increases with technology, then the
contribution of the most productive individuals will not only be
disproportionately large, but will actually grow with time.  When
you reach the point where 90% of a group's output is created by 1%
of its members, you lose big if something (whether Viking raids,
or central planning) drags their productivity down to the average.If we want to get the most out of them, we need to understand these
especially productive people.  What motivates them?  What do they
need to do their jobs?  How do you recognize them? How do you
get them to come and work for you?  And then of course there's the
question, how do you become one?More than MoneyI know a handful of super-hackers, so I sat down and thought about
what they have in common.  Their defining quality is probably that
they really love to program.  Ordinary programmers write code to pay
the bills.  Great hackers think of it as something they do for fun,
and which they're delighted to find people will pay them for.Great programmers are sometimes said to be indifferent to money.
This isn't quite true.  It is true that all they really care about
is doing interesting work.  But if you make enough money, you get
to work on whatever you want, and for that reason hackers are
attracted by the idea of making really large amounts of money.
But as long as they still have to show up for work every day, they
care more about what they do there than how much they get paid for
it.Economically, this is a fact of the greatest importance, because
it means you don't have to pay great hackers anything like what
they're worth.  A great programmer might be ten or a hundred times
as productive as an ordinary one, but he'll consider himself lucky
to get paid three times as much.  As I'll explain later, this is
partly because great hackers don't know how good they are.  But
it's also because money is not the main thing they want.What do hackers want?  Like all craftsmen, hackers like good tools.
In fact, that's an understatement.  Good hackers find it unbearable
to use bad tools.  They'll simply refuse to work on projects with
the wrong infrastructure.At a startup I once worked for, one of the things pinned up on our
bulletin board was an ad from IBM.  It was a picture of an AS400,
and the headline read, I think, "hackers despise
it.'' [1]When you decide what infrastructure to use for a project, you're
not just making a technical decision.  You're also making a social
decision, and this may be the more important of the two.  For
example, if your company wants to write some software, it might
seem a prudent choice to write it in Java.  But when you choose a
language, you're also choosing a community.  The programmers you'll
be able to hire to work on a Java project won't be as
smart as the
ones you could get to work on a project written in Python.
And the quality of your hackers probably matters more than the
language you choose.  Though, frankly, the fact that good hackers
prefer Python to Java should tell you something about the relative
merits of those languages.Business types prefer the most popular languages because they view
languages as standards. They don't want to bet the company on
Betamax.  The thing about languages, though, is that they're not
just standards.  If you have to move bits over a network, by all
means use TCP/IP.  But a programming language isn't just a format.
A programming language is a medium of expression.I've read that Java has just overtaken Cobol as the most popular
language.  As a standard, you couldn't wish for more.  But as a
medium of expression, you could do a lot better.  Of all the great
programmers I can think of, I know of only one who would voluntarily
program in Java.  And of all the great programmers I can think of
who don't work for Sun, on Java, I know of zero.Great hackers also generally insist on using open source software.
Not just because it's better, but because it gives them more control.
Good hackers insist on control.  This is part of what makes them
good hackers:  when something's broken, they need to fix it.  You
want them to feel this way about the software they're writing for
you.  You shouldn't be surprised when they feel the same way about
the operating system.A couple years ago a venture capitalist friend told me about a new
startup he was involved with.  It sounded promising.  But the next
time I talked to him, he said they'd decided to build their software
on Windows NT, and had just hired a very experienced NT developer
to be their chief technical officer.  When I heard this, I thought,
these guys are doomed.  One, the CTO couldn't be a first rate
hacker, because to become an eminent NT developer he would have
had to use NT voluntarily, multiple times, and I couldn't imagine
a great hacker doing that; and two, even if he was good, he'd have
a hard time hiring anyone good to work for him if the project had
to be built on NT. [2]The Final FrontierAfter software, the most important tool to a hacker is probably
his office.  Big companies think the function of office space is to express
rank.  But hackers use their offices for more than that: they
use their office as a place to think in.  And if you're a technology
company, their thoughts are your product.  So making hackers work
in a noisy, distracting environment is like having a paint factory
where the air is full of soot.The cartoon strip Dilbert has a lot to say about cubicles, and with
good reason.  All the hackers I know despise them.  The mere prospect
of being interrupted is enough to prevent hackers from working on
hard problems.  If you want to get real work done in an office with
cubicles, you have two options: work at home, or come in early or
late or on a weekend, when no one else is there.  Don't companies
realize this is a sign that something is broken?  An office
environment is supposed to be something that helps
you work, not something you work despite.Companies like Cisco are proud that everyone there has a cubicle,
even the CEO.  But they're not so advanced as they think; obviously
they still view office space as a badge of rank.  Note too that
Cisco is famous for doing very little product development in house.
They get new technology by buying the startups that created it-- where
presumably the hackers did have somewhere quiet to work.One big company that understands what hackers need is Microsoft.
I once saw a recruiting ad for Microsoft with a big picture of a
door.  Work for us, the premise was, and we'll give you a place to
work where you can actually get work done.   And you know, Microsoft
is remarkable among big companies in that they are able to develop
software in house.  Not well, perhaps, but well enough.If companies want hackers to be productive, they should look at
what they do at home.  At home, hackers can arrange things themselves
so they can get the most done.  And when they work at home, hackers
don't work in noisy, open spaces; they work in rooms with doors.  They
work in cosy, neighborhoody places with people around and somewhere
to walk when they need to mull something over, instead of in glass
boxes set in acres of parking lots.  They have a sofa they can take
a nap on when they feel tired, instead of sitting in a coma at
their desk, pretending to work.  There's no crew of people with
vacuum cleaners that roars through every evening during the prime
hacking hours.  There are no meetings or, God forbid, corporate
retreats or team-building exercises.  And when you look at what
they're doing on that computer, you'll find it reinforces what I
said earlier about tools.  They may have to use Java and Windows
at work, but at home, where they can choose for themselves, you're
more likely to find them using Perl and Linux.Indeed, these statistics about Cobol or Java being the most popular
language can be misleading.  What we ought to look at, if we want
to know what tools are best, is what hackers choose when they can
choose freely-- that is, in projects of their own.  When you ask
that question, you find that open source operating systems already
have a dominant market share, and the number one language is probably
Perl.InterestingAlong with good tools, hackers want interesting projects.  What
makes a project interesting?  Well, obviously overtly sexy
applications like stealth planes or special effects software would
be interesting to work on.  But any application can be interesting
if it poses novel technical challenges.  So it's hard to predict
which problems hackers will like, because some become
interesting only when the people working on them discover a new
kind of solution.  Before ITA
(who wrote the software inside Orbitz),
the people working on airline fare searches probably thought it
was one of the most boring applications imaginable.  But ITA made
it interesting by 
redefining the problem in a more ambitious way.I think the same thing happened at Google.  When Google was founded,
the conventional wisdom among the so-called portals was that search
was boring and unimportant.  But the guys at Google didn't think
search was boring, and that's why they do it so well.This is an area where managers can make a difference.  Like a parent
saying to a child, I bet you can't clean up your whole room in
ten minutes, a good manager can sometimes redefine a problem as a
more interesting one.  Steve Jobs seems to be particularly good at
this, in part simply by having high standards.  There were a lot
of small, inexpensive computers before the Mac.  He redefined the
problem as: make one that's beautiful.  And that probably drove
the developers harder than any carrot or stick could.They certainly delivered.  When the Mac first appeared, you didn't
even have to turn it on to know it would be good; you could tell
from the case.  A few weeks ago I was walking along the street in
Cambridge, and in someone's trash I saw what appeared to be a Mac
carrying case.  I looked inside, and there was a Mac SE.  I carried
it home and plugged it in, and it booted.  The happy Macintosh
face, and then the finder.  My God, it was so simple.  It was just
like ... Google.Hackers like to work for people with high standards.  But it's not
enough just to be exacting.  You have to insist on the right things.
Which usually means that you have to be a hacker yourself.  I've
seen occasional articles about how to manage programmers.  Really
there should be two articles: one about what to do if
you are yourself a programmer, and one about what to do if you're not.  And the 
second could probably be condensed into two words:  give up.The problem is not so much the day to day management.  Really good
hackers are practically self-managing.  The problem is, if you're
not a hacker, you can't tell who the good hackers are.  A similar
problem explains why American cars are so ugly.  I call it the
design paradox.  You might think that you could make your products
beautiful just by hiring a great designer to design them.  But if
you yourself don't have good taste, 
how are you going to recognize
a good designer?  By definition you can't tell from his portfolio.
And you can't go by the awards he's won or the jobs he's had,
because in design, as in most fields, those tend to be driven by
fashion and schmoozing, with actual ability a distant third.
There's no way around it:  you can't manage a process intended to
produce beautiful things without knowing what beautiful is.  American
cars are ugly because American car companies are run by people with
bad taste.Many people in this country think of taste as something elusive,
or even frivolous.  It is neither.  To drive design, a manager must
be the most demanding user of a company's products.  And if you
have really good taste, you can, as Steve Jobs does, make satisfying
you the kind of problem that good people like to work on.Nasty Little ProblemsIt's pretty easy to say what kinds of problems are not interesting:
those where instead of solving a few big, clear, problems, you have
to solve a lot of nasty little ones.  One of the worst kinds of
projects is writing an interface to a piece of software that's
full of bugs.  Another is when you have to customize
something for an individual client's complex and ill-defined needs.
To hackers these kinds of projects are the death of a thousand
cuts.The distinguishing feature of nasty little problems is that you
don't learn anything from them.   Writing a compiler is interesting
because it teaches you what a compiler is.  But writing an interface
to a buggy piece of software doesn't teach you anything, because the
bugs are random.  [3] So it's not just fastidiousness that makes good
hackers avoid nasty little problems.  It's more a question of
self-preservation.  Working on nasty little problems makes you
stupid.  Good hackers avoid it for the same reason models avoid
cheeseburgers.Of course some problems inherently have this character.  And because
of supply and demand, they pay especially well.  So a company that
found a way to get great hackers to work on tedious problems would
be very successful.  How would you do it?One place this happens is in startups.  At our startup we had 
Robert Morris working as a system administrator.  That's like having the
Rolling Stones play at a bar mitzvah.  You can't hire that kind of
talent.  But people will do any amount of drudgery for companies
of which they're the founders.  [4]Bigger companies solve the problem by partitioning the company.
They get smart people to work for them by establishing a separate
R&D department where employees don't have to work directly on
customers' nasty little problems. [5] In this model, the research
department functions like a mine. They produce new ideas; maybe
the rest of the company will be able to use them.You may not have to go to this extreme.  
Bottom-up programming
suggests another way to partition the company: have the smart people
work as toolmakers.  If your company makes software to do x, have
one group that builds tools for writing software of that type, and
another that uses these tools to write the applications.  This way
you might be able to get smart people to write 99% of your code,
but still keep them almost as insulated from users as they would
be in a traditional research department.  The toolmakers would have
users, but they'd only be the company's own developers.  [6]If Microsoft used this approach, their software wouldn't be so full
of security holes, because the less smart people writing the actual
applications wouldn't be doing low-level stuff like allocating
memory.  Instead of writing Word directly in C, they'd be plugging
together big Lego blocks of Word-language.  (Duplo, I believe, is
the technical term.)ClumpingAlong with interesting problems, what good hackers like is other
good hackers.  Great hackers tend to clump together-- sometimes
spectacularly so, as at Xerox Parc.   So you won't attract good
hackers in linear proportion to how good an environment you create
for them.  The tendency to clump means it's more like the square
of the environment.  So it's winner take all.  At any given time,
there are only about ten or twenty places where hackers most want to
work, and if you aren't one of them, you won't just have fewer
great hackers, you'll have zero.Having great hackers is not, by itself, enough to make a company
successful.  It works well for Google and ITA, which are two of
the hot spots right now, but it didn't help Thinking Machines or
Xerox.  Sun had a good run for a while, but their business model
is a down elevator.  In that situation, even the best hackers can't
save you.I think, though, that all other things being equal, a company that
can attract great hackers will have a huge advantage.  There are
people who would disagree with this.  When we were making the rounds
of venture capital firms in the 1990s, several told us that software
companies didn't win by writing great software, but through brand,
and dominating channels, and doing the right deals.They really seemed to believe this, and I think I know why.  I
think what a lot of VCs are looking for, at least unconsciously,
is the next Microsoft.  And of course if Microsoft is your model,
you shouldn't be looking for companies that hope to win by writing
great software.  But VCs are mistaken to look for the next Microsoft,
because no startup can be the next Microsoft unless some other
company is prepared to bend over at just the right moment and be
the next IBM.It's a mistake to use Microsoft as a model, because their whole
culture derives from that one lucky break.  Microsoft is a bad data
point.  If you throw them out, you find that good products do tend
to win in the market.  What VCs should be looking for is the next
Apple, or the next Google.I think Bill Gates knows this.  What worries him about Google is
not the power of their brand, but the fact that they have
better hackers. [7]
RecognitionSo who are the great hackers?  How do you know when you meet one?
That turns out to be very hard.  Even hackers can't tell.  I'm
pretty sure now that my friend Trevor Blackwell is a great hacker.
You may have read on Slashdot how he made his 
own Segway.  The
remarkable thing about this project was that he wrote all the
software in one day (in Python, incidentally).For Trevor, that's
par for the course.  But when I first met him, I thought he was a
complete idiot.  He was standing in Robert Morris's office babbling
at him about something or other, and I remember standing behind
him making frantic gestures at Robert to shoo this nut out of his
office so we could go to lunch.  Robert says he misjudged Trevor
at first too.  Apparently when Robert first met him, Trevor had
just begun a new scheme that involved writing down everything about
every aspect of his life on a stack of index cards, which he carried
with him everywhere.  He'd also just arrived from Canada, and had
a strong Canadian accent and a mullet.The problem is compounded by the fact that hackers, despite their
reputation for social obliviousness, sometimes put a good deal of
effort into seeming smart.  When I was in grad school I used to
hang around the MIT AI Lab occasionally. It was kind of intimidating
at first.  Everyone there spoke so fast.  But after a while I
learned the trick of speaking fast.  You don't have to think any
faster; just use twice as many words to say everything.  With this amount of noise in the signal, it's hard to tell good
hackers when you meet them.  I can't tell, even now.  You also
can't tell from their resumes.  It seems like the only way to judge
a hacker is to work with him on something.And this is the reason that high-tech areas 
only happen around universities.  The active ingredient
here is not so much the professors as the students.  Startups grow up
around universities because universities bring together promising young
people and make them work on the same projects.  The
smart ones learn who the other smart ones are, and together
they cook up new projects of their own.Because you can't tell a great hacker except by working with him,
hackers themselves can't tell how good they are.  This is true to
a degree in most fields.  I've found that people who
are great at something are not so much convinced of their own
greatness as mystified at why everyone else seems so incompetent.
But it's particularly hard for hackers to know how good they are,
because it's hard to compare their work.  This is easier in most
other fields.  In the hundred meters, you know in 10 seconds who's
fastest.  Even in math there seems to be a general consensus about
which problems are hard to solve, and what constitutes a good
solution.  But hacking is like writing.  Who can say which of two
novels is better?  Certainly not the authors.With hackers, at least, other hackers can tell.  That's because,
unlike novelists, hackers collaborate on projects.  When you get
to hit a few difficult problems over the net at someone, you learn
pretty quickly how hard they hit them back.  But hackers can't
watch themselves at work.  So if you ask a great hacker how good
he is, he's almost certain to reply, I don't know.  He's not just
being modest.  He really doesn't know.And none of us know, except about people we've actually worked
with.  Which puts us in a weird situation: we don't know who our
heroes should be.  The hackers who become famous tend to become
famous by random accidents of PR.  Occasionally I need to give an
example of a great hacker, and I never know who to use.  The first
names that come to mind always tend to be people I know personally,
but it seems lame to use them.  So, I think, maybe I should say
Richard Stallman, or Linus Torvalds, or Alan Kay, or someone famous
like that.  But I have no idea if these guys are great hackers.
I've never worked with them on anything.If there is a Michael Jordan of hacking, no one knows, including
him.CultivationFinally, the question the hackers have all been wondering about:
how do you become a great hacker?  I don't know if it's possible
to make yourself into one.  But it's certainly possible to do things
that make you stupid, and if you can make yourself stupid, you
can probably make yourself smart too.The key to being a good hacker may be to work on what you like.
When I think about the great hackers I know, one thing they have
in common is the extreme 
difficulty of making them work 
on anything they
don't want to.  I don't know if this is cause or effect; it may be
both.To do something well you have to love it.  
So to the extent you
can preserve hacking as something you love, you're likely to do it
well.  Try to keep the sense of wonder you had about programming at
age 14.  If you're worried that your current job is rotting your
brain, it probably is.The best hackers tend to be smart, of course, but that's true in
a lot of fields.  Is there some quality that's unique to hackers?
I asked some friends, and the number one thing they mentioned was
curiosity.  
I'd always supposed that all smart people were curious--
that curiosity was simply the first derivative of knowledge.  But
apparently hackers are particularly curious, especially about how
things work.  That makes sense, because programs are in effect
giant descriptions of how things work.Several friends mentioned hackers' ability to concentrate-- their
ability, as one put it, to "tune out everything outside their own
heads.''  I've certainly noticed this.  And I've heard several 
hackers say that after drinking even half a beer they can't program at
all.   So maybe hacking does require some special ability to focus.
Perhaps great hackers can load a large amount of context into their
head, so that when they look at a line of code, they see not just
that line but the whole program around it.  John McPhee
wrote that Bill Bradley's success as a basketball player was due
partly to his extraordinary peripheral vision.  "Perfect'' eyesight
means about 47 degrees of vertical peripheral vision.  Bill Bradley
had 70; he could see the basket when he was looking at the floor.
Maybe great hackers have some similar inborn ability.  (I cheat by
using a very dense language, 
which shrinks the court.)This could explain the disconnect over cubicles.  Maybe the people
in charge of facilities, not having any concentration to shatter,
have no idea that working in a cubicle feels to a hacker like having
one's brain in a blender.  (Whereas Bill, if the rumors of autism
are true, knows all too well.)One difference I've noticed between great hackers and smart people
in general is that hackers are more 
politically incorrect.  To the
extent there is a secret handshake among good hackers, it's when they
know one another well enough to express opinions that would get
them stoned to death by the general public.  And I can see why
political incorrectness would be a useful quality in programming.
Programs are very complex and, at least in the hands of good
programmers, very fluid.  In such situations it's helpful to have
a habit of questioning assumptions.Can you cultivate these qualities?  I don't know.  But you can at
least not repress them.  So here is my best shot at a recipe.  If
it is possible to make yourself into a great hacker, the way to do
it may be to make the following deal with yourself: you never have
to work on boring projects (unless your family will starve otherwise),
and in return, you'll never allow yourself to do a half-assed job.
All the great hackers I know seem to have made that deal, though
perhaps none of them had any choice in the matter.Notes
[1] In fairness, I have to say that IBM makes decent hardware.  I
wrote this on an IBM laptop.[2] They did turn out to be doomed.  They shut down a few months
later.[3] I think this is what people mean when they talk
about the "meaning of life."  On the face of it, this seems an 
odd idea.  Life isn't an expression; how could it have meaning?
But it can have a quality that feels a lot like meaning.  In a project
like a compiler, you have to solve a lot of problems, but the problems
all fall into a pattern, as in a signal.  Whereas when the problems
you have to solve are random, they seem like noise.
[4] Einstein at one point worked designing refrigerators. (He had equity.)[5] It's hard to say exactly what constitutes research in the
computer world, but as a first approximation, it's software that
doesn't have users.I don't think it's publication that makes the best hackers want to work
in research departments.  I think it's mainly not having to have a
three hour meeting with a product manager about problems integrating
the Korean version of Word 13.27 with the talking paperclip.[6] Something similar has been happening for a long time in the
construction industry. When you had a house built a couple hundred
years ago, the local builders built everything in it.  But increasingly
what builders do is assemble components designed and manufactured
by someone else.  This has, like the arrival of desktop publishing,
given people the freedom to experiment in disastrous ways, but it
is certainly more efficient.[7] Google is much more dangerous to Microsoft than Netscape was.
Probably more dangerous than any other company has ever been.  Not
least because they're determined to fight.  On their job listing
page, they say that one of their "core values'' is "Don't be evil.''
From a company selling soybean oil or mining equipment, such a
statement would merely be eccentric.  But I think all of us in the
computer world recognize who that is a declaration of war on.Thanks to Jessica Livingston, Robert Morris, and Sarah Harlin
for reading earlier versions of this talk.

Want to start a startup?  Get funded by
Y Combinator.




January 2006To do something well you have to like it.   That idea is not exactly
novel.  We've got it down to four words: "Do what you love."  But
it's not enough just to tell people that.  Doing what you love is
complicated.The very idea is foreign to what most of us learn as kids.  When I
was a kid, it seemed as if work and fun were opposites by definition.
Life had two states: some of the time adults were making you do
things, and that was called work; the rest of the time you could
do what you wanted, and that was called playing.  Occasionally the
things adults made you do were fun, just as, occasionally, playing
wasn't—for example, if you fell and hurt yourself.  But except
for these few anomalous cases, work was pretty much defined as
not-fun.And it did not seem to be an accident. School, it was implied, was
tedious because it was preparation for grownup work.The world then was divided into two groups, grownups and kids.
Grownups, like some kind of cursed race, had to work.  Kids didn't,
but they did have to go to school, which was a dilute version of
work meant to prepare us for the real thing.  Much as we disliked
school, the grownups all agreed that grownup work was worse, and
that we had it easy.Teachers in particular all seemed to believe implicitly that work
was not fun.  Which is not surprising: work wasn't fun for most of
them.  Why did we have to memorize state capitals instead of playing
dodgeball?  For the same reason they had to watch over a bunch of
kids instead of lying on a beach.  You couldn't just do what you
wanted.I'm not saying we should let little kids do whatever they want.
They may have to be made to work on certain things.  But if we make
kids work on dull stuff, it might be wise to tell them that tediousness
is not the defining quality of work, and indeed that the reason
they have to work on dull stuff now is so they can work on more
interesting stuff later.
[1]Once, when I was about 9 or 10, my father told me I could be whatever
I wanted when I grew up, so long as I enjoyed it.  I remember that
precisely because it seemed so anomalous.  It was like being told
to use dry water.  Whatever I thought he meant, I didn't think he
meant work could literally be fun—fun like playing.  It
took me years to grasp that.JobsBy high school, the prospect of an actual job was on the horizon.
Adults would sometimes come to speak to us about their work, or we
would go to see them at work.  It was always understood that they
enjoyed what they did.  In retrospect I think one may have: the
private jet pilot.  But I don't think the bank manager really did.The main reason they all acted as if they enjoyed their work was
presumably the upper-middle class convention that you're supposed
to.  It would not merely be bad for your career to say that you
despised your job, but a social faux-pas.Why is it conventional to pretend to like what you do?  The first
sentence of this essay explains that.  If you have to like something
to do it well, then the most successful people will all like what
they do.  That's where the upper-middle class tradition comes from.
Just as houses all over America are full of 
chairs
that are, without
the owners even knowing it, nth-degree imitations of chairs designed
250 years ago for French kings, conventional attitudes about work
are, without the owners even knowing it, nth-degree imitations of
the attitudes of people who've done great things.What a recipe for alienation.  By the time they reach an age to
think about what they'd like to do, most kids have been thoroughly
misled about the idea of loving one's work.  School has trained
them to regard work as an unpleasant duty.  Having a job is said
to be even more onerous than schoolwork.  And yet all the adults
claim to like what they do.  You can't blame kids for thinking "I
am not like these people; I am not suited to this world."Actually they've been told three lies: the stuff they've been taught
to regard as work in school is not real work; grownup work is not
(necessarily) worse than schoolwork; and many of the adults around
them are lying when they say they like what they do.The most dangerous liars can be the kids' own parents.  If you take
a boring job to give your family a high standard of living, as so
many people do, you risk infecting your kids with the idea that
work is boring. 
[2]
Maybe it would be better for kids in this one
case if parents were not so unselfish.  A parent who set an example
of loving their work might help their kids more than an expensive
house.
[3]It was not till I was in college that the idea of work finally broke
free from the idea of making a living.  Then the important question
became not how to make money, but what to work on.  Ideally these
coincided, but some spectacular boundary cases (like Einstein in
the patent office) proved they weren't identical.The definition of work was now to make some original contribution
to the world, and in the process not to starve.  But after the habit
of so many years my idea of work still included a large component
of pain.  Work still seemed to require discipline, because only
hard problems yielded grand results, and hard problems couldn't
literally be fun.   Surely one had to force oneself to work on them.If you think something's supposed to hurt, you're less likely to
notice if you're doing it wrong.  That about sums up my experience
of graduate school.BoundsHow much are you supposed to like what you do?  Unless you
know that, you don't know when to stop searching. And if, like most
people, you underestimate it, you'll tend to stop searching too
early.  You'll end up doing something chosen for you by your parents,
or the desire to make money, or prestige—or sheer inertia.Here's an upper bound: Do what you love doesn't mean, do what you
would like to do most this second.  Even Einstein probably
had moments when he wanted to have a cup of coffee, but told himself
he ought to finish what he was working on first.It used to perplex me when I read about people who liked what they
did so much that there was nothing they'd rather do.  There didn't
seem to be any sort of work I liked that much.  If I had a
choice of (a) spending the next hour working on something or (b)
be teleported to Rome and spend the next hour wandering about, was
there any sort of work I'd prefer?  Honestly, no.But the fact is, almost anyone would rather, at any given moment,
float about in the Carribbean, or have sex, or eat some delicious
food, than work on hard problems.  The rule about doing what you
love assumes a certain length of time.  It doesn't mean, do what
will make you happiest this second, but what will make you happiest
over some longer period, like a week or a month.Unproductive pleasures pall eventually.  After a while you get tired
of lying on the beach.  If you want to stay happy, you have to do
something.As a lower bound, you have to like your work more than any unproductive
pleasure.  You have to like what you do enough that the concept of
"spare time" seems mistaken.  Which is not to say you have to spend
all your time working.  You can only work so much before you get
tired and start to screw up.  Then you want to do something else—even something mindless.  But you don't regard this time as the
prize and the time you spend working as the pain you endure to earn
it.I put the lower bound there for practical reasons.  If your work
is not your favorite thing to do, you'll have terrible problems
with procrastination.  You'll have to force yourself to work,  and
when you resort to that the results are distinctly inferior.To be happy I think you have to be doing something you not only
enjoy, but admire.  You have to be able to say, at the end, wow,
that's pretty cool.  This doesn't mean you have to make something.
If you learn how to hang glide, or to speak a foreign language
fluently, that will be enough to make you say, for a while at least,
wow, that's pretty cool.  What there has to be is a test.So one thing that falls just short of the standard, I think, is
reading books.  Except for some books in math and the hard sciences,
there's no test of how well you've read a book, and that's why
merely reading books doesn't quite feel like work.  You have to do
something with what you've read to feel productive.I think the best test is one Gino Lee taught me: to try to do things
that would make your friends say wow.  But it probably wouldn't
start to work properly till about age 22, because most people haven't
had a big enough sample to pick friends from before then.SirensWhat you should not do, I think, is worry about the opinion of
anyone beyond your friends.  You shouldn't worry about prestige.
Prestige is the opinion of the rest of the world.  When you can ask
the opinions of people whose judgement you respect, what does it
add to consider the opinions of people you don't even know? 
[4]This is easy advice to give.  It's hard to follow, especially when
you're young.  
[5]
Prestige is like a powerful magnet that warps
even your beliefs about what you enjoy.  It causes you to work not
on what you like, but what you'd like to like.That's what leads people to try to write novels, for example.  They
like reading novels.  They notice that people who write them win
Nobel prizes.  What could be more wonderful, they think, than to
be a novelist?  But liking the idea of being a novelist is not
enough; you have to like the actual work of novel-writing if you're
going to be good at it; you have to like making up elaborate lies.Prestige is just fossilized inspiration.  If you do anything well
enough, you'll make it prestigious.  Plenty of things we now
consider prestigious were anything but at first.  Jazz comes to
mind—though almost any established art form would do.   So just
do what you like, and let prestige take care of itself.Prestige is especially dangerous to the ambitious.  If you want to
make ambitious people waste their time on errands, the way to do
it is to bait the hook with prestige.  That's the recipe for getting
people to give talks, write forewords, serve on committees, be
department heads, and so on.  It might be a good rule simply to
avoid any prestigious task. If it didn't suck, they wouldn't have
had to make it prestigious.Similarly, if you admire two kinds of work equally, but one is more
prestigious, you should probably choose the other.  Your opinions
about what's admirable are always going to be slightly influenced
by prestige, so if the two seem equal to you, you probably have
more genuine admiration for the less prestigious one.The other big force leading people astray is money.  Money by itself
is not that dangerous.  When something pays well but is regarded
with contempt, like telemarketing, or prostitution, or personal
injury litigation, ambitious people aren't tempted by it.  That
kind of work ends up being done by people who are "just trying to
make a living."  (Tip: avoid any field whose practitioners say
this.)  The danger is when money is combined with prestige, as in,
say, corporate law, or medicine.  A comparatively safe and prosperous
career with some automatic baseline prestige is dangerously tempting
to someone young, who hasn't thought much about what they really
like.The test of whether people love what they do is whether they'd do
it even if they weren't paid for it—even if they had to work at
another job to make a living.  How many corporate lawyers would do
their current work if they had to do it for free, in their spare
time, and take day jobs as waiters to support themselves?This test is especially helpful in deciding between different kinds
of academic work, because fields vary greatly in this respect.  Most
good mathematicians would work on math even if there were no jobs
as math professors, whereas in the departments at the other end of
the spectrum, the availability of teaching jobs is the driver:
people would rather be English professors than work in ad agencies,
and publishing papers is the way you compete for such jobs.  Math
would happen without math departments, but it is the existence of
English majors, and therefore jobs teaching them, that calls into
being all those thousands of dreary papers about gender and identity
in the novels of Conrad.  No one does 
that 
kind of thing for fun.The advice of parents will tend to err on the side of money.  It
seems safe to say there are more undergrads who want to be novelists
and whose parents want them to be doctors than who want to be doctors
and whose parents want them to be novelists.  The kids think their
parents are "materialistic." Not necessarily.  All parents tend to
be more conservative for their kids than they would for themselves,
simply because, as parents, they share risks more than rewards.  If
your eight year old son decides to climb a tall tree, or your teenage
daughter decides to date the local bad boy, you won't get a share
in the excitement, but if your son falls, or your daughter gets
pregnant, you'll have to deal with the consequences.DisciplineWith such powerful forces leading us astray, it's not surprising
we find it so hard to discover what we like to work on.  Most people
are doomed in childhood by accepting the axiom that work = pain.
Those who escape this are nearly all lured onto the rocks by prestige
or money.  How many even discover something they love to work on?
A few hundred thousand, perhaps, out of billions.It's hard to find work you love; it must be, if so few do.  So don't
underestimate this task.  And don't feel bad if you haven't succeeded
yet.  In fact, if you admit to yourself that you're discontented,
you're a step ahead of most people, who are still in denial.  If
you're surrounded by colleagues who claim to enjoy work that you
find contemptible, odds are they're lying to themselves.  Not
necessarily, but probably.Although doing great work takes less discipline than people think—because the way to do great work is to find something you like so
much that you don't have to force yourself to do it—finding
work you love does usually require discipline.   Some people are
lucky enough to know what they want to do when they're 12, and just
glide along as if they were on railroad tracks.  But this seems the
exception.  More often people who do great things have careers with
the trajectory of a ping-pong ball.  They go to school to study A,
drop out and get a job doing B, and then become famous for C after
taking it up on the side.Sometimes jumping from one sort of work to another is a sign of
energy, and sometimes it's a sign of laziness.  Are you dropping
out, or boldly carving a new path?  You often can't tell yourself.
Plenty of people who will later do great things seem to be disappointments
early on, when they're trying to find their niche.Is there some test you can use to keep yourself honest?  One is to
try to do a good job at whatever you're doing, even if you don't
like it.  Then at least you'll know you're not using dissatisfaction
as an excuse for being lazy.  Perhaps more importantly, you'll get
into the habit of doing things well.Another test you can use is: always produce.  For example, if you
have a day job you don't take seriously because you plan to be a
novelist, are you producing?  Are you writing pages of fiction,
however bad?  As long as you're producing, you'll know you're not
merely using the hazy vision of the grand novel you plan to write
one day as an opiate.  The view of it will be obstructed by the all
too palpably flawed one you're actually writing."Always produce" is also a heuristic for finding the work you love.
If you subject yourself to that constraint, it will automatically
push you away from things you think you're supposed to work on,
toward things you actually like.  "Always produce" will discover
your life's work the way water, with the aid of gravity, finds the
hole in your roof.Of course, figuring out what you like to work on doesn't mean you
get to work on it.  That's a separate question.  And if you're
ambitious you have to keep them separate: you have to make a conscious
effort to keep your ideas about what you want from being contaminated
by what seems possible. 
[6]It's painful to keep them apart, because it's painful to observe
the gap between them. So most people pre-emptively lower their
expectations.  For example, if you asked random people on the street
if they'd like to be able to draw like Leonardo, you'd find most
would say something like "Oh, I can't draw."  This is more a statement
of intention than fact; it means, I'm not going to try.  Because
the fact is, if you took a random person off the street and somehow
got them to work as hard as they possibly could at drawing for the
next twenty years, they'd get surprisingly far.  But it would require
a great moral effort; it would mean staring failure in the eye every
day for years.  And so to protect themselves people say "I can't."Another related line you often hear is that not everyone can do
work they love—that someone has to do the unpleasant jobs.  Really?
How do you make them?  In the US the only mechanism for forcing
people to do unpleasant jobs is the draft, and that hasn't been
invoked for over 30 years.  All we can do is encourage people to
do unpleasant work, with money and prestige.If there's something people still won't do, it seems as if society
just has to make do without.  That's what happened with domestic
servants.  For millennia that was the canonical example of a job
"someone had to do."  And yet in the mid twentieth century servants
practically disappeared in rich countries, and the rich have just
had to do without.So while there may be some things someone has to do, there's a good
chance anyone saying that about any particular job is mistaken.
Most unpleasant jobs would either get automated or go undone if no
one were willing to do them.Two RoutesThere's another sense of "not everyone can do work they love"
that's all too true, however.  One has to make a living, and it's
hard to get paid for doing work you love.  There are two routes to
that destination:

  The organic route: as you become more eminent, gradually to
  increase the parts of your job that you like at the expense of
  those you don't.The two-job route: to work at things you don't like to get money
  to work on things you do.

The organic route is more common.  It happens naturally to anyone
who does good work.  A young architect has to take whatever work
he can get, but if he does well he'll gradually be in a position
to pick and choose among projects.  The disadvantage of this route
is that it's slow and uncertain.  Even tenure is not real freedom.The two-job route has several variants depending on how long you
work for money at a time.  At one extreme is the "day job," where
you work regular hours at one job to make money, and work on what
you love in your spare time.  At the other extreme you work at
something till you make enough not to 
have to work for money again.The two-job route is less common than the organic route, because
it requires a deliberate choice.  It's also more dangerous.  Life
tends to get more expensive as you get older, so it's easy to get
sucked into working longer than you expected at the money job.
Worse still, anything you work on changes you.  If you work too
long on tedious stuff, it will rot your brain.  And the best paying
jobs are most dangerous, because they require your full attention.The advantage of the two-job route is that it lets you jump over
obstacles.  The landscape of possible jobs isn't flat; there are
walls of varying heights between different kinds of work. 
[7]
The trick of maximizing the parts of your job that you like can get you
from architecture to product design, but not, probably, to music.
If you make money doing one thing and then work on another, you
have more freedom of choice.Which route should you take?  That depends on how sure you are of
what you want to do, how good you are at taking orders, how much
risk you can stand, and the odds that anyone will pay (in your
lifetime) for what you want to do.  If you're sure of the general
area you want to work in and it's something people are likely to
pay you for, then you should probably take the organic route.  But
if you don't know what you want to work on, or don't like to take
orders, you may want to take the two-job route, if you can stand
the risk.Don't decide too soon.  Kids who know early what they want to do
seem impressive, as if they got the answer to some math question
before the other kids.  They have an answer, certainly, but odds
are it's wrong.A friend of mine who is a quite successful doctor complains constantly
about her job.  When people applying to medical school ask her for
advice, she wants to shake them and yell "Don't do it!"  (But she
never does.) How did she get into this fix?  In high school she
already wanted to be a doctor.  And she is so ambitious and determined
that she overcame every obstacle along the way—including,
unfortunately, not liking it.Now she has a life chosen for her by a high-school kid.When you're young, you're given the impression that you'll get
enough information to make each choice before you need to make it.
But this is certainly not so with work.  When you're deciding what
to do, you have to operate on ridiculously incomplete information.
Even in college you get little idea what various types of work are
like.  At best you may have a couple internships, but not all jobs
offer internships, and those that do don't teach you much more about
the work than being a batboy teaches you about playing baseball.In the design of lives, as in the design of most other things, you
get better results if you use flexible media.  So unless you're
fairly sure what you want to do, your best bet may be to choose a
type of work that could turn into either an organic or two-job
career.  That was probably part of the reason I chose computers.
You can be a professor, or make a lot of money, or morph it into
any number of other kinds of work.It's also wise, early on, to seek jobs that let you do many different
things, so you can learn faster what various kinds of work are like.
Conversely, the extreme version of the two-job route is dangerous
because it teaches you so little about what you like.  If you work
hard at being a bond trader for ten years, thinking that you'll
quit and write novels when you have enough money, what happens when
you quit and then discover that you don't actually like writing
novels?Most people would say, I'd take that problem.  Give me a million
dollars and I'll figure out what to do.  But it's harder than it
looks.  Constraints give your life shape.  Remove them and most
people have no idea what to do: look at what happens to those who
win lotteries or inherit money.  Much as everyone thinks they want
financial security, the happiest people are not those who have it,
but those who like what they do.  So a plan that promises freedom
at the expense of knowing what to do"""

In [7]:
Long_Context_30K = """<s> 
The best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.
<s><s>  

Want to start a startup?  Get funded by
Y Combinator.




July 2004(This essay is derived from a talk at Oscon 2004.)
A few months ago I finished a new 
book, 
and in reviews I keep
noticing words like "provocative'' and "controversial.'' To say
nothing of "idiotic.''I didn't mean to make the book controversial.  I was trying to make
it efficient.  I didn't want to waste people's time telling them
things they already knew.  It's more efficient just to give them
the diffs.  But I suppose that's bound to yield an alarming book.EdisonsThere's no controversy about which idea is most controversial:
the suggestion that variation in wealth might not be as big a
problem as we think.I didn't say in the book that variation in wealth was in itself a
good thing.  I said in some situations it might be a sign of good
things.  A throbbing headache is not a good thing, but it can be
a sign of a good thing-- for example, that you're recovering
consciousness after being hit on the head.Variation in wealth can be a sign of variation in productivity.
(In a society of one, they're identical.) And that
is almost certainly a good thing: if your society has no variation
in productivity, it's probably not because everyone is Thomas
Edison.  It's probably because you have no Thomas Edisons.In a low-tech society you don't see much variation in productivity.
If you have a tribe of nomads collecting sticks for a fire, how
much more productive is the best stick gatherer going to be than
the worst?  A factor of two?  Whereas when you hand people a complex tool
like a computer, the variation in what they can do with
it is enormous.That's not a new idea.  Fred Brooks wrote about it in 1974, and
the study he quoted was published in 1968.  But I think he
underestimated the variation between programmers.  He wrote about productivity in lines
of code:  the best programmers can solve a given problem in a tenth
the time.  But what if the problem isn't given? In programming, as
in many fields, the hard part isn't solving problems, but deciding
what problems to solve.  Imagination is hard to measure, but
in practice it dominates the kind of productivity that's measured
in lines of code.Productivity varies in any field, but there are few in which it
varies so much.  The variation between programmers
is so great that it becomes a difference in kind.  I don't
think this is something intrinsic to programming, though.  In every field,
technology magnifies differences in productivity.  I think what's
happening in programming is just that we have a lot of technological
leverage.  But in every field the lever is getting longer, so the
variation we see is something that more and more fields will see
as time goes on.  And the success of companies, and countries, will
depend increasingly on how they deal with it.If variation in productivity increases with technology, then the
contribution of the most productive individuals will not only be
disproportionately large, but will actually grow with time.  When
you reach the point where 90% of a group's output is created by 1%
of its members, you lose big if something (whether Viking raids,
or central planning) drags their productivity down to the average.If we want to get the most out of them, we need to understand these
especially productive people.  What motivates them?  What do they
need to do their jobs?  How do you recognize them? How do you
get them to come and work for you?  And then of course there's the
question, how do you become one?More than MoneyI know a handful of super-hackers, so I sat down and thought about
what they have in common.  Their defining quality is probably that
they really love to program.  Ordinary programmers write code to pay
the bills.  Great hackers think of it as something they do for fun,
and which they're delighted to find people will pay them for.Great programmers are sometimes said to be indifferent to money.
This isn't quite true.  It is true that all they really care about
is doing interesting work.  But if you make enough money, you get
to work on whatever you want, and for that reason hackers are
attracted by the idea of making really large amounts of money.
But as long as they still have to show up for work every day, they
care more about what they do there than how much they get paid for
it.Economically, this is a fact of the greatest importance, because
it means you don't have to pay great hackers anything like what
they're worth.  A great programmer might be ten or a hundred times
as productive as an ordinary one, but he'll consider himself lucky
to get paid three times as much.  As I'll explain later, this is
partly because great hackers don't know how good they are.  But
it's also because money is not the main thing they want.What do hackers want?  Like all craftsmen, hackers like good tools.
In fact, that's an understatement.  Good hackers find it unbearable
to use bad tools.  They'll simply refuse to work on projects with
the wrong infrastructure.At a startup I once worked for, one of the things pinned up on our
bulletin board was an ad from IBM.  It was a picture of an AS400,
and the headline read, I think, "hackers despise
it.'' [1]When you decide what infrastructure to use for a project, you're
not just making a technical decision.  You're also making a social
decision, and this may be the more important of the two.  For
example, if your company wants to write some software, it might
seem a prudent choice to write it in Java.  But when you choose a
language, you're also choosing a community.  The programmers you'll
be able to hire to work on a Java project won't be as
smart as the
ones you could get to work on a project written in Python.
And the quality of your hackers probably matters more than the
language you choose.  Though, frankly, the fact that good hackers
prefer Python to Java should tell you something about the relative
merits of those languages.Business types prefer the most popular languages because they view
languages as standards. They don't want to bet the company on
Betamax.  The thing about languages, though, is that they're not
just standards.  If you have to move bits over a network, by all
means use TCP/IP.  But a programming language isn't just a format.
A programming language is a medium of expression.I've read that Java has just overtaken Cobol as the most popular
language.  As a standard, you couldn't wish for more.  But as a
medium of expression, you could do a lot better.  Of all the great
programmers I can think of, I know of only one who would voluntarily
program in Java.  And of all the great programmers I can think of
who don't work for Sun, on Java, I know of zero.Great hackers also generally insist on using open source software.
Not just because it's better, but because it gives them more control.
Good hackers insist on control.  This is part of what makes them
good hackers:  when something's broken, they need to fix it.  You
want them to feel this way about the software they're writing for
you.  You shouldn't be surprised when they feel the same way about
the operating system.A couple years ago a venture capitalist friend told me about a new
startup he was involved with.  It sounded promising.  But the next
time I talked to him, he said they'd decided to build their software
on Windows NT, and had just hired a very experienced NT developer
to be their chief technical officer.  When I heard this, I thought,
these guys are doomed.  One, the CTO couldn't be a first rate
hacker, because to become an eminent NT developer he would have
had to use NT voluntarily, multiple times, and I couldn't imagine
a great hacker doing that; and two, even if he was good, he'd have
a hard time hiring anyone good to work for him if the project had
to be built on NT. [2]The Final FrontierAfter software, the most important tool to a hacker is probably
his office.  Big companies think the function of office space is to express
rank.  But hackers use their offices for more than that: they
use their office as a place to think in.  And if you're a technology
company, their thoughts are your product.  So making hackers work
in a noisy, distracting environment is like having a paint factory
where the air is full of soot.The cartoon strip Dilbert has a lot to say about cubicles, and with
good reason.  All the hackers I know despise them.  The mere prospect
of being interrupted is enough to prevent hackers from working on
hard problems.  If you want to get real work done in an office with
cubicles, you have two options: work at home, or come in early or
late or on a weekend, when no one else is there.  Don't companies
realize this is a sign that something is broken?  An office
environment is supposed to be something that helps
you work, not something you work despite.Companies like Cisco are proud that everyone there has a cubicle,
even the CEO.  But they're not so advanced as they think; obviously
they still view office space as a badge of rank.  Note too that
Cisco is famous for doing very little product development in house.
They get new technology by buying the startups that created it-- where
presumably the hackers did have somewhere quiet to work.One big company that understands what hackers need is Microsoft.
I once saw a recruiting ad for Microsoft with a big picture of a
door.  Work for us, the premise was, and we'll give you a place to
work where you can actually get work done.   And you know, Microsoft
is remarkable among big companies in that they are able to develop
software in house.  Not well, perhaps, but well enough.If companies want hackers to be productive, they should look at
what they do at home.  At home, hackers can arrange things themselves
so they can get the most done.  And when they work at home, hackers
don't work in noisy, open spaces; they work in rooms with doors.  They
work in cosy, neighborhoody places with people around and somewhere
to walk when they need to mull something over, instead of in glass
boxes set in acres of parking lots.  They have a sofa they can take
a nap on when they feel tired, instead of sitting in a coma at
their desk, pretending to work.  There's no crew of people with
vacuum cleaners that roars through every evening during the prime
hacking hours.  There are no meetings or, God forbid, corporate
retreats or team-building exercises.  And when you look at what
they're doing on that computer, you'll find it reinforces what I
said earlier about tools.  They may have to use Java and Windows
at work, but at home, where they can choose for themselves, you're
more likely to find them using Perl and Linux.Indeed, these statistics about Cobol or Java being the most popular
language can be misleading.  What we ought to look at, if we want
to know what tools are best, is what hackers choose when they can
choose freely-- that is, in projects of their own.  When you ask
that question, you find that open source operating systems already
have a dominant market share, and the number one language is probably
Perl.InterestingAlong with good tools, hackers want interesting projects.  What
makes a project interesting?  Well, obviously overtly sexy
applications like stealth planes or special effects software would
be interesting to work on.  But any application can be interesting
if it poses novel technical challenges.  So it's hard to predict
which problems hackers will like, because some become
interesting only when the people working on them discover a new
kind of solution.  Before ITA
(who wrote the software inside Orbitz),
the people working on airline fare searches probably thought it
was one of the most boring applications imaginable.  But ITA made
it interesting by 
redefining the problem in a more ambitious way.I think the same thing happened at Google.  When Google was founded,
the conventional wisdom among the so-called portals was that search
was boring and unimportant.  But the guys at Google didn't think
search was boring, and that's why they do it so well.This is an area where managers can make a difference.  Like a parent
saying to a child, I bet you can't clean up your whole room in
ten minutes, a good manager can sometimes redefine a problem as a
more interesting one.  Steve Jobs seems to be particularly good at
this, in part simply by having high standards.  There were a lot
of small, inexpensive computers before the Mac.  He redefined the
problem as: make one that's beautiful.  And that probably drove
the developers harder than any carrot or stick could.They certainly delivered.  When the Mac first appeared, you didn't
even have to turn it on to know it would be good; you could tell
from the case.  A few weeks ago I was walking along the street in
Cambridge, and in someone's trash I saw what appeared to be a Mac
carrying case.  I looked inside, and there was a Mac SE.  I carried
it home and plugged it in, and it booted.  The happy Macintosh
face, and then the finder.  My God, it was so simple.  It was just
like ... Google.Hackers like to work for people with high standards.  But it's not
enough just to be exacting.  You have to insist on the right things.
Which usually means that you have to be a hacker yourself.  I've
seen occasional articles about how to manage programmers.  Really
there should be two articles: one about what to do if
you are yourself a programmer, and one about what to do if you're not.  And the 
second could probably be condensed into two words:  give up.The problem is not so much the day to day management.  Really good
hackers are practically self-managing.  The problem is, if you're
not a hacker, you can't tell who the good hackers are.  A similar
problem explains why American cars are so ugly.  I call it the
design paradox.  You might think that you could make your products
beautiful just by hiring a great designer to design them.  But if
you yourself don't have good taste, 
how are you going to recognize
a good designer?  By definition you can't tell from his portfolio.
And you can't go by the awards he's won or the jobs he's had,
because in design, as in most fields, those tend to be driven by
fashion and schmoozing, with actual ability a distant third.
There's no way around it:  you can't manage a process intended to
produce beautiful things without knowing what beautiful is.  American
cars are ugly because American car companies are run by people with
bad taste.Many people in this country think of taste as something elusive,
or even frivolous.  It is neither.  To drive design, a manager must
be the most demanding user of a company's products.  And if you
have really good taste, you can, as Steve Jobs does, make satisfying
you the kind of problem that good people like to work on.Nasty Little ProblemsIt's pretty easy to say what kinds of problems are not interesting:
those where instead of solving a few big, clear, problems, you have
to solve a lot of nasty little ones.  One of the worst kinds of
projects is writing an interface to a piece of software that's
full of bugs.  Another is when you have to customize
something for an individual client's complex and ill-defined needs.
To hackers these kinds of projects are the death of a thousand
cuts.The distinguishing feature of nasty little problems is that you
don't learn anything from them.   Writing a compiler is interesting
because it teaches you what a compiler is.  But writing an interface
to a buggy piece of software doesn't teach you anything, because the
bugs are random.  [3] So it's not just fastidiousness that makes good
hackers avoid nasty little problems.  It's more a question of
self-preservation.  Working on nasty little problems makes you
stupid.  Good hackers avoid it for the same reason models avoid
cheeseburgers.Of course some problems inherently have this character.  And because
of supply and demand, they pay especially well.  So a company that
found a way to get great hackers to work on tedious problems would
be very successful.  How would you do it?One place this happens is in startups.  At our startup we had 
Robert Morris working as a system administrator.  That's like having the
Rolling Stones play at a bar mitzvah.  You can't hire that kind of
talent.  But people will do any amount of drudgery for companies
of which they're the founders.  [4]Bigger companies solve the problem by partitioning the company.
They get smart people to work for them by establishing a separate
R&D department where employees don't have to work directly on
customers' nasty little problems. [5] In this model, the research
department functions like a mine. They produce new ideas; maybe
the rest of the company will be able to use them.You may not have to go to this extreme.  
Bottom-up programming
suggests another way to partition the company: have the smart people
work as toolmakers.  If your company makes software to do x, have
one group that builds tools for writing software of that type, and
another that uses these tools to write the applications.  This way
you might be able to get smart people to write 99% of your code,
but still keep them almost as insulated from users as they would
be in a traditional research department.  The toolmakers would have
users, but they'd only be the company's own developers.  [6]If Microsoft used this approach, their software wouldn't be so full
of security holes, because the less smart people writing the actual
applications wouldn't be doing low-level stuff like allocating
memory.  Instead of writing Word directly in C, they'd be plugging
together big Lego blocks of Word-language.  (Duplo, I believe, is
the technical term.)ClumpingAlong with interesting problems, what good hackers like is other
good hackers.  Great hackers tend to clump together-- sometimes
spectacularly so, as at Xerox Parc.   So you won't attract good
hackers in linear proportion to how good an environment you create
for them.  The tendency to clump means it's more like the square
of the environment.  So it's winner take all.  At any given time,
there are only about ten or twenty places where hackers most want to
work, and if you aren't one of them, you won't just have fewer
great hackers, you'll have zero.Having great hackers is not, by itself, enough to make a company
successful.  It works well for Google and ITA, which are two of
the hot spots right now, but it didn't help Thinking Machines or
Xerox.  Sun had a good run for a while, but their business model
is a down elevator.  In that situation, even the best hackers can't
save you.I think, though, that all other things being equal, a company that
can attract great hackers will have a huge advantage.  There are
people who would disagree with this.  When we were making the rounds
of venture capital firms in the 1990s, several told us that software
companies didn't win by writing great software, but through brand,
and dominating channels, and doing the right deals.They really seemed to believe this, and I think I know why.  I
think what a lot of VCs are looking for, at least unconsciously,
is the next Microsoft.  And of course if Microsoft is your model,
you shouldn't be looking for companies that hope to win by writing
great software.  But VCs are mistaken to look for the next Microsoft,
because no startup can be the next Microsoft unless some other
company is prepared to bend over at just the right moment and be
the next IBM.It's a mistake to use Microsoft as a model, because their whole
culture derives from that one lucky break.  Microsoft is a bad data
point.  If you throw them out, you find that good products do tend
to win in the market.  What VCs should be looking for is the next
Apple, or the next Google.I think Bill Gates knows this.  What worries him about Google is
not the power of their brand, but the fact that they have
better hackers. [7]
RecognitionSo who are the great hackers?  How do you know when you meet one?
That turns out to be very hard.  Even hackers can't tell.  I'm
pretty sure now that my friend Trevor Blackwell is a great hacker.
You may have read on Slashdot how he made his 
own Segway.  The
remarkable thing about this project was that he wrote all the
software in one day (in Python, incidentally).For Trevor, that's
par for the course.  But when I first met him, I thought he was a
complete idiot.  He was standing in Robert Morris's office babbling
at him about something or other, and I remember standing behind
him making frantic gestures at Robert to shoo this nut out of his
office so we could go to lunch.  Robert says he misjudged Trevor
at first too.  Apparently when Robert first met him, Trevor had
just begun a new scheme that involved writing down everything about
every aspect of his life on a stack of index cards, which he carried
with him everywhere.  He'd also just arrived from Canada, and had
a strong Canadian accent and a mullet.The problem is compounded by the fact that hackers, despite their
reputation for social obliviousness, sometimes put a good deal of
effort into seeming smart.  When I was in grad school I used to
hang around the MIT AI Lab occasionally. It was kind of intimidating
at first.  Everyone there spoke so fast.  But after a while I
learned the trick of speaking fast.  You don't have to think any
faster; just use twice as many words to say everything.  With this amount of noise in the signal, it's hard to tell good
hackers when you meet them.  I can't tell, even now.  You also
can't tell from their resumes.  It seems like the only way to judge
a hacker is to work with him on something.And this is the reason that high-tech areas 
only happen around universities.  The active ingredient
here is not so much the professors as the students.  Startups grow up
around universities because universities bring together promising young
people and make them work on the same projects.  The
smart ones learn who the other smart ones are, and together
they cook up new projects of their own.Because you can't tell a great hacker except by working with him,
hackers themselves can't tell how good they are.  This is true to
a degree in most fields.  I've found that people who
are great at something are not so much convinced of their own
greatness as mystified at why everyone else seems so incompetent.
But it's particularly hard for hackers to know how good they are,
because it's hard to compare their work.  This is easier in most
other fields.  In the hundred meters, you know in 10 seconds who's
fastest.  Even in math there seems to be a general consensus about
which problems are hard to solve, and what constitutes a good
solution.  But hacking is like writing.  Who can say which of two
novels is better?  Certainly not the authors.With hackers, at least, other hackers can tell.  That's because,
unlike novelists, hackers collaborate on projects.  When you get
to hit a few difficult problems over the net at someone, you learn
pretty quickly how hard they hit them back.  But hackers can't
watch themselves at work.  So if you ask a great hacker how good
he is, he's almost certain to reply, I don't know.  He's not just
being modest.  He really doesn't know.And none of us know, except about people we've actually worked
with.  Which puts us in a weird situation: we don't know who our
heroes should be.  The hackers who become famous tend to become
famous by random accidents of PR.  Occasionally I need to give an
example of a great hacker, and I never know who to use.  The first
names that come to mind always tend to be people I know personally,
but it seems lame to use them.  So, I think, maybe I should say
Richard Stallman, or Linus Torvalds, or Alan Kay, or someone famous
like that.  But I have no idea if these guys are great hackers.
I've never worked with them on anything.If there is a Michael Jordan of hacking, no one knows, including
him.CultivationFinally, the question the hackers have all been wondering about:
how do you become a great hacker?  I don't know if it's possible
to make yourself into one.  But it's certainly possible to do things
that make you stupid, and if you can make yourself stupid, you
can probably make yourself smart too.The key to being a good hacker may be to work on what you like.
When I think about the great hackers I know, one thing they have
in common is the extreme 
difficulty of making them work 
on anything they
don't want to.  I don't know if this is cause or effect; it may be
both.To do something well you have to love it.  
So to the extent you
can preserve hacking as something you love, you're likely to do it
well.  Try to keep the sense of wonder you had about programming at
age 14.  If you're worried that your current job is rotting your
brain, it probably is.The best hackers tend to be smart, of course, but that's true in
a lot of fields.  Is there some quality that's unique to hackers?
I asked some friends, and the number one thing they mentioned was
curiosity.  
I'd always supposed that all smart people were curious--
that curiosity was simply the first derivative of knowledge.  But
apparently hackers are particularly curious, especially about how
things work.  That makes sense, because programs are in effect
giant descriptions of how things work.Several friends mentioned hackers' ability to concentrate-- their
ability, as one put it, to "tune out everything outside their own
heads.''  I've certainly noticed this.  And I've heard several 
hackers say that after drinking even half a beer they can't program at
all.   So maybe hacking does require some special ability to focus.
Perhaps great hackers can load a large amount of context into their
head, so that when they look at a line of code, they see not just
that line but the whole program around it.  John McPhee
wrote that Bill Bradley's success as a basketball player was due
partly to his extraordinary peripheral vision.  "Perfect'' eyesight
means about 47 degrees of vertical peripheral vision.  Bill Bradley
had 70; he could see the basket when he was looking at the floor.
Maybe great hackers have some similar inborn ability.  (I cheat by
using a very dense language, 
which shrinks the court.)This could explain the disconnect over cubicles.  Maybe the people
in charge of facilities, not having any concentration to shatter,
have no idea that working in a cubicle feels to a hacker like having
one's brain in a blender.  (Whereas Bill, if the rumors of autism
are true, knows all too well.)One difference I've noticed between great hackers and smart people
in general is that hackers are more 
politically incorrect.  To the
extent there is a secret handshake among good hackers, it's when they
know one another well enough to express opinions that would get
them stoned to death by the general public.  And I can see why
political incorrectness would be a useful quality in programming.
Programs are very complex and, at least in the hands of good
programmers, very fluid.  In such situations it's helpful to have
a habit of questioning assumptions.Can you cultivate these qualities?  I don't know.  But you can at
least not repress them.  So here is my best shot at a recipe.  If
it is possible to make yourself into a great hacker, the way to do
it may be to make the following deal with yourself: you never have
to work on boring projects (unless your family will starve otherwise),
and in return, you'll never allow yourself to do a half-assed job.
All the great hackers I know seem to have made that deal, though
perhaps none of them had any choice in the matter.Notes
[1] In fairness, I have to say that IBM makes decent hardware.  I
wrote this on an IBM laptop.[2] They did turn out to be doomed.  They shut down a few months
later.[3] I think this is what people mean when they talk
about the "meaning of life."  On the face of it, this seems an 
odd idea.  Life isn't an expression; how could it have meaning?
But it can have a quality that feels a lot like meaning.  In a project
like a compiler, you have to solve a lot of problems, but the problems
all fall into a pattern, as in a signal.  Whereas when the problems
you have to solve are random, they seem like noise.
[4] Einstein at one point worked designing refrigerators. (He had equity.)[5] It's hard to say exactly what constitutes research in the
computer world, but as a first approximation, it's software that
doesn't have users.I don't think it's publication that makes the best hackers want to work
in research departments.  I think it's mainly not having to have a
three hour meeting with a product manager about problems integrating
the Korean version of Word 13.27 with the talking paperclip.[6] Something similar has been happening for a long time in the
construction industry. When you had a house built a couple hundred
years ago, the local builders built everything in it.  But increasingly
what builders do is assemble components designed and manufactured
by someone else.  This has, like the arrival of desktop publishing,
given people the freedom to experiment in disastrous ways, but it
is certainly more efficient.[7] Google is much more dangerous to Microsoft than Netscape was.
Probably more dangerous than any other company has ever been.  Not
least because they're determined to fight.  On their job listing
page, they say that one of their "core values'' is "Don't be evil.''
From a company selling soybean oil or mining equipment, such a
statement would merely be eccentric.  But I think all of us in the
computer world recognize who that is a declaration of war on.Thanks to Jessica Livingston, Robert Morris, and Sarah Harlin
for reading earlier versions of this talk.

Want to start a startup?  Get funded by
Y Combinator.




January 2006To do something well you have to like it.   That idea is not exactly
novel.  We've got it down to four words: "Do what you love."  But
it's not enough just to tell people that.  Doing what you love is
complicated.The very idea is foreign to what most of us learn as kids.  When I
was a kid, it seemed as if work and fun were opposites by definition.
Life had two states: some of the time adults were making you do
things, and that was called work; the rest of the time you could
do what you wanted, and that was called playing.  Occasionally the
things adults made you do were fun, just as, occasionally, playing
wasn't—for example, if you fell and hurt yourself.  But except
for these few anomalous cases, work was pretty much defined as
not-fun.And it did not seem to be an accident. School, it was implied, was
tedious because it was preparation for grownup work.The world then was divided into two groups, grownups and kids.
Grownups, like some kind of cursed race, had to work.  Kids didn't,
but they did have to go to school, which was a dilute version of
work meant to prepare us for the real thing.  Much as we disliked
school, the grownups all agreed that grownup work was worse, and
that we had it easy.Teachers in particular all seemed to believe implicitly that work
was not fun.  Which is not surprising: work wasn't fun for most of
them.  Why did we have to memorize state capitals instead of playing
dodgeball?  For the same reason they had to watch over a bunch of
kids instead of lying on a beach.  You couldn't just do what you
wanted.I'm not saying we should let little kids do whatever they want.
They may have to be made to work on certain things.  But if we make
kids work on dull stuff, it might be wise to tell them that tediousness
is not the defining quality of work, and indeed that the reason
they have to work on dull stuff now is so they can work on more
interesting stuff later.
[1]Once, when I was about 9 or 10, my father told me I could be whatever
I wanted when I grew up, so long as I enjoyed it.  I remember that
precisely because it seemed so anomalous.  It was like being told
to use dry water.  Whatever I thought he meant, I didn't think he
meant work could literally be fun—fun like playing.  It
took me years to grasp that.JobsBy high school, the prospect of an actual job was on the horizon.
Adults would sometimes come to speak to us about their work, or we
would go to see them at work.  It was always understood that they
enjoyed what they did.  In retrospect I think one may have: the
private jet pilot.  But I don't think the bank manager really did.The main reason they all acted as if they enjoyed their work was
presumably the upper-middle class convention that you're supposed
to.  It would not merely be bad for your career to say that you
despised your job, but a social faux-pas.Why is it conventional to pretend to like what you do?  The first
sentence of this essay explains that.  If you have to like something
to do it well, then the most successful people will all like what
they do.  That's where the upper-middle class tradition comes from.
Just as houses all over America are full of 
chairs
that are, without
the owners even knowing it, nth-degree imitations of chairs designed
250 years ago for French kings, conventional attitudes about work
are, without the owners even knowing it, nth-degree imitations of
the attitudes of people who've done great things.What a recipe for alienation.  By the time they reach an age to
think about what they'd like to do, most kids have been thoroughly
misled about the idea of loving one's work.  School has trained
them to regard work as an unpleasant duty.  Having a job is said
to be even more onerous than schoolwork.  And yet all the adults
claim to like what they do.  You can't blame kids for thinking "I
am not like these people; I am not suited to this world."Actually they've been told three lies: the stuff they've been taught
to regard as work in school is not real work; grownup work is not
(necessarily) worse than schoolwork; and many of the adults around
them are lying when they say they like what they do.The most dangerous liars can be the kids' own parents.  If you take
a boring job to give your family a high standard of living, as so
many people do, you risk infecting your kids with the idea that
work is boring. 
[2]
Maybe it would be better for kids in this one
case if parents were not so unselfish.  A parent who set an example
of loving their work might help their kids more than an expensive
house.
[3]It was not till I was in college that the idea of work finally broke
free from the idea of making a living.  Then the important question
became not how to make money, but what to work on.  Ideally these
coincided, but some spectacular boundary cases (like Einstein in
the patent office) proved they weren't identical.The definition of work was now to make some original contribution
to the world, and in the process not to starve.  But after the habit
of so many years my idea of work still included a large component
of pain.  Work still seemed to require discipline, because only
hard problems yielded grand results, and hard problems couldn't
literally be fun.   Surely one had to force oneself to work on them.If you think something's supposed to hurt, you're less likely to
notice if you're doing it wrong.  That about sums up my experience
of graduate school.BoundsHow much are you supposed to like what you do?  Unless you
know that, you don't know when to stop searching. And if, like most
people, you underestimate it, you'll tend to stop searching too
early.  You'll end up doing something chosen for you by your parents,
or the desire to make money, or prestige—or sheer inertia.Here's an upper bound: Do what you love doesn't mean, do what you
would like to do most this second.  Even Einstein probably
had moments when he wanted to have a cup of coffee, but told himself
he ought to finish what he was working on first.It used to perplex me when I read about people who liked what they
did so much that there was nothing they'd rather do.  There didn't
seem to be any sort of work I liked that much.  If I had a
choice of (a) spending the next hour working on something or (b)
be teleported to Rome and spend the next hour wandering about, was
there any sort of work I'd prefer?  Honestly, no.But the fact is, almost anyone would rather, at any given moment,
float about in the Carribbean, or have sex, or eat some delicious
food, than work on hard problems.  The rule about doing what you
love assumes a certain length of time.  It doesn't mean, do what
will make you happiest this second, but what will make you happiest
over some longer period, like a week or a month.Unproductive pleasures pall eventually.  After a while you get tired
of lying on the beach.  If you want to stay happy, you have to do
something.As a lower bound, you have to like your work more than any unproductive
pleasure.  You have to like what you do enough that the concept of
"spare time" seems mistaken.  Which is not to say you have to spend
all your time working.  You can only work so much before you get
tired and start to screw up.  Then you want to do something else—even something mindless.  But you don't regard this time as the
prize and the time you spend working as the pain you endure to earn
it.I put the lower bound there for practical reasons.  If your work
is not your favorite thing to do, you'll have terrible problems
with procrastination.  You'll have to force yourself to work,  and
when you resort to that the results are distinctly inferior.To be happy I think you have to be doing something you not only
enjoy, but admire.  You have to be able to say, at the end, wow,
that's pretty cool.  This doesn't mean you have to make something.
If you learn how to hang glide, or to speak a foreign language
fluently, that will be enough to make you say, for a while at least,
wow, that's pretty cool.  What there has to be is a test.So one thing that falls just short of the standard, I think, is
reading books.  Except for some books in math and the hard sciences,
there's no test of how well you've read a book, and that's why
merely reading books doesn't quite feel like work.  You have to do
something with what you've read to feel productive.I think the best test is one Gino Lee taught me: to try to do things
that would make your friends say wow.  But it probably wouldn't
start to work properly till about age 22, because most people haven't
had a big enough sample to pick friends from before then.SirensWhat you should not do, I think, is worry about the opinion of
anyone beyond your friends.  You shouldn't worry about prestige.
Prestige is the opinion of the rest of the world.  When you can ask
the opinions of people whose judgement you respect, what does it
add to consider the opinions of people you don't even know? 
[4]This is easy advice to give.  It's hard to follow, especially when
you're young.  
[5]
Prestige is like a powerful magnet that warps
even your beliefs about what you enjoy.  It causes you to work not
on what you like, but what you'd like to like.That's what leads people to try to write novels, for example.  They
like reading novels.  They notice that people who write them win
Nobel prizes.  What could be more wonderful, they think, than to
be a novelist?  But liking the idea of being a novelist is not
enough; you have to like the actual work of novel-writing if you're
going to be good at it; you have to like making up elaborate lies.Prestige is just fossilized inspiration.  If you do anything well
enough, you'll make it prestigious.  Plenty of things we now
consider prestigious were anything but at first.  Jazz comes to
mind—though almost any established art form would do.   So just
do what you like, and let prestige take care of itself.Prestige is especially dangerous to the ambitious.  If you want to
make ambitious people waste their time on errands, the way to do
it is to bait the hook with prestige.  That's the recipe for getting
people to give talks, write forewords, serve on committees, be
department heads, and so on.  It might be a good rule simply to
avoid any prestigious task. If it didn't suck, they wouldn't have
had to make it prestigious.Similarly, if you admire two kinds of work equally, but one is more
prestigious, you should probably choose the other.  Your opinions
about what's admirable are always going to be slightly influenced
by prestige, so if the two seem equal to you, you probably have
more genuine admiration for the less prestigious one.The other big force leading people astray is money.  Money by itself
is not that dangerous.  When something pays well but is regarded
with contempt, like telemarketing, or prostitution, or personal
injury litigation, ambitious people aren't tempted by it.  That
kind of work ends up being done by people who are "just trying to
make a living."  (Tip: avoid any field whose practitioners say
this.)  The danger is when money is combined with prestige, as in,
say, corporate law, or medicine.  A comparatively safe and prosperous
career with some automatic baseline prestige is dangerously tempting
to someone young, who hasn't thought much about what they really
like.The test of whether people love what they do is whether they'd do
it even if they weren't paid for it—even if they had to work at
another job to make a living.  How many corporate lawyers would do
their current work if they had to do it for free, in their spare
time, and take day jobs as waiters to support themselves?This test is especially helpful in deciding between different kinds
of academic work, because fields vary greatly in this respect.  Most
good mathematicians would work on math even if there were no jobs
as math professors, whereas in the departments at the other end of
the spectrum, the availability of teaching jobs is the driver:
people would rather be English professors than work in ad agencies,
and publishing papers is the way you compete for such jobs.  Math
would happen without math departments, but it is the existence of
English majors, and therefore jobs teaching them, that calls into
being all those thousands of dreary papers about gender and identity
in the novels of Conrad.  No one does 
that 
kind of thing for fun.The advice of parents will tend to err on the side of money.  It
seems safe to say there are more undergrads who want to be novelists
and whose parents want them to be doctors than who want to be doctors
and whose parents want them to be novelists.  The kids think their
parents are "materialistic." Not necessarily.  All parents tend to
be more conservative for their kids than they would for themselves,
simply because, as parents, they share risks more than rewards.  If
your eight year old son decides to climb a tall tree, or your teenage
daughter decides to date the local bad boy, you won't get a share
in the excitement, but if your son falls, or your daughter gets
pregnant, you'll have to deal with the consequences.DisciplineWith such powerful forces leading us astray, it's not surprising
we find it so hard to discover what we like to work on.  Most people
are doomed in childhood by accepting the axiom that work = pain.
Those who escape this are nearly all lured onto the rocks by prestige
or money.  How many even discover something they love to work on?
A few hundred thousand, perhaps, out of billions.It's hard to find work you love; it must be, if so few do.  So don't
underestimate this task.  And don't feel bad if you haven't succeeded
yet.  In fact, if you admit to yourself that you're discontented,
you're a step ahead of most people, who are still in denial.  If
you're surrounded by colleagues who claim to enjoy work that you
find contemptible, odds are they're lying to themselves.  Not
necessarily, but probably.Although doing great work takes less discipline than people think—because the way to do great work is to find something you like so
much that you don't have to force yourself to do it—finding
work you love does usually require discipline.   Some people are
lucky enough to know what they want to do when they're 12, and just
glide along as if they were on railroad tracks.  But this seems the
exception.  More often people who do great things have careers with
the trajectory of a ping-pong ball.  They go to school to study A,
drop out and get a job doing B, and then become famous for C after
taking it up on the side.Sometimes jumping from one sort of work to another is a sign of
energy, and sometimes it's a sign of laziness.  Are you dropping
out, or boldly carving a new path?  You often can't tell yourself.
Plenty of people who will later do great things seem to be disappointments
early on, when they're trying to find their niche.Is there some test you can use to keep yourself honest?  One is to
try to do a good job at whatever you're doing, even if you don't
like it.  Then at least you'll know you're not using dissatisfaction
as an excuse for being lazy.  Perhaps more importantly, you'll get
into the habit of doing things well.Another test you can use is: always produce.  For example, if you
have a day job you don't take seriously because you plan to be a
novelist, are you producing?  Are you writing pages of fiction,
however bad?  As long as you're producing, you'll know you're not
merely using the hazy vision of the grand novel you plan to write
one day as an opiate.  The view of it will be obstructed by the all
too palpably flawed one you're actually writing."Always produce" is also a heuristic for finding the work you love.
If you subject yourself to that constraint, it will automatically
push you away from things you think you're supposed to work on,
toward things you actually like.  "Always produce" will discover
your life's work the way water, with the aid of gravity, finds the
hole in your roof.Of course, figuring out what you like to work on doesn't mean you
get to work on it.  That's a separate question.  And if you're
ambitious you have to keep them separate: you have to make a conscious
effort to keep your ideas about what you want from being contaminated
by what seems possible. 
[6]It's painful to keep them apart, because it's painful to observe
the gap between them. So most people pre-emptively lower their
expectations.  For example, if you asked random people on the street
if they'd like to be able to draw like Leonardo, you'd find most
would say something like "Oh, I can't draw."  This is more a statement
of intention than fact; it means, I'm not going to try.  Because
the fact is, if you took a random person off the street and somehow
got them to work as hard as they possibly could at drawing for the
next twenty years, they'd get surprisingly far.  But it would require
a great moral effort; it would mean staring failure in the eye every
day for years.  And so to protect themselves people say "I can't."Another related line you often hear is that not everyone can do
work they love—that someone has to do the unpleasant jobs.  Really?
How do you make them?  In the US the only mechanism for forcing
people to do unpleasant jobs is the draft, and that hasn't been
invoked for over 30 years.  All we can do is encourage people to
do unpleasant work, with money and prestige.If there's something people still won't do, it seems as if society
just has to make do without.  That's what happened with domestic
servants.  For millennia that was the canonical example of a job
"someone had to do."  And yet in the mid twentieth century servants
practically disappeared in rich countries, and the rich have just
had to do without.So while there may be some things someone has to do, there's a good
chance anyone saying that about any particular job is mistaken.
Most unpleasant jobs would either get automated or go undone if no
one were willing to do them.Two RoutesThere's another sense of "not everyone can do work they love"
that's all too true, however.  One has to make a living, and it's
hard to get paid for doing work you love.  There are two routes to
that destination:

  The organic route: as you become more eminent, gradually to
  increase the parts of your job that you like at the expense of
  those you don't.The two-job route: to work at things you don't like to get money
  to work on things you do.

The organic route is more common.  It happens naturally to anyone
who does good work.  A young architect has to take whatever work
he can get, but if he does well he'll gradually be in a position
to pick and choose among projects.  The disadvantage of this route
is that it's slow and uncertain.  Even tenure is not real freedom.The two-job route has several variants depending on how long you
work for money at a time.  At one extreme is the "day job," where
you work regular hours at one job to make money, and work on what
you love in your spare time.  At the other extreme you work at
something till you make enough not to 
have to work for money again.The two-job route is less common than the organic route, because
it requires a deliberate choice.  It's also more dangerous.  Life
tends to get more expensive as you get older, so it's easy to get
sucked into working longer than you expected at the money job.
Worse still, anything you work on changes you.  If you work too
long on tedious stuff, it will rot your brain.  And the best paying
jobs are most dangerous, because they require your full attention.The advantage of the two-job route is that it lets you jump over
obstacles.  The landscape of possible jobs isn't flat; there are
walls of varying heights between different kinds of work. 
[7]
The trick of maximizing the parts of your job that you like can get you
from architecture to product design, but not, probably, to music.
If you make money doing one thing and then work on another, you
have more freedom of choice.Which route should you take?  That depends on how sure you are of
what you want to do, how good you are at taking orders, how much
risk you can stand, and the odds that anyone will pay (in your
lifetime) for what you want to do.  If you're sure of the general
area you want to work in and it's something people are likely to
pay you for, then you should probably take the organic route.  But
if you don't know what you want to work on, or don't like to take
orders, you may want to take the two-job route, if you can stand
the risk.Don't decide too soon.  Kids who know early what they want to do
seem impressive, as if they got the answer to some math question
before the other kids.  They have an answer, certainly, but odds
are it's wrong.A friend of mine who is a quite successful doctor complains constantly
about her job.  When people applying to medical school ask her for
advice, she wants to shake them and yell "Don't do it!"  (But she
never does.) How did she get into this fix?  In high school she
already wanted to be a doctor.  And she is so ambitious and determined
that she overcame every obstacle along the way—including,
unfortunately, not liking it.Now she has a life chosen for her by a high-school kid.When you're young, you're given the impression that you'll get
enough information to make each choice before you need to make it.
But this is certainly not so with work.  When you're deciding what
to do, you have to operate on ridiculously incomplete information.
Even in college you get little idea what various types of work are
like.  At best you may have a couple internships, but not all jobs
offer internships, and those that do don't teach you much more about
the work than being a batboy teaches you about playing baseball.In the design of lives, as in the design of most other things, you
get better results if you use flexible media.  So unless you're
fairly sure what you want to do, your best bet may be to choose a
type of work that could turn into either an organic or two-job
career.  That was probably part of the reason I chose computers.
You can be a professor, or make a lot of money, or morph it into
any number of other kinds of work.It's also wise, early on, to seek jobs that let you do many different
things, so you can learn faster what various kinds of work are like.
Conversely, the extreme version of the two-job route is dangerous
because it teaches you so little about what you like.  If you work
hard at being a bond trader for ten years, thinking that you'll
quit and write novels when you have enough money, what happens when
you quit and then discover that you don't actually like writing
novels?Most people would say, I'd take that problem.  Give me a million
dollars and I'll figure out what to do.  But it's harder than it
looks.  Constraints give your life shape.  Remove them and most
people have no idea what to do: look at what happens to those who
win lotteries or inherit money.  Much as everyone thinks they want
financial security, the happiest people are not those who have it,
but those who like what they do.  So a plan that promises freedom
at the expense of knowing what to do with it may not be as good as
it seems.Whichever route you take, expect a struggle.  Finding work you love
is very difficult.  Most people fail.  Even if you succeed, it's
rare to be free to work on what you want till your thirties or
forties.  But if you have the destination in sight you'll be more
likely to arrive at it.  If you know you can love work, you're in
the home stretch, and if you know what work you love, you're
practically there.Notes[1]
Currently we do the opposite: when we make kids do boring work,
like arithmetic drills, instead of admitting frankly that it's
boring, we try to disguise it with superficial decorations.[2]
One father told me about a related phenomenon: he found himself
concealing from his family how much he liked his work.  When he
wanted to go to work on a saturday, he found it easier to say that
it was because he "had to" for some reason, rather than admitting
he preferred to work than stay home with them.[3]
Something similar happens with suburbs.  Parents move to suburbs
to raise their kids in a safe environment, but suburbs are so dull
and artificial that by the time they're fifteen the kids are convinced
the whole world is boring.[4]
I'm not saying friends should be the only audience for your
work.  The more people you can help, the better.  But friends should
be your compass.[5]
Donald Hall said young would-be poets were mistaken to be so
obsessed with being published.  But you can imagine what it would
do for a 24 year old to get a poem published in The New Yorker.
Now to people he meets at parties he's a real poet.  Actually he's
no better or worse than he was before, but to a clueless audience
like that, the approval of an official authority makes all the
difference.   So it's a harder problem than Hall realizes.  The
reason the young care so much about prestige is that the people
they want to impress are not very discerning.[6]
This is isomorphic to the principle that you should prevent
your beliefs about how things are from being contaminated by how
you wish they were.  Most people let them mix pretty promiscuously.
The continuing popularity of religion is the most visible index of
that.[7]
A more accurate metaphor would be to say that the graph of jobs
is not very well connected.Thanks to Trevor Blackwell, Dan Friedman, Sarah Harlin,
Jessica Livingston, Jackie McDonough, Robert Morris, Peter Norvig, 
David Sloo, and Aaron Swartz
for reading drafts of this.May 2006(This essay is derived from a keynote at Xtech.)Could you reproduce Silicon Valley elsewhere, or is there something
unique about it?It wouldn't be surprising if it were hard to reproduce in other
countries, because you couldn't reproduce it in most of the US
either.  What does it take to make a silicon valley even here?What it takes is the right people.  If you could get the right ten
thousand people to move from Silicon Valley to Buffalo, Buffalo
would become Silicon Valley.  
[1]That's a striking departure from the past.  Up till a couple decades
ago, geography was destiny for cities.  All great cities were located
on waterways, because cities made money by trade, and water was the
only economical way to ship.Now you could make a great city anywhere, if you could get the right
people to move there.  So the question of how to make a silicon
valley becomes: who are the right people, and how do you get them
to move?Two TypesI think you only need two kinds of people to create a technology
hub: rich people and nerds.  They're the limiting reagents in the
reaction that produces startups, because they're the only ones
present when startups get started.  Everyone else will move.Observation bears this out: within the US, towns have become startup
hubs if and only if they have both rich people and nerds.  Few
startups happen in Miami, for example, because although it's full
of rich people, it has few nerds.  It's not the kind of place nerds
like.Whereas Pittsburgh has the opposite problem: plenty of nerds, but
no rich people.  The top US Computer Science departments are said
to be MIT, Stanford, Berkeley, and Carnegie-Mellon.  MIT yielded
Route 128.  Stanford and Berkeley yielded Silicon Valley.  But
Carnegie-Mellon?  The record skips at that point.  Lower down the
list, the University of Washington yielded a high-tech community
in Seattle, and the University of Texas at Austin yielded one in
Austin.  But what happened in Pittsburgh?  And in Ithaca, home of
Cornell, which is also high on the list?I grew up in Pittsburgh and went to college at Cornell, so I can
answer for both.  The weather is terrible,  particularly in winter,
and there's no interesting old city to make up for it, as there is
in Boston.  Rich people don't want to live in Pittsburgh or Ithaca.
So while there are plenty of hackers who could start startups,
there's no one to invest in them.Not BureaucratsDo you really need the rich people?  Wouldn't it work to have the
government invest in the nerds?  No, it would not.  Startup investors
are a distinct type of rich people.  They tend to have a lot of
experience themselves in the technology business.  This (a) helps
them pick the right startups, and (b) means they can supply advice
and connections as well as money.  And the fact that they have a
personal stake in the outcome makes them really pay attention.Bureaucrats by their nature are the exact opposite sort of people
from startup investors. The idea of them making startup investments
is comic.  It would be like mathematicians running Vogue-- or
perhaps more accurately, Vogue editors running a math journal.
[2]Though indeed, most things bureaucrats do, they do badly.   We just
don't notice usually, because they only have to compete against
other bureaucrats.  But as startup investors they'd have to compete
against pros with a great deal more experience and motivation.Even corporations that have in-house VC groups generally forbid
them to make their own investment decisions.  Most are only allowed
to invest in deals where some reputable private VC firm is willing
to act as lead investor.Not BuildingsIf you go to see Silicon Valley, what you'll see are buildings.
But it's the people that make it Silicon Valley, not the buildings.
I read occasionally about attempts to set up "technology
parks" in other places, as if the active ingredient of Silicon
Valley were the office space.  An article about Sophia Antipolis
bragged that companies there included Cisco, Compaq, IBM, NCR, and
Nortel.  Don't the French realize these aren't startups?Building office buildings for technology companies won't get you a
silicon valley, because the key stage in the life of a startup
happens before they want that kind of space.  The key stage is when
they're three guys operating out of an apartment.  Wherever the
startup is when it gets funded, it will stay.  The defining quality
of Silicon Valley is not that Intel or Apple or Google have offices
there, but that they were started there.So if you want to reproduce Silicon Valley, what you need to reproduce
is those two or three founders sitting around a kitchen table
deciding to start a company.  And to reproduce that you need those
people.UniversitiesThe exciting thing is, all you need are the people.  If you could
attract a critical mass of nerds and investors to live somewhere,
you could reproduce Silicon Valley.  And both groups are highly
mobile.  They'll go where life is good.  So what makes a place good
to them?What nerds like is other nerds.  Smart people will go wherever other
smart people are.  And in particular, to great universities.  In
theory there could be other ways to attract them, but so far
universities seem to be indispensable.  Within the US, there are
no technology hubs without first-rate universities-- or at least,
first-rate computer science departments.So if you want to make a silicon valley, you not only need a
university, but one of the top handful in the world.  It has to be
good enough to act as a magnet, drawing the best people from thousands
of miles away.  And that means it has to stand up to existing magnets
like MIT and Stanford.This sounds hard.  Actually it might be easy.  My professor friends,
when they're deciding where they'd like to work, consider one thing
above all: the quality of the other faculty.  What attracts professors
is good colleagues.  So if you managed to recruit, en masse, a
significant number of the best young researchers, you could create
a first-rate university from nothing overnight.  And you could do
that for surprisingly little.  If you paid 200 people hiring bonuses
of $3 million apiece, you could put together a faculty that would
bear comparison with any in the world.  And from that point the
chain reaction would be self-sustaining.  So whatever it costs to
establish a mediocre university, for an additional half billion or
so you could have a great one.  
[3]PersonalityHowever, merely creating a new university would not be enough to
start a silicon valley. The university is just the seed.  It has
to be planted in the right soil, or it won't germinate.  Plant it
in the wrong place, and you just create Carnegie-Mellon.To spawn startups, your university has to be in a town that has
attractions other than the university.  It has to be a place where
investors want to live, and students want to stay after they graduate.The two like much the same things, because most startup investors
are nerds themselves.  So what do nerds look for in a town?  Their
tastes aren't completely different from other people's, because a
lot of the towns they like most in the US are also big tourist
destinations: San Francisco, Boston, Seattle.   But their tastes
can't be quite mainstream either, because they dislike other big
tourist destinations, like New York, Los Angeles, and Las Vegas.There has been a lot written lately about the "creative class." The
thesis seems to be that as wealth derives increasingly from ideas,
cities will prosper only if they attract those who have them.  That
is certainly true; in fact it was the basis of Amsterdam's prosperity
400 years ago.A lot of nerd tastes they share with the creative class in general.
For example, they like well-preserved old neighborhoods instead of
cookie-cutter suburbs, and locally-owned shops and restaurants
instead of national chains.  Like the rest of the creative class,
they want to live somewhere with personality.What exactly is personality?  I think it's the feeling that each
building is the work of a distinct group of people.  A town with
personality is one that doesn't feel mass-produced.  So if you want
to make a startup hub-- or any town to attract the "creative class"--
you probably have to ban large development projects.
When a large tract has been developed by a single organization, you
can always tell. 
[4]Most towns with personality are old, but they don't have to be.
Old towns have two advantages: they're denser, because they were
laid out before cars, and they're more varied, because they were
built one building at a time.  You could have both now.  Just have
building codes that ensure density, and ban large scale developments.A corollary is that you have to keep out the biggest developer of
all: the government.  A government that asks "How can we build a
silicon valley?" has probably ensured failure by the way they framed
the question.  You don't build a silicon valley; you let one grow.NerdsIf you want to attract nerds, you need more than a town with
personality.  You need a town with the right personality.  Nerds
are a distinct subset of the creative class, with different tastes
from the rest.  You can see this most clearly in New York, which
attracts a lot of creative people, but few nerds. 
[5]What nerds like is the kind of town where people walk around smiling.
This excludes LA, where no one walks at all, and also New York,
where people walk, but not smiling. When I was in grad school in
Boston, a friend came to visit from New York.  On the subway back
from the airport she asked "Why is everyone smiling?"  I looked and
they weren't smiling.  They just looked like they were compared to
the facial expressions she was used to.If you've lived in New York, you know where these facial expressions
come from.  It's the kind of place where your mind may be excited,
but your body knows it's having a bad time.  People don't so much
enjoy living there as endure it for the sake of the excitement.
And if you like certain kinds of excitement, New York is incomparable.
It's a hub of glamour, a magnet for all the shorter half-life
isotopes of style and fame.Nerds don't care about glamour, so to them the appeal of New York
is a mystery.  People who like New York will pay a fortune for a
small, dark, noisy apartment in order to live in a town where the
cool people are really cool.  A nerd looks at that deal and sees
only: pay a fortune for a small, dark, noisy apartment.Nerds will pay a premium to live in a town where the smart people
are really smart, but you don't have to pay as much for that.  It's
supply and demand: glamour is popular, so you have to pay a lot for
it.Most nerds like quieter pleasures.  They like cafes instead of
clubs; used bookshops instead of fashionable clothing shops; hiking
instead of dancing; sunlight instead of tall buildings.  A nerd's
idea of paradise is Berkeley or Boulder.YouthIt's the young nerds who start startups, so it's those specifically
the city has to appeal to.  The startup hubs in the US are all
young-feeling towns.  This doesn't mean they have to be new.
Cambridge has the oldest town plan in America, but it feels young
because it's full of students.What you can't have, if you want to create a silicon valley, is a
large, existing population of stodgy people.  It would be a waste
of time to try to reverse the fortunes of a declining industrial town
like Detroit or Philadelphia by trying to encourage startups.  Those
places have too much momentum in the wrong direction.  You're better
off starting with a blank slate in the form of a small town.  Or
better still, if there's a town young people already flock to, that
one.The Bay Area was a magnet for the young and optimistic for decades
before it was associated with technology.  It was a place people
went in search of something new.  And so it became synonymous with
California nuttiness.  There's still a lot of that there.  If you
wanted to start a new fad-- a new way to focus one's "energy," for
example, or a new category of things not to eat-- the Bay Area would
be the place to do it.  But a place that tolerates oddness in the
search for the new is exactly what you want in a startup hub, because
economically that's what startups are.  Most good startup ideas
seem a little crazy; if they were obviously good ideas, someone
would have done them already.(How many people are going to want computers in their houses?
What, another search engine?)That's the connection between technology and liberalism.  Without
exception the high-tech cities in the US are also the most liberal.
But it's not because liberals are smarter that this is so.  It's
because liberal cities tolerate odd ideas, and smart people by
definition have odd ideas.Conversely, a town that gets praised for being "solid" or representing
"traditional values" may be a fine place to live, but it's never
going to succeed as a startup hub.  The 2004 presidential election,
though a disaster in other respects, conveniently supplied us with
a county-by-county 
map of such places.  
[6]To attract the young, a town must have an intact center.  In most
American cities the center has been abandoned, and the growth, if
any, is in the suburbs.  Most American cities have been turned
inside out. But none of the startup hubs has: not San Francisco,
or Boston, or Seattle.  They all have intact centers.
[7]
My guess is that no city with a dead center could be turned into a
startup hub.  Young people don't want to live in the suburbs.Within the US, the two cities I think could most easily be turned
into new silicon valleys are Boulder and Portland.  Both have the
kind of effervescent feel that attracts the young.  They're each
only a great university short of becoming a silicon valley, if they
wanted to.TimeA great university near an attractive town.  Is that all it takes?
That was all it took to make the original Silicon Valley.  Silicon
Valley traces its origins to William Shockley, one of the inventors
of the transistor.  He did the research that won him the Nobel Prize
at Bell Labs, but when he started his own company in 1956 he moved
to Palo Alto to do it.   At the time that was an odd thing to do.
Why did he?  Because he had grown up there and remembered how nice
it was.  Now Palo Alto is suburbia, but then it was a charming
college town-- a charming college town with perfect weather and San
Francisco only an hour away.The companies that rule Silicon Valley now are all descended in
various ways from Shockley Semiconductor.  Shockley was a difficult
man, and in 1957 his top people-- "the traitorous eight"-- left to
start a new company, Fairchild Semiconductor.  Among them were
Gordon Moore and Robert Noyce, who went on to found Intel, and
Eugene Kleiner, who founded the VC firm Kleiner Perkins.  Forty-two
years later, Kleiner Perkins funded Google, and the partner responsible
for the deal was John Doerr, who came to Silicon Valley in 1974 to
work for Intel.So although a lot of the newest companies in Silicon Valley don't
make anything out of silicon, there always seem to be multiple links
back to Shockley.  There's a lesson here: startups beget startups.
People who work for startups start their own.  People who get rich
from startups fund new ones.  I suspect this kind of organic growth
is the only way to produce a startup hub, because it's the only way
to grow the expertise you need.That has two important implications.  The first is that you need
time to grow a silicon valley.  The university you could create in
a couple years, but the startup community around it has to grow
organically.   The cycle time is limited by the time it takes a
company to succeed, which probably averages about five years.The other implication of the organic growth hypothesis is that you
can't be somewhat of a startup hub.  You either have a self-sustaining
chain reaction, or not.  Observation confirms this too: cities
either have a startup scene, or they don't.  There is no middle
ground.  Chicago has the third largest metropolitan area in America.
As source of startups it's negligible compared to Seattle, number 15.The good news is that the initial seed can be quite small.  Shockley
Semiconductor, though itself not very successful, was big enough.
It brought a critical mass of experts in an important new technology
together in a place they liked enough to stay.CompetingOf course, a would-be silicon valley faces an obstacle the original
one didn't: it has to compete with Silicon Valley.  Can that be
done?  Probably.One of Silicon Valley's biggest advantages is its venture capital
firms.  This was not a factor in Shockley's day, because VC funds
didn't exist.  In fact, Shockley Semiconductor and Fairchild
Semiconductor were not startups at all in our sense.  They were
subsidiaries-- of Beckman Instruments and Fairchild Camera and
Instrument respectively.  Those companies were apparently willing
to establish subsidiaries wherever the experts wanted to live.Venture investors, however, prefer to fund startups within an hour's
drive.  For one, they're more likely to notice startups nearby.
But when they do notice startups in other towns they prefer them
to move.  They don't want to have to travel to attend board meetings,
and in any case the odds of succeeding are higher in a startup hub.The centralizing effect of venture firms is a double one: they cause
startups to form around them, and those draw in more startups through
acquisitions.  And although the first may be weakening because it's
now so cheap to start some startups, the second seems as strong as ever.
Three of the most admired
"Web 2.0" companies were started outside the usual startup hubs,
but two of them have already been reeled in through acquisitions.Such centralizing forces make it harder for new silicon valleys to
get started.  But by no means impossible.  Ultimately power rests
with the founders.  A startup with the best people will beat one
with funding from famous VCs, and a startup that was sufficiently
successful would never have to move.  So a town that
could exert enough pull over the right people could resist and
perhaps even surpass Silicon Valley.For all its power, Silicon Valley has a great weakness: the paradise
Shockley found in 1956 is now one giant parking lot.  San Francisco
and Berkeley are great, but they're forty miles away.  Silicon
Valley proper is soul-crushing suburban sprawl.  It
has fabulous weather, which makes it significantly better than the
soul-crushing sprawl of most other American cities.  But a competitor
that managed to avoid sprawl would have real leverage.  All a city
needs is to be the kind of place the next traitorous eight look at
and say "I want to stay here," and that would be enough to get the
chain reaction started.Notes[1]
It's interesting to consider how low this number could be
made.  I suspect five hundred would be enough, even if they could
bring no assets with them.  Probably just thirty, if I could pick them, 
would be enough to turn Buffalo into a significant startup hub.[2]
Bureaucrats manage to allocate research funding moderately
well, but only because (like an in-house VC fund) they outsource
most of the work of selection.  A professor at a famous university
who is highly regarded by his peers will get funding, pretty much
regardless of the proposal.  That wouldn't work for startups, whose
founders aren't sponsored by organizations, and are often unknowns.[3]
You'd have to do it all at once, or at least a whole department
at a time, because people would be more likely to come if they
knew their friends were.  And you should probably start from scratch,
rather than trying to upgrade an existing university, or much energy
would be lost in friction.[4]
Hypothesis: Any plan in which multiple independent buildings
are gutted or demolished to be "redeveloped" as a single project
is a net loss of personality for the city, with the exception of
the conversion of buildings not previously public, like warehouses.[5]
A few startups get started in New York, but less
than a tenth as many per capita as in Boston, and mostly
in less nerdy fields like finance and media.[6]
Some blue counties are false positives (reflecting the
remaining power of Democractic party machines), but there are no
false negatives.  You can safely write off all the red counties.[7]
Some "urban renewal" experts took a shot at destroying Boston's
in the 1960s, leaving the area around city hall a bleak wasteland,
but most neighborhoods successfully resisted them.Thanks to Chris Anderson, Trevor Blackwell, Marc Hedlund,
Jessica Livingston, Robert Morris, Greg Mcadoo, Fred Wilson,
and Stephen Wolfram for
reading drafts of this, and to Ed Dumbill for inviting me to speak.(The second part of this talk became Why Startups
Condense in America.)January 2015Corporate Development, aka corp dev, is the group within companies
that buys other companies. If you're talking to someone from corp
dev, that's why, whether you realize it yet or not.It's usually a mistake to talk to corp dev unless (a) you want to
sell your company right now and (b) you're sufficiently likely to
get an offer at an acceptable price.  In practice that means startups
should only talk to corp dev when they're either doing really well
or really badly.  If you're doing really badly, meaning the company
is about to die, you may as well talk to them, because you have
nothing to lose. And if you're doing really well, you can safely
talk to them, because you both know the price will have to be high,
and if they show the slightest sign of wasting your time, you'll
be confident enough to tell them to get lost.The danger is to companies in the middle.  Particularly to young
companies that are growing fast, but haven't been doing it for long
enough to have grown big yet.  It's usually a mistake for a promising
company less than a year old even to talk to corp dev.But it's a mistake founders constantly make.  When someone from
corp dev wants to meet, the founders tell themselves they should
at least find out what they want.  Besides, they don't want to
offend Big Company by refusing to meet.Well, I'll tell you what they want.  They want to talk about buying
you.  That's what the title "corp dev" means.   So before agreeing
to meet with someone from corp dev, ask yourselves, "Do we want to
sell the company right now?"  And if the answer is no, tell them
"Sorry, but we're focusing on growing the company."  They won't be
offended.  And certainly the founders of Big Company won't be
offended. If anything they'll think more highly of you.  You'll
remind them of themselves.  They didn't sell either; that's why
they're in a position now to buy other companies.
[1]Most founders who get contacted by corp dev already know what it
means.  And yet even when they know what corp dev does and know
they don't want to sell, they take the meeting.  Why do they do it?
The same mix of denial and wishful thinking that underlies most
mistakes founders make. It's flattering to talk to someone who wants
to buy you.  And who knows, maybe their offer will be surprisingly
high.  You should at least see what it is, right?No.  If they were going to send you an offer immediately by email,
sure, you might as well open it.  But that is not how conversations
with corp dev work.  If you get an offer at all, it will be at the
end of a long and unbelievably distracting process.  And if the
offer is surprising, it will be surprisingly low.Distractions are the thing you can least afford in a startup.  And
conversations with corp dev are the worst sort of distraction,
because as well as consuming your attention they undermine your
morale.  One of the tricks to surviving a grueling process is not
to stop and think how tired you are.  Instead you get into a sort
of flow. 
[2]
Imagine what it would do to you if at mile 20 of a
marathon, someone ran up beside you and said "You must feel really
tired.  Would you like to stop and take a rest?"  Conversations
with corp dev are like that but worse, because the suggestion of
stopping gets combined in your mind with the imaginary high price
you think they'll offer.And then you're really in trouble.  If they can, corp dev people
like to turn the tables on you. They like to get you to the point
where you're trying to convince them to buy instead of them trying
to convince you to sell.  And surprisingly often they succeed.This is a very slippery slope, greased with some of the most powerful
forces that can work on founders' minds, and attended by an experienced
professional whose full time job is to push you down it.Their tactics in pushing you down that slope are usually fairly
brutal. Corp dev people's whole job is to buy companies, and they
don't even get to choose which.  The only way their performance is
measured is by how cheaply they can buy you, and the more ambitious
ones will stop at nothing to achieve that. For example, they'll
almost always start with a lowball offer, just to see if you'll
take it. Even if you don't, a low initial offer will demoralize you
and make you easier to manipulate.And that is the most innocent of their tactics. Just wait till
you've agreed on a price and think you have a done deal, and then
they come back and say their boss has vetoed the deal and won't do
it for more than half the agreed upon price. Happens all the time.
If you think investors can behave badly, it's nothing compared to
what corp dev people can do.  Even corp dev people at companies
that are otherwise benevolent.I remember once complaining to a
friend at Google about some nasty trick their corp dev people had
pulled on a YC startup."What happened to Don't be Evil?" I asked."I don't think corp dev got the memo," he replied.The tactics you encounter in M&A conversations can be like nothing
you've experienced in the otherwise comparatively 
upstanding world
of Silicon Valley.  It's as if a chunk of genetic material from the
old-fashioned robber baron business world got incorporated into the
startup world.
[3]The simplest way to protect yourself is to use the trick that John
D. Rockefeller, whose grandfather was an alcoholic, used to protect
himself from becoming one.  He once told a Sunday school class

  Boys, do you know why I never became a drunkard?  Because I never
  took the first drink.

Do you want to sell your company right now?  Not eventually, right
now.  If not, just don't take the first meeting.  They won't be
offended.  And you in turn will be guaranteed to be spared one of
the worst experiences that can happen to a startup.If you do want to sell, there's another set of 
techniques
 for doing
that.  But the biggest mistake founders make in dealing with corp
dev is not doing a bad job of talking to them when they're ready
to, but talking to them before they are.  So if you remember only
the title of this essay, you already know most of what you need to
know about M&A in the first year.Notes[1]
I'm not saying you should never sell.  I'm saying you should
be clear in your own mind about whether you want to sell or not,
and not be led by manipulation or wishful thinking into trying to
sell earlier than you otherwise would have.[2]
In a startup, as in most competitive sports, the task at hand
almost does this for you; you're too busy to feel tired.  But when
you lose that protection, e.g. at the final whistle, the fatigue
hits you like a wave.  To talk to corp dev is to let yourself feel
it mid-game.[3]
To be fair, the apparent misdeeds of corp dev people are magnified
by the fact that they function as the face of a large organization
that often doesn't know its own mind.  Acquirers can be surprisingly
indecisive about acquisitions, and their flakiness is indistinguishable
from dishonesty by the time it filters down to you.Thanks to Marc Andreessen, Jessica Livingston, Geoff
Ralston, and Qasar Younis for reading drafts of this.September 2007In high school I decided I was going to study philosophy in college.
I had several motives, some more honorable than others.  One of the
less honorable was to shock people.  College was regarded as job
training where I grew up, so studying philosophy seemed an impressively
impractical thing to do.  Sort of like slashing holes in your clothes
or putting a safety pin through your ear, which were other forms
of impressive impracticality then just coming into fashion.But I had some more honest motives as well.  I thought studying
philosophy would be a shortcut straight to wisdom.  All the people
majoring in other things would just end up with a bunch of domain
knowledge.  I would be learning what was really what.I'd tried to read a few philosophy books.  Not recent ones; you
wouldn't find those in our high school library.  But I tried to
read Plato and Aristotle.  I doubt I believed I understood them,
but they sounded like they were talking about something important.
I assumed I'd learn what in college.The summer before senior year I took some college classes.  I learned
a lot in the calculus class, but I didn't learn much in Philosophy
101.  And yet my plan to study philosophy remained intact.  It was
my fault I hadn't learned anything.  I hadn't read the books we
were assigned carefully enough.  I'd give Berkeley's Principles
of Human Knowledge another shot in college.  Anything so admired
and so difficult to read must have something in it, if one could
only figure out what.Twenty-six years later, I still don't understand Berkeley.  I have
a nice edition of his collected works.  Will I ever read it?  Seems
unlikely.The difference between then and now is that now I understand why
Berkeley is probably not worth trying to understand.  I think I see
now what went wrong with philosophy, and how we might fix it.WordsI did end up being a philosophy major for most of college.  It
didn't work out as I'd hoped.  I didn't learn any magical truths
compared to which everything else was mere domain knowledge.  But
I do at least know now why I didn't.  Philosophy doesn't really
have a subject matter in the way math or history or most other
university subjects do.  There is no core of knowledge one must
master.  The closest you come to that is a knowledge of what various
individual philosophers have said about different topics over the
years.  Few were sufficiently correct that people have forgotten
who discovered what they discovered.Formal logic has some subject matter. I took several classes in
logic.  I don't know if I learned anything from them.
[1]
It does seem to me very important to be able to flip ideas around in
one's head: to see when two ideas don't fully cover the space of
possibilities, or when one idea is the same as another but with a
couple things changed.  But did studying logic teach me the importance
of thinking this way, or make me any better at it?  I don't know.There are things I know I learned from studying philosophy.  The
most dramatic I learned immediately, in the first semester of
freshman year, in a class taught by Sydney Shoemaker.  I learned
that I don't exist.  I am (and you are) a collection of cells that
lurches around driven by various forces, and calls itself I.  But
there's no central, indivisible thing that your identity goes with.
You could conceivably lose half your brain and live.  Which means
your brain could conceivably be split into two halves and each
transplanted into different bodies.  Imagine waking up after such
an operation.  You have to imagine being two people.The real lesson here is that the concepts we use in everyday life
are fuzzy, and break down if pushed too hard.  Even a concept as
dear to us as I.  It took me a while to grasp this, but when I
did it was fairly sudden, like someone in the nineteenth century
grasping evolution and realizing the story of creation they'd been
told as a child was all wrong. 
[2]
Outside of math there's a limit
to how far you can push words; in fact, it would not be a bad
definition of math to call it the study of terms that have precise
meanings.  Everyday words are inherently imprecise.  They work well
enough in everyday life that you don't notice.  Words seem to work,
just as Newtonian physics seems to.  But you can always make them
break if you push them far enough.I would say that this has been, unfortunately for philosophy, the
central fact of philosophy.  Most philosophical debates are not
merely afflicted by but driven by confusions over words.  Do we
have free will?  Depends what you mean by "free." Do abstract ideas
exist?  Depends what you mean by "exist."Wittgenstein is popularly credited with the idea that most philosophical
controversies are due to confusions over language.  I'm not sure
how much credit to give him.  I suspect a lot of people realized
this, but reacted simply by not studying philosophy, rather than
becoming philosophy professors.How did things get this way?  Can something people have spent
thousands of years studying really be a waste of time?  Those are
interesting questions.  In fact, some of the most interesting
questions you can ask about philosophy.  The most valuable way to
approach the current philosophical tradition may be neither to get
lost in pointless speculations like Berkeley, nor to shut them down
like Wittgenstein, but to study it as an example of reason gone
wrong.HistoryWestern philosophy really begins with Socrates, Plato, and Aristotle.
What we know of their predecessors comes from fragments and references
in later works; their doctrines could be described as speculative
cosmology that occasionally strays into analysis.  Presumably they
were driven by whatever makes people in every other society invent
cosmologies.
[3]With Socrates, Plato, and particularly Aristotle, this tradition
turned a corner.  There started to be a lot more analysis.  I suspect
Plato and Aristotle were encouraged in this by progress in math.
Mathematicians had by then shown that you could figure things out
in a much more conclusive way than by making up fine sounding stories
about them.  
[4]People talk so much about abstractions now that we don't realize
what a leap it must have been when they first started to.  It was
presumably many thousands of years between when people first started
describing things as hot or cold and when someone asked "what is
heat?"  No doubt it was a very gradual process.  We don't know if
Plato or Aristotle were the first to ask any of the questions they
did.  But their works are the oldest we have that do this on a large
scale, and there is a freshness (not to say naivete) about them
that suggests some of the questions they asked were new to them,
at least.Aristotle in particular reminds me of the phenomenon that happens
when people discover something new, and are so excited by it that
they race through a huge percentage of the newly discovered territory
in one lifetime.  If so, that's evidence of how new this kind of
thinking was. 
[5]This is all to explain how Plato and Aristotle can be very impressive
and yet naive and mistaken.  It was impressive even to ask the
questions they did.  That doesn't mean they always came up with
good answers.  It's not considered insulting to say that ancient
Greek mathematicians were naive in some respects, or at least lacked
some concepts that would have made their lives easier.  So I hope
people will not be too offended if I propose that ancient philosophers
were similarly naive.  In particular, they don't seem to have fully
grasped what I earlier called the central fact of philosophy: that
words break if you push them too far."Much to the surprise of the builders of the first digital computers,"
Rod Brooks wrote, "programs written for them usually did not work."
[6]
Something similar happened when people first started trying
to talk about abstractions.  Much to their surprise, they didn't
arrive at answers they agreed upon.  In fact, they rarely seemed
to arrive at answers at all.They were in effect arguing about artifacts induced by sampling at
too low a resolution.The proof of how useless some of their answers turned out to be is
how little effect they have.  No one after reading Aristotle's
Metaphysics does anything differently as a result.
[7]Surely I'm not claiming that ideas have to have practical applications
to be interesting?  No, they may not have to.  Hardy's boast that
number theory had no use whatsoever wouldn't disqualify it.  But
he turned out to be mistaken.  In fact, it's suspiciously hard to
find a field of math that truly has no practical use.  And Aristotle's
explanation of the ultimate goal of philosophy in Book A of the
Metaphysics implies that philosophy should be useful too.Theoretical KnowledgeAristotle's goal was to find the most general of general principles.
The examples he gives are convincing: an ordinary worker builds
things a certain way out of habit; a master craftsman can do more
because he grasps the underlying principles.  The trend is clear:
the more general the knowledge, the more admirable it is.  But then
he makes a mistake—possibly the most important mistake in the
history of philosophy.  He has noticed that theoretical knowledge
is often acquired for its own sake, out of curiosity, rather than
for any practical need.  So he proposes there are two kinds of
theoretical knowledge: some that's useful in practical matters and
some that isn't.  Since people interested in the latter are interested
in it for its own sake, it must be more noble.  So he sets as his
goal in the Metaphysics the exploration of knowledge that has no
practical use.  Which means no alarms go off when he takes on grand
but vaguely understood questions and ends up getting lost in a sea
of words.His mistake was to confuse motive and result.  Certainly, people
who want a deep understanding of something are often driven by
curiosity rather than any practical need.  But that doesn't mean
what they end up learning is useless.  It's very valuable in practice
to have a deep understanding of what you're doing; even if you're
never called on to solve advanced problems, you can see shortcuts
in the solution of simple ones, and your knowledge won't break down
in edge cases, as it would if you were relying on formulas you
didn't understand.  Knowledge is power.  That's what makes theoretical
knowledge prestigious.  It's also what causes smart people to be
curious about certain things and not others; our DNA is not so
disinterested as we might think.So while ideas don't have to have immediate practical applications
to be interesting, the kinds of things we find interesting will
surprisingly often turn out to have practical applications.The reason Aristotle didn't get anywhere in the Metaphysics was
partly that he set off with contradictory aims: to explore the most
abstract ideas, guided by the assumption that they were useless.
He was like an explorer looking for a territory to the north of
him, starting with the assumption that it was located to the south.And since his work became the map used by generations of future
explorers, he sent them off in the wrong direction as well. 
[8]
Perhaps worst of all, he protected them from both the criticism of
outsiders and the promptings of their own inner compass by establishing
the principle that the most noble sort of theoretical knowledge had
to be useless.The Metaphysics is mostly a failed experiment.  A few ideas from
it turned out to be worth keeping; the bulk of it has had no effect
at all.  The Metaphysics is among the least read of all famous
books.  It's not hard to understand the way Newton's Principia
is, but the way a garbled message is.Arguably it's an interesting failed experiment.  But unfortunately
that was not the conclusion Aristotle's successors derived from
works like the Metaphysics. 
[9]
Soon after, the western world
fell on intellectual hard times.  Instead of version 1s to be
superseded, the works of Plato and Aristotle became revered texts
to be mastered and discussed.  And so things remained for a shockingly
long time.  It was not till around 1600 (in Europe, where the center
of gravity had shifted by then) that one found people confident
enough to treat Aristotle's work as a catalog of mistakes.  And
even then they rarely said so outright.If it seems surprising that the gap was so long, consider how little
progress there was in math between Hellenistic times and the
Renaissance.In the intervening years an unfortunate idea took hold:  that it
was not only acceptable to produce works like the Metaphysics,
but that it was a particularly prestigious line of work, done by a
class of people called philosophers.  No one thought to go back and
debug Aristotle's motivating argument.  And so instead of correcting
the problem Aristotle discovered by falling into it—that you can
easily get lost if you talk too loosely about very abstract ideas—they 
continued to fall into it.The SingularityCuriously, however, the works they produced continued to attract
new readers.  Traditional philosophy occupies a kind of singularity
in this respect.  If you write in an unclear way about big ideas,
you produce something that seems tantalizingly attractive to
inexperienced but intellectually ambitious students.  Till one knows
better, it's hard to distinguish something that's hard to understand
because the writer was unclear in his own mind from something like
a mathematical proof that's hard to understand because the ideas
it represents are hard to understand.  To someone who hasn't learned
the difference, traditional philosophy seems extremely attractive:
as hard (and therefore impressive) as math, yet broader in scope.
That was what lured me in as a high school student.This singularity is even more singular in having its own defense
built in.  When things are hard to understand, people who suspect
they're nonsense generally keep quiet.  There's no way to prove a
text is meaningless.  The closest you can get is to show that the
official judges of some class of texts can't distinguish them from
placebos. 
[10]And so instead of denouncing philosophy, most people who suspected
it was a waste of time just studied other things.  That alone is
fairly damning evidence, considering philosophy's claims.  It's
supposed to be about the ultimate truths. Surely all smart people
would be interested in it, if it delivered on that promise.Because philosophy's flaws turned away the sort of people who might
have corrected them, they tended to be self-perpetuating.  Bertrand
Russell wrote in a letter in 1912:

  Hitherto the people attracted to philosophy have been mostly those
  who loved the big generalizations, which were all wrong, so that
  few people with exact minds have taken up the subject.
[11]

His response was to launch Wittgenstein at it, with dramatic results.I think Wittgenstein deserves to be famous not for the discovery
that most previous philosophy was a waste of time, which judging
from the circumstantial evidence must have been made by every smart
person who studied a little philosophy and declined to pursue it
further, but for how he acted in response.
[12]
Instead of quietly
switching to another field, he made a fuss, from inside.  He was
Gorbachev.The field of philosophy is still shaken from the fright Wittgenstein
gave it. 
[13]
Later in life he spent a lot of time talking about
how words worked.  Since that seems to be allowed, that's what a
lot of philosophers do now.  Meanwhile, sensing a vacuum in the
metaphysical speculation department, the people who used to do
literary criticism have been edging Kantward, under new names like
"literary theory," "critical theory," and when they're feeling
ambitious, plain "theory."  The writing is the familiar word salad:

  Gender is not like some of the other grammatical modes which
  express precisely a mode of conception without any reality that
  corresponds to the conceptual mode, and consequently do not express
  precisely something in reality by which the intellect could be
  moved to conceive a thing the way it does, even where that motive
  is not something in the thing as such.
  [14]

The singularity I've described is not going away.  There's a market
for writing that sounds impressive and can't be disproven. There
will always be both supply and demand.  So if one group abandons
this territory, there will always be others ready to occupy it.A ProposalWe may be able to do better.  Here's an intriguing possibility.
Perhaps we should do what Aristotle meant to do, instead of what
he did.  The goal he announces in the Metaphysics seems one worth
pursuing: to discover the most general truths.  That sounds good.
But instead of trying to discover them because they're useless,
let's try to discover them because they're useful.I propose we try again, but that we use that heretofore despised
criterion, applicability, as a guide to keep us from wondering
off into a swamp of abstractions.  Instead of trying to answer the
question:

  What are the most general truths?

let's try to answer the question

  Of all the useful things we can say, which are the most general?

The test of utility I propose is whether we cause people who read
what we've written to do anything differently afterward.  Knowing
we have to give definite (if implicit) advice will keep us from
straying beyond the resolution of the words we're using.The goal is the same as Aristotle's; we just approach it from a
different direction.As an example of a useful, general idea, consider that of the
controlled experiment.  There's an idea that has turned out to be
widely applicable.  Some might say it's part of science, but it's
not part of any specific science; it's literally meta-physics (in
our sense of "meta").   The idea of evolution is another. It turns
out to have quite broad applications—for example, in genetic
algorithms and even product design.  Frankfurt's distinction between
lying and bullshitting seems a promising recent example.
[15]These seem to me what philosophy should look like: quite general
observations that would cause someone who understood them to do
something differently.Such observations will necessarily be about things that are imprecisely
defined.  Once you start using words with precise meanings, you're
doing math.  So starting from utility won't entirely solve the
problem I described above—it won't flush out the metaphysical
singularity.  But it should help.  It gives people with good
intentions a new roadmap into abstraction.  And they may thereby
produce things that make the writing of the people with bad intentions
look bad by comparison.One drawback of this approach is that it won't produce the sort of
writing that gets you tenure.  And not just because it's not currently
the fashion.  In order to get tenure in any field you must not
arrive at conclusions that members of tenure committees can disagree
with.  In practice there are two kinds of solutions to this problem.
In math and the sciences, you can prove what you're saying, or at
any rate adjust your conclusions so you're not claiming anything
false ("6 of 8 subjects had lower blood pressure after the treatment").
In the humanities you can either avoid drawing any definite conclusions
(e.g. conclude that an issue is a complex one), or draw conclusions
so narrow that no one cares enough to disagree with you.The kind of philosophy I'm advocating won't be able to take either
of these routes.  At best you'll be able to achieve the essayist's
standard of proof, not the mathematician's or the experimentalist's.
And yet you won't be able to meet the usefulness test without
implying definite and fairly broadly applicable conclusions.  Worse
still, the usefulness test will tend to produce results that annoy
people: there's no use in telling people things they already believe,
and people are often upset to be told things they don't.Here's the exciting thing, though.  Anyone can do this.  Getting
to general plus useful by starting with useful and cranking up the
generality may be unsuitable for junior professors trying to get
tenure, but it's better for everyone else, including professors who
already have it.  This side of the mountain is a nice gradual slope.
You can start by writing things that are useful but very specific,
and then gradually make them more general.  Joe's has good burritos.
What makes a good burrito?  What makes good food?  What makes
anything good?  You can take as long as you want.  You don't have
to get all the way to the top of the mountain.  You don't have to
tell anyone you're doing philosophy.If it seems like a daunting task to do philosophy, here's an
encouraging thought.  The field is a lot younger than it seems.
Though the first philosophers in the western tradition lived about
2500 years ago, it would be misleading to say the field is 2500
years old, because for most of that time the leading practitioners
weren't doing much more than writing commentaries on Plato or
Aristotle while watching over their shoulders for the next invading
army.  In the times when they weren't, philosophy was hopelessly
intermingled with religion.  It didn't shake itself free till a
couple hundred years ago, and even then was afflicted by the
structural problems I've described above.  If I say this, some will
say it's a ridiculously overbroad and uncharitable generalization,
and others will say it's old news, but here goes: judging from their
works, most philosophers up to the present have been wasting their
time.  So in a sense the field is still at the first step. 
[16]That sounds a preposterous claim to make.  It won't seem so
preposterous in 10,000 years.  Civilization always seems old, because
it's always the oldest it's ever been.  The only way to say whether
something is really old or not is by looking at structural evidence,
and structurally philosophy is young; it's still reeling from the
unexpected breakdown of words.Philosophy is as young now as math was in 1500.  There is a lot
more to discover.Notes
[1]
In practice formal logic is not much use, because despite
some progress in the last 150 years we're still only able to formalize
a small percentage of statements.  We may never do that much better,
for the same reason 1980s-style "knowledge representation" could
never have worked; many statements may have no representation more
concise than a huge, analog brain state.[2]
It was harder for Darwin's contemporaries to grasp this than
we can easily imagine.  The story of creation in the Bible is not
just a Judeo-Christian concept; it's roughly what everyone must
have believed since before people were people.  The hard part of
grasping evolution was to realize that species weren't, as they
seem to be, unchanging, but had instead evolved from different,
simpler organisms over unimaginably long periods of time.Now we don't have to make that leap.  No one in an industrialized
country encounters the idea of evolution for the first time as an
adult.  Everyone's taught about it as a child, either as truth or
heresy.[3]
Greek philosophers before Plato wrote in verse.  This must
have affected what they said.  If you try to write about the nature
of the world in verse, it inevitably turns into incantation.  Prose
lets you be more precise, and more tentative.[4]
Philosophy is like math's
ne'er-do-well brother.  It was born when Plato and Aristotle looked
at the works of their predecessors and said in effect "why can't
you be more like your brother?"  Russell was still saying the same
thing 2300 years later.Math is the precise half of the most abstract ideas, and philosophy
the imprecise half.  It's probably inevitable that philosophy will
suffer by comparison, because there's no lower bound to its precision.
Bad math is merely boring, whereas bad philosophy is nonsense.  And
yet there are some good ideas in the imprecise half.[5]
Aristotle's best work was in logic and zoology, both of which
he can  be said to have invented.  But the most dramatic departure
from his predecessors was a new, much more analytical style of
thinking.  He was arguably the first scientist.[6]
Brooks, Rodney, Programming in Common Lisp, Wiley, 1985, p.
94.[7]
Some would say we depend on Aristotle more than we realize,
because his ideas were one of the ingredients in our common culture.
Certainly a lot of the words we use have a connection with Aristotle,
but it seems a bit much to suggest that we wouldn't have the concept
of the essence of something or the distinction between matter and
form if Aristotle hadn't written about them.One way to see how much we really depend on Aristotle would be to
diff European culture with Chinese: what ideas did European culture
have in 1800 that Chinese culture didn't, in virtue of Aristotle's
contribution?[8]
The meaning of the word "philosophy" has changed over time.
In ancient times it covered a broad range of topics, comparable in
scope to our "scholarship" (though without the methodological
implications).  Even as late as Newton's time it included what we
now call "science."  But core of the subject today is still what
seemed to Aristotle the core: the attempt to discover the most
general truths.Aristotle didn't call this "metaphysics."  That name got assigned
to it because the books we now call the Metaphysics came after
(meta = after) the Physics in the standard edition of Aristotle's
works compiled by Andronicus of Rhodes three centuries later.  What
we call "metaphysics" Aristotle called "first philosophy."[9]
Some of Aristotle's immediate successors may have realized
this, but it's hard to say because most of their works are lost.[10]
Sokal, Alan, "Transgressing the Boundaries: Toward a Transformative
Hermeneutics of Quantum Gravity," Social Text 46/47, pp. 217-252.Abstract-sounding nonsense seems to be most attractive when it's
aligned with some axe the audience already has to grind.  If this
is so we should find it's most popular with groups that are (or
feel) weak.  The powerful don't need its reassurance.[11]
Letter to Ottoline Morrell, December 1912.  Quoted in:Monk, Ray, Ludwig Wittgenstein: The Duty of Genius, Penguin, 1991,
p. 75.[12]
A preliminary result, that all metaphysics between Aristotle
and 1783 had been a waste of time, is due to I. Kant.[13]
Wittgenstein asserted a sort of mastery to which the inhabitants
of early 20th century Cambridge seem to have been peculiarly
vulnerable—perhaps partly because so many had been raised religious
and then stopped believing, so had a vacant space in their heads
for someone to tell them what to do (others chose Marx or Cardinal
Newman), and partly because a quiet, earnest place like Cambridge
in that era had no natural immunity to messianic figures, just as
European politics then had no natural immunity to dictators.[14]
This is actually from the Ordinatio of Duns Scotus (ca.
1300), with "number" replaced by "gender."  Plus ca change.Wolter, Allan (trans), Duns Scotus: Philosophical Writings, Nelson,
1963, p. 92.[15]
Frankfurt, Harry, On Bullshit,  Princeton University Press,
2005.[16]
Some introductions to philosophy now take the line that
philosophy is worth studying as a process rather than for any
particular truths you'll learn.  The philosophers whose works they
cover would be rolling in their graves at that.  They hoped they
were doing more than serving as examples of how to argue: they hoped
they were getting results.  Most were wrong, but it doesn't seem
an impossible hope.This argument seems to me like someone in 1500 looking at the lack
of results achieved by alchemy and saying its value was as a process.
No, they were going about it wrong.  It turns out it is possible
to transmute lead into gold (though not economically at current
energy prices), but the route to that knowledge was to
backtrack and try another approach.Thanks to Trevor Blackwell, Paul Buchheit, Jessica Livingston, 
Robert Morris, Mark Nitzberg, and Peter Norvig for reading drafts of this.September 2017The most valuable insights are both general and surprising. 
F = ma for example. But general and surprising is a hard
combination to achieve. That territory tends to be picked
clean, precisely because those insights are so valuable.Ordinarily, the best that people can do is one without the
other: either surprising without being general (e.g.
gossip), or general without being surprising (e.g.
platitudes).Where things get interesting is the moderately valuable
insights.  You get those from small additions of whichever
quality was missing.  The more common case is a small
addition of generality: a piece of gossip that's more than
just gossip, because it teaches something interesting about
the world. But another less common approach is to focus on
the most general ideas and see if you can find something new
to say about them. Because these start out so general, you
only need a small delta of novelty to produce a useful
insight.A small delta of novelty is all you'll be able to get most
of the time. Which means if you take this route, your ideas
will seem a lot like ones that already exist. Sometimes
you'll find you've merely rediscovered an idea that did
already exist.  But don't be discouraged.  Remember the huge
multiplier that kicks in when you do manage to think of
something even a little new.Corollary: the more general the ideas you're talking about,
the less you should worry about repeating yourself.  If you
write enough, it's inevitable you will.  Your brain is much
the same from year to year and so are the stimuli that hit
it. I feel slightly bad when I find I've said something
close to what I've said before, as if I were plagiarizing
myself. But rationally one shouldn't.  You won't say
something exactly the same way the second time, and that
variation increases the chance you'll get that tiny but
critical delta of novelty.And of course, ideas beget ideas.  (That sounds 
familiar.)
An idea with a small amount of novelty could lead to one
with more. But only if you keep going. So it's doubly
important not to let yourself be discouraged by people who
say there's not much new about something you've discovered.
"Not much new" is a real achievement when you're talking
about the most general ideas. It's not true that there's nothing new under the sun.  There
are some domains where there's almost nothing new.  But
there's a big difference between nothing and almost nothing,
when it's multiplied by the area under the sun.
Thanks to Sam Altman, Patrick Collison, and Jessica
Livingston for reading drafts of this.January 2003(This article is derived from a keynote talk at the fall 2002 meeting
of NEPLS.)Visitors to this country are often surprised to find that
Americans like to begin a conversation by asking "what do you do?"
I've never liked this question.  I've rarely had a
neat answer to it.  But I think I have finally solved the problem.
Now, when someone asks me what I do, I look them straight
in the eye and say "I'm designing a 
new dialect of Lisp."   
I recommend this answer to anyone who doesn't like being asked what
they do.  The conversation will turn immediately to other topics.I don't consider myself to be doing research on programming languages.
I'm just designing one, in the same way that someone might design
a building or a chair or a new typeface.
I'm not trying to discover anything new.  I just want
to make a language that will be good to program in.  In some ways,
this assumption makes life a lot easier.The difference between design and research seems to be a question
of new versus good.  Design doesn't have to be new, but it has to  
be good.  Research doesn't have to be good, but it has to be new.
I think these two paths converge at the top: the best design
surpasses its predecessors by using new ideas, and the best research
solves problems that are not only new, but actually worth solving.
So ultimately we're aiming for the same destination, just approaching
it from different directions.What I'm going to talk about today is what your target looks like
from the back.  What do you do differently when you treat
programming languages as a design problem instead of a research topic?The biggest difference is that you focus more on the user.
Design begins by asking, who is this
for and what do they need from it?  A good architect,
for example, does not begin by creating a design that he then
imposes on the users, but by studying the intended users and figuring
out what they need.Notice I said "what they need," not "what they want."  I don't mean
to give the impression that working as a designer means working as 
a sort of short-order cook, making whatever the client tells you
to.  This varies from field to field in the arts, but
I don't think there is any field in which the best work is done by
the people who just make exactly what the customers tell them to.The customer is always right in
the sense that the measure of good design is how well it works
for the user.  If you make a novel that bores everyone, or a chair
that's horribly uncomfortable to sit in, then you've done a bad
job, period.  It's no defense to say that the novel or the chair  
is designed according to the most advanced theoretical principles.And yet, making what works for the user doesn't mean simply making
what the user tells you to.  Users don't know what all the choices
are, and are often mistaken about what they really want.The answer to the paradox, I think, is that you have to design
for the user, but you have to design what the user needs, not simply  
what he says he wants.
It's much like being a doctor.  You can't just treat a patient's
symptoms.  When a patient tells you his symptoms, you have to figure
out what's actually wrong with him, and treat that.This focus on the user is a kind of axiom from which most of the
practice of good design can be derived, and around which most design
issues center.If good design must do what the user needs, who is the user?  When
I say that design must be for users, I don't mean to imply that good 
design aims at some kind of  
lowest common denominator.  You can pick any group of users you
want.  If you're designing a tool, for example, you can design it
for anyone from beginners to experts, and what's good design
for one group might be bad for another.  The point
is, you have to pick some group of users.  I don't think you can
even talk about good or bad design except with
reference to some intended user.You're most likely to get good design if the intended users include
the designer himself.  When you design something
for a group that doesn't include you, it tends to be for people
you consider to be less sophisticated than you, not more sophisticated.That's a problem, because looking down on the user, however benevolently,
seems inevitably to corrupt the designer.
I suspect that very few housing
projects in the US were designed by architects who expected to live
in them.   You can see the same thing
in programming languages.  C, Lisp, and Smalltalk were created for
their own designers to use.  Cobol, Ada, and Java, were created   
for other people to use.If you think you're designing something for idiots, the odds are
that you're not designing something good, even for idiots.
Even if you're designing something for the most sophisticated
users, though, you're still designing for humans.  It's different 
in research.  In math you
don't choose abstractions because they're
easy for humans to understand; you choose whichever make the
proof shorter.  I think this is true for the sciences generally.
Scientific ideas are not meant to be ergonomic.Over in the arts, things are very different.  Design is
all about people.  The human body is a strange
thing, but when you're designing a chair,
that's what you're designing for, and there's no way around it.
All the arts have to pander to the interests and limitations
of humans.   In painting, for example, all other things being
equal a painting with people in it will be more interesting than
one without.  It is not merely an accident of history that
the great paintings of the Renaissance are all full of people.
If they hadn't been, painting as a medium wouldn't have the prestige
that it does.Like it or not, programming languages are also for people,
and I suspect the human brain is just as lumpy and idiosyncratic
as the human body.  Some ideas are easy for people to grasp
and some aren't.  For example, we seem to have a very limited
capacity for dealing with detail.  It's this fact that makes
programing languages a good idea in the first place; if we
could handle the detail, we could just program in machine
language.Remember, too, that languages are not
primarily a form for finished programs, but something that
programs have to be developed in.  Anyone in the arts could
tell you that you might want different mediums for the
two situations.  Marble, for example, is a nice, durable
medium for finished ideas, but a hopelessly inflexible one
for developing new ideas.A program, like a proof,
is a pruned version of a tree that in the past has had
false starts branching off all over it.  So the test of
a language is not simply how clean the finished program looks
in it, but how clean the path to the finished program was.
A design choice that gives you elegant finished programs
may not give you an elegant design process.  For example, 
I've written a few macro-defining macros full of nested
backquotes that look now like little gems, but writing them
took hours of the ugliest trial and error, and frankly, I'm still
not entirely sure they're correct.We often act as if the test of a language were how good
finished programs look in it.
It seems so convincing when you see the same program
written in two languages, and one version is much shorter.
When you approach the problem from the direction of the
arts, you're less likely to depend on this sort of
test.  You don't want to end up with a programming
language like marble.For example, it is a huge win in developing software to
have an interactive toplevel, what in Lisp is called a
read-eval-print loop.  And when you have one this has
real effects on the design of the language.  It would not
work well for a language where you have to declare
variables before using them, for example.  When you're
just typing expressions into the toplevel, you want to be 
able to set x to some value and then start doing things
to x.  You don't want to have to declare the type of x
first.  You may dispute either of the premises, but if
a language has to have a toplevel to be convenient, and
mandatory type declarations are incompatible with a
toplevel, then no language that makes type declarations  
mandatory could be convenient to program in.In practice, to get good design you have to get close, and stay
close, to your users.  You have to calibrate your ideas on actual
users constantly, especially in the beginning.  One of the reasons
Jane Austen's novels are so good is that she read them out loud to
her family.  That's why she never sinks into self-indulgently arty
descriptions of landscapes,
or pretentious philosophizing.  (The philosophy's there, but it's
woven into the story instead of being pasted onto it like a label.)
If you open an average "literary" novel and imagine reading it out loud
to your friends as something you'd written, you'll feel all too
keenly what an imposition that kind of thing is upon the reader.In the software world, this idea is known as Worse is Better.
Actually, there are several ideas mixed together in the concept of
Worse is Better, which is why people are still arguing about
whether worse
is actually better or not.  But one of the main ideas in that
mix is that if you're building something new, you should get a
prototype in front of users as soon as possible.The alternative approach might be called the Hail Mary strategy.
Instead of getting a prototype out quickly and gradually refining
it, you try to create the complete, finished, product in one long
touchdown pass.  As far as I know, this is a
recipe for disaster.  Countless startups destroyed themselves this
way during the Internet bubble.  I've never heard of a case
where it worked.What people outside the software world may not realize is that
Worse is Better is found throughout the arts.
In drawing, for example, the idea was discovered during the
Renaissance.  Now almost every drawing teacher will tell you that
the right way to get an accurate drawing is not to
work your way slowly around the contour of an object, because errors will
accumulate and you'll find at the end that the lines don't meet.
Instead you should draw a few quick lines in roughly the right place,
and then gradually refine this initial sketch.In most fields, prototypes
have traditionally been made out of different materials.
Typefaces to be cut in metal were initially designed  
with a brush on paper.  Statues to be cast in bronze   
were modelled in wax.  Patterns to be embroidered on tapestries
were drawn on paper with ink wash.  Buildings to be
constructed from stone were tested on a smaller scale in wood.What made oil paint so exciting, when it
first became popular in the fifteenth century, was that you
could actually make the finished work from the prototype.
You could make a preliminary drawing if you wanted to, but you
weren't held to it; you could work out all the details, and
even make major changes, as you finished the painting.You can do this in software too.  A prototype doesn't have to
be just a model; you can refine it into the finished product.
I think you should always do this when you can.  It lets you
take advantage of new insights you have along the way.  But
perhaps even more important, it's good for morale.Morale is key in design.  I'm surprised people
don't talk more about it.  One of my first
drawing teachers told me: if you're bored when you're
drawing something, the drawing will look boring.
For example, suppose you have to draw a building, and you
decide to draw each brick individually.  You can do this
if you want, but if you get bored halfway through and start
making the bricks mechanically instead of observing each one,   
the drawing will look worse than if you had merely suggested
the bricks.Building something by gradually refining a prototype is good
for morale because it keeps you engaged.  In software, my  
rule is: always have working code.  If you're writing
something that you'll be able to test in an hour, then you
have the prospect of an immediate reward to motivate you.
The same is true in the arts, and particularly in oil painting.
Most painters start with a blurry sketch and gradually
refine it.
If you work this way, then in principle
you never have to end the day with something that actually
looks unfinished.  Indeed, there is even a saying among
painters: "A painting is never finished, you just stop
working on it."  This idea will be familiar to anyone who
has worked on software.Morale is another reason that it's hard to design something
for an unsophisticated user.   It's hard to stay interested in
something you don't like yourself.  To make something  
good, you have to be thinking, "wow, this is really great,"
not "what a piece of shit; those fools will love it."Design means making things for humans.  But it's not just the
user who's human.  The designer is human too.Notice all this time I've been talking about "the designer."
Design usually has to be under the control of a single person to
be any good.   And yet it seems to be possible for several people
to collaborate on a research project.  This seems to
me one of the most interesting differences between research and
design.There have been famous instances of collaboration in the arts,
but most of them seem to have been cases of molecular bonding rather
than nuclear fusion.  In an opera it's common for one person to
write the libretto and another to write the music.   And during the Renaissance, 
journeymen from northern
Europe were often employed to do the landscapes in the
backgrounds of Italian paintings.  But these aren't true collaborations.
They're more like examples of Robert Frost's
"good fences make good neighbors."  You can stick instances
of good design together, but within each individual project,
one person has to be in control.I'm not saying that good design requires that one person think
of everything.  There's nothing more valuable than the advice
of someone whose judgement you trust.  But after the talking is
done, the decision about what to do has to rest with one person.Why is it that research can be done by collaborators and  """

In [8]:
TEMPLATE = "{Context} Question: {Question} Answer:"
prompt = TEMPLATE.format(Context=Long_Context_30K, Question="What is the best thing to do in San Francisco ?")
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    adapter_output = model.generate(**input_ids, use_cache=True, min_new_tokens=16, max_new_tokens=64)

    with model.disable_adapter():
        base_output = model.generate(**input_ids, use_cache=True, max_new_tokens=64)
# response = tokenizer.decode()
base_response = tokenizer.decode(base_output[0][input_ids.input_ids.shape[-1]:], skip_special_tokens=True).strip()
adapter_response = tokenizer.decode(adapter_output[0][input_ids.input_ids.shape[-1]:], skip_special_tokens=True).strip()

print("--" * 20 + "base_response" + "--" * 20)
print(base_response)
print("--" * 20 + "adapter_response" + "--" * 20)
print(adapter_response)

/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


----------------------------------------base_response----------------------------------------
10
I's
I's 10.
I'm 10.
I'm a lot of the 10.
I'm 10.0.0.0.0.0.0.0.0.0.0.0
----------------------------------------adapter_response----------------------------------------
It's a recipe for disaster.### To create the complete, finished, product in one touchdown pass.### It's a Hail Mary strategy.### It's a strategy for creating the prototype, not a prototype, and it's a strategy for creating the finished,
